In [6]:
import nltk

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

## Let's get the data and clean it up a bit

In [8]:
all_mbti = pd.read_csv('data/Essay_data.csv')

# List of mbti types 
type_labels = ['ISTJ', 'ISFJ', 'INFJ', 'INTJ', 
               'ISTP', 'ISFP', 'INFP', 'INTP', 
               'ESTP', 'ESFP', 'ENFP', 'ENTP', 
               'ESTJ', 'ESFJ', 'ENFJ', 'ENTJ']

In [149]:
all_mbti.tail()

,I/E,N/S,T/F,J/P,tokens,lemma,type
62,E,?,?,?,[nan],[nan],E???
63,I,N,T,P,"[day, looked, familiar, faces., started, looki...","[on, my, first, day, i, first, looked, for, fa...",INTP
64,E,N,F,P,"[enjoyed, time, edsa, far., love, meeting, new...","[i, have, enjoyed, my, time, at, edsa, so, far...",ENFP
65,I,S,F,J,"[year, huge, change, ,, involving, aspects, :,...","[this, year, wa, a, huge, one, of, change, for...",ISFJ
66,E,N,F,P,"[experience, edsa, sparked, lot, introspection...","[my, experience, at, edsa, ha, been, one, that...",ENFP


In [119]:
I = (all_mbti['I/E'].value_counts()[1] / (all_mbti['I/E'].value_counts()[1] + all_mbti['I/E'].value_counts()[0])*100)
E = (all_mbti['I/E'].value_counts()[0] / (all_mbti['I/E'].value_counts()[1] + all_mbti['I/E'].value_counts()[0])*100)
N = (all_mbti['N/S'].value_counts()[1] / (all_mbti['N/S'].value_counts()[1] + all_mbti['N/S'].value_counts()[0])*100)
S = (all_mbti['N/S'].value_counts()[0] / (all_mbti['N/S'].value_counts()[1] + all_mbti['N/S'].value_counts()[0])*100)
T = (all_mbti['T/F'].value_counts()[1] / (all_mbti['T/F'].value_counts()[1] + all_mbti['T/F'].value_counts()[0])*100)
F = (all_mbti['T/F'].value_counts()[0] / (all_mbti['T/F'].value_counts()[1] + all_mbti['T/F'].value_counts()[0])*100)
J = (all_mbti['J/P'].value_counts()[1] / (all_mbti['J/P'].value_counts()[1] + all_mbti['J/P'].value_counts()[0])*100)
P = (all_mbti['J/P'].value_counts()[0] / (all_mbti['J/P'].value_counts()[1] + all_mbti['J/P'].value_counts()[0])*100)
                                        
print ("Introversion (I)     -     Extroversion (E):\t")
print(f" {all_mbti['I/E'].value_counts()[1]} | {I:.4}%         -      {E:.4}% |  {all_mbti['I/E'].value_counts()[0]}")
print ("\nIntuition (N)        -     Sensing (S):\t\t")
print(f" {all_mbti['N/S'].value_counts()[1]} | {N:.4}%         -      {S:.4}% |  {all_mbti['N/S'].value_counts()[0]}")
print ("\nThinking (T)         -     Feeling (F):\t\t")
print(f" {all_mbti['T/F'].value_counts()[1]} | {T:.4}%         -      {F:.4}% |   {all_mbti['T/F'].value_counts()[0]}")
print ("\nJudging (J)          -     Perceiving (P):\t\t")
print(f" {all_mbti['J/P'].value_counts()[1]} | {J:.4}%         -      {P:.4}% |   {all_mbti['J/P'].value_counts()[0]}")

Introversion (I)     -     Extroversion (E):	
 20 | 29.85%         -      70.15% |  47

Intuition (N)        -     Sensing (S):		
 20 | 30.77%         -      69.23% |  45

Thinking (T)         -     Feeling (F):		
 30 | 45.45%         -      54.55% |   36

Judging (J)          -     Perceiving (P):		
 29 | 44.62%         -      55.38% |   36


## Removing Noise

In [8]:
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
all_mbti['post'] = all_mbti['post'].replace(to_replace = pattern_url, value = subs_url, regex = True)

### Remove punctuation

In [10]:
# first we make everything lower case to remove some noise from capitalisation
all_mbti['Essay'] = all_mbti['Essay'].str.lower()

In [11]:
import string
# these are the chars that count as punctuation. Let's remove the punctuation
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [13]:
all_mbti['post'] = all_mbti['post'].apply(remove_punctuation)

In [19]:
all_mbti.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
I/E      67 non-null object
N/S      67 non-null object
T/F      67 non-null object
J/P      67 non-null object
Essay    64 non-null object
dtypes: object(5)
memory usage: 2.7+ KB


## [Tokenising](http://www.nltk.org/howto/tokenize.html) 

A tokenizer divides text into a sequence of tokens, which roughly correspond to "words". (see the [Stanford Tokeniser](https://nlp.stanford.edu/software/tokenizer.shtml))  We will use tokenisers to clean up the data, making it ready for analysis.

In [11]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize, TreebankWordTokenizer

In [12]:
all_mbti['Essay'] = all_mbti['Essay'].astype(str)

In [13]:
# we will use the TreeBankWordTokenizer since it is MUCH quicker than the word_tokenise function
tokeniser = TreebankWordTokenizer()
all_mbti['tokens'] = all_mbti['Essay'].apply(tokeniser.tokenize)

## [Lemmatization](https://pythonprogramming.net/lemmatizing-nltk-tutorial/)

In [14]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mohammad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [15]:
def mbti_lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]    

In [16]:
# lemmatize all words in dataframe
all_mbti['lemma'] = all_mbti['tokens'].apply(mbti_lemma, args=(lemmatizer, ))

In [121]:
#for i, t in enumerate(all_mbti.iloc[268702]['tokens']):    
#    print ('{:20s} --> {:10s}'.format(t, all_mbti.iloc[268702]['lemma'][i]))

## [Stop Words](http://johnlaudun.org/20130126-nltk-stopwords/)

Stop words are words which do not contain important significance to be used in Search Queries. Usually these words are filtered out from search queries because they return a vast amount of unnecessary information.  See this [blog post](http://xpo6.com/list-of-english-stop-words/) for more information.

In [20]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stopw = list(ENGLISH_STOP_WORDS)

In [21]:
def remove_stop_words(tokens):    
    return [t for t in tokens if t not in stopw]

Let's leave the stop words in for now so that we can test the following **Hypothesis**:
* Introverts tend to use the word **`I`** more than extroverts
* Conversely, Extroverts tend to favour the word **`you`**

In case you want to run the analysis again without stop words! Be warned, this can take long with the pandas apply function

In [22]:
all_mbti['tokens'] = all_mbti['tokens'].apply(remove_stop_words)

In [23]:
all_mbti.drop('Essay', axis=1, inplace=True)

In [24]:
all_mbti.head()

,I/E,N/S,T/F,J/P,tokens,lemma
0,I,S,T,J,"[4, months, edsa, filled, new, experiences, ;,...","[my, first, 4, month, at, the, edsa, have, bee..."
1,I,N,F,J,"[joined, academy, crossroads, sorts, life., ac...","[i, joined, the, academy, being, at, a, crossr..."
2,E,N,F,J,"[far, experience, positive, definitely, value,...","[so, far, my, experience, ha, been, positive, ..."
3,I,N,F,J,"[fortunate, opportunity, join, academy, year.,...","[i, have, been, very, fortunate, to, have, the..."
4,I,N,T,J,"[looking, got, academy, right, ,, confidently,...","[looking, back, to, when, one, got, to, the, a..."


In [25]:
all_mbti['type'] = all_mbti['I/E'] + all_mbti['N/S'] + all_mbti['T/F'] + all_mbti['J/P']

In [33]:
I = all_mbti[all_mbti['I/E']=='I']['tokens']

In [34]:
I.head()

0    [4, months, edsa, filled, new, experiences, ;,...
1    [joined, academy, crossroads, sorts, life., ac...
3    [fortunate, opportunity, join, academy, year.,...
4    [looking, got, academy, right, ,, confidently,...
5    [overall, experience, academy, far, great, som...
Name: tokens, dtype: object

In [35]:
E = all_mbti[all_mbti['I/E']=='E']['tokens']

In [36]:
E.head()

2     [far, experience, positive, definitely, value,...
13    [journey, exciting., stress, constructive, sen...
16    [coming, academy, best, thing, happened, grate...
19    [experience, far, great, bit, anxious, excited...
25    [experience, explore, data, science, academy, ...
Name: tokens, dtype: object

In [37]:
N = all_mbti[all_mbti['N/S']=='N']['tokens']

In [38]:
S = all_mbti[all_mbti['N/S']=='S']['tokens']

In [39]:
T = all_mbti[all_mbti['T/F']=='T']['tokens']

In [40]:
T.head()

0     [4, months, edsa, filled, new, experiences, ;,...
4     [looking, got, academy, right, ,, confidently,...
7     [attending, explore, data, science, academy, p...
8     [wonderful, place, ,, place, waking, morning, ...
10    [essay, discussing, personal, experience, rega...
Name: tokens, dtype: object

In [41]:
F = all_mbti[all_mbti['T/F']=='F']['tokens']

In [42]:
J = all_mbti[all_mbti['J/P']=='J']['tokens']

In [43]:
P = all_mbti[all_mbti['J/P']=='P']['tokens']

In [44]:
P.head()

5     [overall, experience, academy, far, great, som...
6     [got, academy, ,, felt, like, did, n't, belong...
8     [wonderful, place, ,, place, waking, morning, ...
9     [start, academy, quite, tricking, meeting, new...
12    [experience, academy, enjoyable, far., learnin...
Name: tokens, dtype: object

### [Bag of words](https://www.packtpub.com/mapt/book/application_development/9781849513609/7/ch07lvl1sec73/bag-of-words-feature-extraction)

Text feature extraction is the process of transforming what is essentially a list of words into a feature set that is usable by a classifier. The NLTK classifiers expect dict style feature sets, so we must therefore transform our text into a dict. The Bag of Words model is the simplest method; it constructs a word presence feature set from all the words of an instance.

In [45]:
def bag_of_words_count(words, word_dict={}):
    """ this function takes in a list of words and returns a dictionary 
        with each word as a key, and the value represents the number of 
        times that word appeared"""
    for word in words:
        if word in word_dict.keys():
            word_dict[word] += 1
        else:
            word_dict[word] = 1
    return word_dict

In [48]:
JtXt = "  first 4 months edsa filled many new experiences ; challenges , others joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people everyone met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates within edsa , provided insight gained experience tools like python & sql. also cherish able build network data scientists tomorrow well established industry professionals.. -- -working together lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious con would noise distraction - least say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience would say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes full responsibility part ) members first group noobs sql & python , gather analyse phases really exposed technical deficiencies resulted lot frustration came time delivering tasks. experience took comfort zone important lesson patience us all. second group high level experience , strong team co-ordinator team members took full responsibility tasks. even though work complex comfortable team dynamic. -- -how decisions teams made consistent make decisions -- - team decisions groups made deliberation. decisions easier make person presented clear points opinion was. group dynamic strong decision making felt natural , similiar make decision own. -- -how rational emotional decision making process -- - somebody technical knowledge could provide interpretation best solution. problem required research would take place would rely rational logical thinking arrive solution. regarding emotions , used open communication channels facilitators supervisors discuss emotions opposed letting emotions making project decisions. -- -overall -- - previous role electrical designer smitten data analytics within electricity metering sector. edsa provided platform pursue passion broader scope data using market leading tools importantly upgrade skill-set thus opening new opportunities within workplace .  joined academy crossroads sorts life. academy offered opportunity pivot career engineering data science unfairly dismissed struggling find employment result. meeting 100+ ( faculty students ) new people would part life whole year something get used , even though worked multinational companies. ability deal people different backgrounds , beliefs , etc. tested group work. needless say lead tension one 's life team mates pulling weight. team dynamics always learning process/curve , thankfully bad experience till now. needless say , people try cheat system , act dishonestly sometimes plain lazy. many opinionated thus rendered incapable making great contribution. way things always consider everyone 's well-being , opinions contributions. somehow persons think ok cynical unhelpful. end day , supposed learn become data scientists. work , improve daily. .  far experience positive definitely see value course. experience working different people valuable me. worked well attitude everyone team deliver take responsibility. difficult step back trust team work , however cases team members exceeded expectations. felt bit frustrated team members applied taken substandard lazy approach problems. like get stuck something get immediate results improve there. think approach worked well however also learnt allot team members structured task based approach. allot time spent tangents add value projects however tangents either paid results learnings took them. decision making approach generate ideas create structured work flow around ideas distribute work among individuals take interested work flow. conflicting ideas typically took democratic approach combined healthy debate. find build relationships people 'm teams with. get know best. strongest connections team members also people academy learn from. took team bit data collecting tangent last sprint necessarily productive use time could avoided understood problem better. one case probably assertive team , however way like make decisions get stuck workflow stream adapt go n't supervisor direction point. find environment stimulative enjoy going around teams see working on. good relationship people happy share working great. also found environment distracting times , would like help people get satisfaction , however 'm also that. working different people mixing regularly something im excited everyone different learnings experience theres something learn everyone .  fortunate opportunity join academy year. sure career choice , academy provided great chance change career paths. help choosing new path allow eventually reach larger goals life. moving cape town joining academy challenging , already knew two people academy joining. however made bit difficult making new friends. typically reserved , easily make new friends , especially already good friends , case see reason make new friends. working teams academy great challenge me. normally prefer work pace working team forced cooperate people adjust skill levels. sometimes meant taking work allocated me. addition , found difficult working individuals invested projects aiming learning much experience intend to. within teams , leadership quite lacking. typically like forcing onto people , circumstances , forced take lead projects. taking lead tried best lead behind , making sure consensus within team deciding. generally due lack experience one teams , meant think plan , execution thereof , followed team implementing without challenging ideas laid out. approach worked well , would liked greater input team , unsustainable left would render team incapable functioning properly. team , people much competent four members , decisions much easier take. good idea , still elected leader , role much organic time , much balanced consensual way making decisions. however , completion tasks effectively still perfect left. teams struggled making decisions , never experienced conflict generally emotional making decisions .  looking back one got academy right , confidently say grown intrapersonally interpersonally optimistic perception working also gradually shifted realistic end spectrum. came nothing positive outlook journey gotten opportunity embark , however , learned couple valuable lessons - one stand value understanding teammates expecting one understand you. put far put cap assertive nature gotten lot f pressure given opportunity even sharpen softer skills tends help quite lot even outside academy good vibe generator - one deflects negative energy positive energies whatever spheres happen find in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. learned focus making sure ’ expected delivered timely miss beauty process focussing relatively small hurdles ’ vast amount captivating material engage in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. 've also learned never bring emotions workplace people tend step sensitive things without noticing- example someone staring shoulder commenting work 're quite unpleasant kills moral detaching focusing 's required best coping mechanisms .  attending explore data science academy already proven , continues amazing experience. ’ quite astonishing realize much ’ learned mastered past couple months , chief among would ( subject essay ) , teamwork. meeting new people daunting task ( including ) . however academy different start. sheer excitement students especially explore team ( supervisors facilitators ) acted social catalyst , smoothing social awkwardness , knew getting along like house fire. working teams people completely different backgrounds personalities challenging throughout first project , said , also interesting see individual adapts accommodate team help build team dynamic. starting cape town water crisis project honestly idea expect , certainly ’ expect team dynamic play huge role success deliverables. team overcome different levels knowledge , skills , self motivation. couple late nights flared tempers managed get work done , realised overlooked crucial part teamwork. one need self motivation , also create environment team motivate comfortable enough ask help ’ feeling overwhelmed struggling get grips difficult concept. team definitely lacking aspect , take chin regards deliverables , walked away learning priceless lesson moving next sprint. assigned new teams rsa stock market sprint , refreshing hear ’ team struggled certain aspects team dynamics. get go approached structured interactive way things. decided stand meetings every morning report previous day ’ goals well setting new goals day ahead. already made huge difference motivation team. another part success dividing work ’ get overwhelmed individuals rather sharing workload one-on-ones relevant team members gain deeper understanding section work. approach even advantages thought. structure communication team members lead lower stress levels ( even though still ever present ) , resulting cooler tempers clear headed decisions less emotionally-driven first project. goes show “ soft ” skills important technical skills working teams. ’ wait learn brilliant diverse people academy. '' individual commitment group effort -- makes team work , company work , society work , civilization work. '' -- vince lombardi  essay discussing personal experience regards academy like meeting new people especially introverted person am. speak different teams , different different felt , learned different characters. comparing decision making done teams versus personal decision making. getting plane cape town without place stay friends city big risk , certain plan academy kept praying want scam plot get specific building get kidnapped. fairly surprised get meet people walks life , also kind questioning legit , mean hard question , sounded way good true good things learned ’ happen people like us , guess time shine receive blessings. ’ think academy , blessing. said , experience academy ups downs like always ends up.i learned lot , fallen love data science , things small mindedness used consider geek ’ s. learned push , always try learn , practice computers always evolving. since met people different personalities academy , personality clashes occurred 1 biggest lessons , able work different people different personalities. working teams ’ easy thought would first , everyone idea assumed supposed done different opinions could dismissed everyone 's opinion valuable working team , opinions somehow seemed clash. working teams taught patient , quality possess whatsoever. second time changed teams though seemed though everyone gotten idea working teams ’ like , basically dipping toes working teams thing first sprint , second sprint bit confident meant worked better second teams , improving go. teams , split work equally amongst , making sure everyone gets chance tackle important lessons like coding dashboard. group coordinator state views , he/she would speak supervisor decision would made way , supervisor would guide us right direction hearing thought right way.its almost concept use guess making decisions , weigh options , see one best one worst outcome obviously go best outcome. conclusion working teams skill gaining developing everyday , ’ mastered , heck difficult master many personalities , getting .  coming academy best thing ever happened grateful opportunity. learned academy going encourage teamwork excited meet new people different spheres life exchange ideas network it. working teams thus far ok. things like team commitment work work ethic thereof. comes decision making n't enjoy part felt sense dictatorship member would defy group 's decision still want impose best team start delegating work teammates based vision. felt defied purpose brainstorming planning objectives. role played team always conscientize group imperativeness working within scope prioritizing required us form basis going structure project would help decision making. supervisor meetings reporting based `` '' , `` '' . sign individualism collective project perpetrated unnecessary competition. enjoyed networking variety people academy also participate win hackathon hosted city capetown one best networks team collaborations went well me. like random conversations technology trying link 'd connect inspire other. brief summary work prefer plan try much possible stick plan always room changes. consolidation key helps deciding conflicting ideas .  past months one challenging thrilling experience life. coming , expected engage people knew introvert , experience academy would force engaging people. something totally outside comfort zone. ready ; much prepared , ready challenged understood growth reached uncomfortable spaces. experience taught could imagined. learnt fearless embrace greatness within me. come across shy , willingness push things enabled speak room full people made realize get nervous things actual fact get nervous things capable also fail. way scared inabilities potential. understand still lot learn fully comfortable know feel still lot knowledge gained. coming bit theoretical statistical background , interesting see techniques learnt university applied real world scenarios. coming space , space full well equipped individuals , felt bit intimidated idea career going lacked confidence really good at. experience far definitely given better understanding skill set career take , importantly come learn alone. able meet lots people also similar situation , simply shown worries worth worrying things fall place. environment quite relaxed , yet taught behave professional workplace. support peers also tremendous. observing everyday events supervisors taught teamwork , people come together get things done . although sometimes remind use inside voice , feel adapting environment relatively well. going forward , expect learn develop skills .  experiences explore data science academy experience meeting lots new people great one , months already learned lot , continue daily level. terms meeting new people much gain , especially terms offered different perspectives , challenged way thinking. also taken comfort zone turn taught lot handle different personalities. experience working together lab teams much appreciated close real world experience , 've already similar environment throughout university always found enjoyable interact people. level productivity performance exponentially increased , mainly deadlines needed meet wanting disappoint teammates , generally working others helps achieving individual. say interpersonal skills increased , example speaking listening others. come realize collaborating others helped identify strengths weaknesses , basically self-aware. experience decisions made teams was/am consistent like make decisions. lot people considerate want attain goal , appreciated , far people worked take time listen person say try take consideration relevant concise , decisions made depending everyone felt. experience rational emotional decision-making process been. decisions made rational group took time take people ’ emotions consideration possible. ’ obvious always agree completely certain things , occured , time taken explain person things certain way beneficial way disagreeing member thought. accepting possibility incorrect important role , especially comes working others. conclusion , experience positive anything academy .  attending academy first time like first year , however difference everyone age. idea meeting people really cool however issue forced work people things tend bit shaky everyone standards. make matters interesting grades directly dependent interaction team. varsity must admit best experiences working group tend find similar roles characters. however bit optimistic given diverse ages academy assumption folk work experience hopefully mature. furthermore , come realize establishing solid authentic team dynamics 's also lot stressful actually may seem without trying fake sake team trying accommodating. real issue trust decisions tends made around dominant personalities perspective may particularly best interest team whole. although always agree approach would agree fundamentally understanding skill set equally distributed also end day delivery project. overall happy rational decision making process. divide trust conquer works best teams something come learn. within teams lucky may find someone work ethic dedicated case would likely help could find someone understands work complements style. think initially everyone problem admitting n't get something deliverables due. journey emotional especially towards end tend give less less input become concerned delivery .  good fun experience though 's often case pressure. lab allowed enough time practice get used coding , improved efficiency coding working computer. coming academy n't used idea working team , along year got better , knowing someone count , rely go need help. also got know personal level makes easy understand , strength weaknesses allows us form strong team. sometimes disagreements think right , effective solving , lucky enough co-operative team members since beginning year. though n't got know many colleagues ones got know would say pleasure meeting  experience explore data science academy great far. goals explore year develop & master skills data science well improve leadership skills. felt empowered content ’ taught. four months , believe already learnt skills would allow enter & succeed job market. next 7 months plan spend refining data science skills using 1 hour evenings revising content learnt class. regards second goal ( improve leadership skills ) , ’ fortune team great coordinator & excellent team dynamics. also great team coordinator team. ’ taken many lessons experiences read book. given practical lessons improve leadership skills. ’ two teams far , worked first 1. coordinator nudged us always making progress towards bigger goal. 2. teammates voluntarily picked segment project would responsible for. 3. loose deadlines agreed , deadlines also allowances members equip skills required fulfil parts project. 4. team met twice week discuss progress. 5. time member grew comfortable asking help others well given feedback. team dynamic encouraged member give part project all. inevitably , team members technically advanced others ended taking advanced work rest , however time felt like overburden. given positive experience eager also try coordinator next sprint came around. new teams assigned & volunteered coordinate. aspects believe could done better. 1. ’ commonly accepted bigger goal. 2. ’ involve team identifying small goals were. worked work plan overnight & presented team. team loved plan , individual responsibilities work assigned. 3. led situation members team clear responsibilities were. 4. result power leadership , members team ended bulk work. able pull together & present satisfactory project , however believe could led better & helped team deliver far better project. member team would also proud contributed fully project. learnt great deal experience & would like give coordinating another try , like skill believe leadership learnt practice .  explore data science academy , road never thought would take lifetime. love passion cars , accepted data science internship one last thing never saw coming. love travelling exploring new avenues `` let 's '' tip. first day coming explore data science academy ( edsa ) beyond expectations , visually appealing facilities provided us. first could believe , cape town , comfort zone , pushing something involve anything studied. came expectations , however , told something need give order leave experience back future. speech supervisors gave us 22nd january reassured make mistake coming here. first week given work , install softwares would using throughout year , thought would pre-installed arrived. first , introvert side steering wheel interact lot people besides housemate. started interacting people water crisis sprint started. team people different background ; bioinformatics sciences , computer science , economic statistics mix , different parts south africa ( mostly gauteng ) . one things stood type people would start conversations loud side come order make everybody feel comfortable. first week gather sprint took team start making progress , thursday already figured system worked everyone order use something show supervisor. put weaknesses strengths table , started communicating better team. results shown supervisor told us think would deliver delivered , however , delivered end fundamental , faith would best one best. love good music , adventure socializing led connecting people group. time fundamentals spent helping people , building friendships would carry edsa year even greater , learning languages like python sql , channeling mind data scientist mode. weekends colleagues would tour cape town going beach , hike mountains could see edsa lab , visiting malls theme parks around cape town. python , one complex languages learn. well , thought since programming background. two teammates experience working languages , changing one language another , fairly doable. advantage since two people could consult whenever ran problem python. presentation fundamental one laid back fundamentals face since preplanned wanted add slides clear storyline. fact split fundamentals tore us apart , however , vowed exit whatsapp group created. review week , weekend , people never talked friends edsa lab suggested go theme park near canal walk mall take minds water crisis sprint. character part outting shy , yet interesting backgrounds. still friends them. stock market sprint came flew by. really feeling team since lot judgemental people felt like knew better. lacked communication , managed pull find sprint one piece. events bcx hosted highlights first months edsa , amount inpirational people engaged events. felt like meant every time spoke lucky waiting us greater wildest dreams. week takes cup. never expected go home least july. honestly grateful spending year edsa lab , could chose anything else .  academy amazing , love petry dish talent ! handful genius characters here.. backgrounds unique stories , explore data science academy best place develop data analysis skills. love time ! far , 've experienced whole lot pressure change something better. spend time studying , better feel , comfortable become closer get quality employee entrepreneur. everyday come office , feel home , learn something new.. something directly use business application. look around entire building get inspired start business 've learned here. speak among business academy helping us realize ideas clearly. without information back ideas , 's difficult starting. availability information outstanding. really appreciate nest. last two teams great work with. achieved bunch , , thanks resources available. first sprint tough though. team , realized strengths taught other. n't problem sql python used technologies before. understanding statistics applications data given difficult. worst , water crisis data n't collected well. n't worst experience data taught things career perfect. perfect skills data cleaning. realize limited number solutions data analysis. perfect outcomes data science , could breeze work future. believe hyper prepared things.. game theory data science would prove beneficial. emotionally , 'm great. academy brings best worst skills provides controlled environment mend holes analytical judgement. relief , emotionally physically becomes process bettering daily , bit bit without forcing stress .  experience edsa far combination challenging , fascinating , unpredictable. coming academy , excited would get meet new people learn new skills ; anticipated might somewhat difficult engaging new people ( since introverted ) coming grips new technical ideas ( creative technophile ) , since loved good challenge , still excited immerse experience. surprise , challenges faced none anticipated generally able interact cordially new people understand technicalities program. greatest challenge far working lab. environment many people ( different personalities , habits , learning techniques preferences ) forced natural inclination isolate working. easily distracted , think carefully navigate space simulate environment conducive learning. also found team consider optimally ration work , challenging , especially beginning n't yet good understanding 's strengths weaknesses. said , gotten know fairly well , able make decisions believed would amplify strengths diminish weaknesses could produce results best possible quality. sense , much decision-making teams rational rather emotional. would apply logic reason wanted achieve thought might able , found always room empathetic thoughtful ( thereby engaging emotions rather reason ) , sake team member , alter original plan. somewhat consistent personally make decisions : think times base decisions think yield best results terms trying achieve , even happens inconvenient taxing process ; times , especially decision made great consequence , make decisions based emotions believe 'rational ' 'logical ' decision likely hurt someone 's feelings cast bad light. , learned awful lot : preferences , strengths weaknesses , character ... found learning curve steep emotions rather turbulent throughout process , enjoyed every part challenge could mean future -- prospect holds together worst days exhilarates best days .  experience edsa interesting one thus far. pushed get shell interact people. since academy realised smarter think lot valuable knowledge insight share. biggest fear coming academy social aspect interact different people , since able overcome fear. first placed groups literally sweating , heart rate high felt sick stomach idea group strangers. second time split new groups ’ nervous. become social enjoy meeting working new people now. working groups challenge me. frustrating everyone different everyone different approach learning process project management. ’ learned lot working others. took backseat things even knew better way things. spoke saw helped team steer team better direction. become assertive. thought passive person ’ assertive save life , think assertive ’ find daunting. biggest frustration working teams dealing people think know everything ’ willing open learning something new fellow teammates. person like previous group. person would dictate team ’ want collaborate team , dominating personality rest us shy. realised none us ’ speak ’ fail sprint forced breakout shyness. think still fairly shy comes work fight shyness. decisions previous group initially made one person. prefer problems discussed everyone team different possible solutions explored come collaborative solution. found current team. still issues , people letting pride emotions get way work , people distracted ’ committed project. working labs challenge. lot distractions noise focus easily , prefer work home , get much done home. people come academy socialise anything distracting .  past 5 months explore academy challenging. make great adjustments order here. positive negative experiences. one positive experiences much learned. met lot people highly intelligent interests. worked 2 different teams. prefer work alone working team makes sense due sheer volume work. first team experience incredibly frustrating. team members highly introverted technically strong made discussions assigning tasks difficult. even tasks assigned accountability meet deadlines. prefer chip away steadily work opposed rushing close deadline finish. working second team much better. met regularly , communicated lot. tasks assigned everyone accountable. team members helpful towards .  experiences academy far life-changing say least. meeting smart , passionate , hardworking ambitious people different backgrounds happen many similar interests mine interesting journey. introvert prefers working alone pace within space , quickly find way adapt polish social skills order socialize 99 strangers going learning data science with. understood word go need socialize form meaningful relationships fellow interns could see forming strong network may helpful many years come career , long internship. academy ’ emphasis importance team work concept took warm to. teams exposing different individuals varying academic , social , work geographical background different personalities : introverts , extroverts , intuitive , artistic even really funny characters. teammates phenomenal experienced fields previous exposure to. learned lot them. equally ’ also encountered independent disagreeeable teammates required adjust levels patience , tolerance understanding may necessarily share similar beliefs me. need patient tolerant people ’ points view approaches solving completing team tasks. teams , due differing views , opinions approaches use democracy ( voting ) order reach decisions within groups. although easy one resort taking emotionally-driven decisions due one ’ firm beliefs ways things high level emotional intelligence critical favourable team dynamics. academy based cape town , city ’ visited holidays before. moving cape town challenges required solid ‘ family ’ away family fortunately found fellow interns resourceful many ways even outside ‘ labs ’ . example group share information accomodation , transport around city , social events extramural activities. chance test team-building skills virtues regular group sessions nuri believe helpful giving idea relate people less formal settings. credit due supervisors , facilitators oprerations officer ; helpful making internship experience manageable definitely one memorable years come. commitment programme makes easier people personalities settle nicely within diverse small community .  experience academy exceptional. meeting new people amazing , specifically working them. much challenging first inception relate , time things gets better begin know get use one another. odds events one two group member want participate ( eg. missing meetings ) thing way oppose agreed group , really problem participating also productive , especially meet group expectations group works , members end work. means group take collaborative decision planning expect everyone deliver agreed planned. lab one best way work , especially comes group work , find collaborative useful. 's unfortunate group members choose work home leave early come late. found behavior hinder progress , especially tasks divided sub groups among group members. critical aspect group always communication. members usually communicate inability complete task late work due. members get emotional approach , end day always encourage one another see right side approach task project hand. working many different people taught rely others , means trust member work , even better me. fun leaning since day one .  social perspective , experiences academy refreshing impact personal development. prefer things myself. reflected interest gaming , reading fan-fiction writing. however , willing interact others necessary. enjoy paintball friends far sitting lounge rather making casual small talk. applies workplace well essentially things together. even talking , inevitably get topics related coding such. generally people ( outside academy known years ) seem identify introvert give misconception poor communication skills n't always case. many issues communicating team mates others class regarding various topics ranging work-related hobbies ( especially sport , gaming anime ) . prefer finish roleplaying video game lounge talk also like talking science nerdy things. words , need socialise , problems so. intuition observation valuable traits workplace data scientist team-based project. working teams allowed utilise both. found often analysis others n't done apply intuition. managed thing. different academics would essentially everything myself. regard , team-based nature course done well. often think good one working others stimulate us potentially become good trait previously thought n't naturally suited to. case , working teams sort force us find ways overcome weakness make strength instead. said , team dynamics sometimes involved lack coordinated planning among team members. good keep things open , applied plan rather plan. decide certain approach solving problem involves coding , dashboard , n't mean ca n't improvise. agree improvisation ultimate way things. make plan rely open-minded thinking plan n't appear working , improvise. generally experience academy , emotion seems rule roost logic seen potential barrier team work cooperation. mean students end becoming dependent affect teamwork negatively long term. overall course excelled bringing personality traits relevance workplace , providing opportunities overcome weaknesses utilise strengths instead .  academy challenging experience. expected prior coding knowledge help minimal help , greatest challenge statistics. slowing getting point n't actually try understand concept thoroughly rather get basic theory focus application. overall , insightful journey , learning interact different personalities group dynamics. really excited leaning nlp regression incorporate app development monetize it. genuinely regret applying academy although days doubt hurdles remembering greatness opportunity always great motivator .  big part joining academy year meeting working lots new people. unexpectedly positive experience. despite everyone different backgrounds , sense camaraderie interns friendly. even though everyone tested graded environment non-competitive people willing help other. made enjoyable work lab. however , noise bustle lab sometimes overwhelming draining find productive sometimes spend hour two day lab working home quiet. challenging aspect year far teamwork. first project ( cape town water crisis ) times conflict team due team member contributing much therefore shifting workload team members. led resentment team members , expected. team projects work rarely equally divided team managed complete project despite difficulties. reflecting experience grateful done big share work gave opportunity learn lot would otherwise. second project ( jse stock market sprint ) much shorter time span therefore team approached project different. team divide work time everyone involved deliverables. hard meant trusting people work without directly involved dependant complete it. process decision making teams democratic. team discusses problems possible approaches usually obvious decision everyone agrees on. decision making far tended rational team members emotional. emotional part projects stress faced final deadlines neared. , experience academy far positive learnt lot. hope improve way approach team work upcoming sprints .  ‘ greatest opportunity part 100 chosen individuals wonderful programme. explore data science academy ( edsa ) environment makes feel welcome perfect terms collaboration networking different individuals interesting views personalities. ’ hard distinguish edsa work environment. type environment prepare real job. edsa provides one opportunity learn work diverse group people terms career background , culture religion. working groups different individuals project exciting. throughout time , ’ learned lot working teams. first ’ learned team work achieve terms results learning new things. secondly , come decision making , everyone opinion coming final decision another thing. learned ’ voicing opinion , convincing fellow team members indeed opinion fruitful. comes team strategies embark particular project , ’ one size fits situations. team come strategy going solve assigned problem. difficult , especially serve team coordinator serve , time complete different team whole set different personalities characters. instance learned strategy based skills , capabilities mostly team ’ attitude towards project hand. knowing strengths weaknesses team members good start. far learned lot. ’ still intrigued eager learn lot more. feel like becoming real data scientist day passes thanks edsa giving wonderful opportunity .  explore data science academy 19 weeks ( counted ) ; let ’ simplify 3 months shall ? exciting working group challenging problems , feel like world possibilities opened us. exciting overwhelming met many new people. right away seem made name myself. proud , even excited people feel someone come help - practical emotional. helping others role comfortable since typically excelled academic environment , ’ admit disappointment people ’ ask help more. like thought expert ( even like feel like expert , find happens less grow older ( experienced ? wiser ? one old greeks said something wisdom knowing ’ know ) . aspect larger group find overwhelming trying remember everyone , especially since many people seem know am. enjoy learning people ’ backgrounds amazed great selection people attending college. experience working intensely type team entirely new. worked teams school , university work , never amount diversity team. never sitting next team members everyday ; majority one ’ efforts focussed team work. immediate challenge first team working people skills ways operating different own. realised career worked people , background training. made less adaptable less tolerant different approaches problem. trying learn relax give people ’ method approaching problems chance. felt comfortable group leader first sprint , uncomfortable giving reigns second sprint onwards. second project usurped actual leader number occasions seldom pulled back. formed real bond first team , despite ( ) struggles. second team relationship seems formal , although things went much smoothly team arguably effective. found frustrating deal reserved people. hard read difficult reserve judgement ’ produce tangible outputs. still trying figure best deal people , draw shell patience hear trying say. also need consciously tell offer may valuable ( remain unconvinced evidence right table ) . experience also underlined much rely external encouragement praise , played interactions students staff. enjoy teaching students , specifically one one small groups , alot immediate feedback. first weeks felt really supported facilitators , later weeks felt like abandoned newer , exciting projects. similarly felt much secure first mentor ’ hands-on , available , approach second facilitator ’ reserved , hands-off approach .  young age always sheltered , reserved terrified getting comfort zone. knew year would challenge ’ challenged break shell. first day got real knew one person hundred. push open interact crowd ( struggle considering ’ shy standing front people makes extremely nervous ) . lucky people academy friendly understanding. working teams never problem , issue able fully center attention speak thoughts. taught importance communicate ideas without shy. one biggest challenge working team dealing teammates ’ fully committed tasks assigned them. personally prefer get things done certain way bit challenge team timelines ’ met team members ’ putting much effort everyone else. lab exposes us different personalities one needs learn work together effectively many personalities many conflicting personalities could recipe disaster. could say lucky teams assigned ’ difficult. understood importance contribution communication. misunderstanding ’ address becomes conflict , way able prevent it. teams decision-making process pretty much same. generally , challenge put team , communicate strengths weaknesses , able distribute tasks together based strengths weaknesses. decisions need made , every member given opportunity give opinion decide together team taking consideration every member 's contribution. overall , experience far great. learning new things everyday. workload get challenging time time managed. slowly coming shell , talking , interacting opening .  experience academy thus far potent insightful year , changed life become person really enjoys importance cooperating others. brought together amazing minds space really helped acquire soft hard skills short space time , indeed helped elevate way learning new technical subjects efficiently effectively. also , management team large especially supervisors instrumental always improvising make sure led way help us walk away well informed junior data scientists end year , hence work tirelessly making sure break obstacle attractive easily comprehended possible us order put hard work. addition , nuri formidable way helping us cope psychological illnesses caused intensity program , always go extra mile making sure one feels solitary. supporting structure throughout program importance , help us understand essence behind excruciating vulnerability management enable see work not. last words , want close saying means life , 'm grateful opportunity , best thing ever happened life besides varsity , 'm pretty sure set life greater things achievements .  time academy far challenging , yet extremely rewarding self-fulfilling - especially regards working people different backgrounds domains expertise. although meeting different people interesting , concerned bring different people together towards achieving common goal. two teams worked , often one take lead , drawing ambition past experience relation data science related fields. challenging , since used - comfortable - working alone. hold personal contributions team-based projects highest standards , therefore difficult team members show dedication team-based projects do. working teams challenged personality positive negative ways. like control , therefore difficult allow people take lead different aspects projects , especially objectively feel experience completing particular objective. however , learned impossible control aspects project therefore welcomed aided contributions team members shown willingness aptitude contribute. contrast , difficult form motivate team members show apathy towards achieving common goal. although somewhat frustrating , ensured never derailed team following initial plan objectives. felt comfortable role coordinator , relished opportunity deliver high-quality deliverables high pressure situations. drive work ethic good , always productive worker. over-analysed spent excessive amounts time subjects interesting challenging personal level , instead focusing time pressing matters. generally tend spend time challenging interesting concepts , tend postpone often neglect routine administrative tasks. creates frantic dash line time runs , creates unnecessary anxiety. conscious shortcoming , actively try remind prioritise complete pressing matters regardless personal interest. know often regarded serious fellow team mates , since always focused driven comes task hand. little time chit-chat tend engage conversations related intriguing philosophical subjects. feel comfortable responsive. however , learned casual conversation regarding sports hobbies helps learn team members , forges professional , understanding relationships team members. overall , work experience academy enlightening one , far perfect. lots learn , eager improve aspect career benefit team mates .  hardest moments life learning new things , matter simple easy might be. difficulty met new people academy. truth talkative nature , ’ always found difficult start conversations students anyone academy. however , amazing experience discover learn different students ’ cultural backgrounds diversities , interest data science life whole. academy , realized people learn discover interaction others. new environments nervous , yet exciting ones. explore data science academy best example scenarios. academy provided great exposure working environment challenges ought face work teams. able work team prerequisite skill needed data scientist. learning curve allowed self disciplined , self-motivated introspection little effort group work affect entire team ’ progress thus , results poor performance entire team work. people different cultural , educational programming backgrounds , learning difference right , right right became key determining factor understanding work teams. months academy , feel close becoming data scientist before. hunger efforts put understand data science seem rewarding now. great feeling among first group given opportunity learn data science. although pressure much times , hope believe everything designed meant prepare us real world working environments .  22nd january 2018 , time 7:00. seemed like normal day time. hit , difference location. 'm western cape , cape town precise. moved away home embark journey become data scientist explore data science academy ( edsa ) .today 8th may 2018 lot happened since ... first week got meet students different family background different level education. met explore team , included supervisors , facilitators nuri. told would working groups fives. nervous got used everything own. actually terrified. first group called optimize prime ( still cracks ) . made nervous everyone post matric qualification. beginning thought underdog , realised competition. mean must 've reason teamed guys , learn contribute wherever can. next group , named wolves woodstock ( named movie wolves wallstreet ) , confident engaging. developed previous group. even headed one sub projects. experience amazing great. even sister n't believe me. first group given opportunity work cape town water crisis , investigated demand supply factors affected cape town water crisis. project got use sql powerbi first time. saw another use python n't know capability , analysing data. second group got explore rsa stock market data. used data predict target return individual companies. interesting project accuracy predictions , limitations models. group assigned supervisors. got chance mentored two different supervisors. wish could create table comparison , two different , effective. times stats , python others gets much. group sessions nuri , refresh .  experience edsa great far. massive learning curve academically also personally. took myers-briggs test twice : , last night earlier today got slightly different results. first time came letter combination infp second time isfj. though slightly different , feel accurate overall representation various attributes exhibit. seen diplomat-mediator well ( second time taking test ) defender sentinel. naturally reserved nature , unfriendly , prefer working alone comfortable enough sit alone group people. however , approached , come across friendly. able read various personality types groups fairly quickly , allows quickly adapt different situations needed group times. n't usually take charge , know end goal something achieve , prefer clear cut plan achieve , one challenging parts working group , made need step comfort zone recently make decisions voice myself. times experienced stress due fact felt frustrated able comprehend certain concepts quick enough significant work project , deadlines much issue. overall , feel thoroughly enjoyed learning group dynamics far edsa .  nan  like meeting new people similar interests , many new people , connecting people time goes assigned new group. n't spoken students merely strong introverts , recent group nominated group coordinator take lead every meeting , every interaction acknowledge personal level group members feeling day , something never really thought experienced , even grew every mondays share weekend , really appreciated put position challenge grow personal academically. like make decisions based merits decision efficient come attention efficient time take account people 's feelings , made decisions past group people feel included left , worth noting individuals contribute project way , barely showed academy , never written one single line code left member take care making dashboard , presentation ultimately present , two us took uncomfortable route diplomacy , pleaded every morning attend briefings supervisor us , went group training sessions made impact , next group , understood academy meant worked together well. stallions , magnificent , sudden paradigm changed , enjoyed one idea best things , originally typed minutes meetings another member suggested trello effective , learnt something new , excited , day , well every day went bed knowing something ’ know woke ’ data scientist ask for. one important lesson got working two different groups leave time guide team work flow everyone miserable end project , however everything planned trello ( ’ trello ) ‘ tees ’ crossed ‘ eyes ’ dotted , pleasant experience bring best everyone involved , team creativity grow levels one planned beginning. , irrespective put group work , sit maybe second session “ getting know ” map strategy roles assigned , significant insignificant .  nan  edsa reflections 24 november 2017 , filled joy , excitement anxious time opened email edsa titled “ congratulations ! ’ accepted explore data science academy ! ” fast forward 22 january 2018 , ready meet new people ( possibly form new friendships ) , meet edsa team learn lies ahead 2018 , overwelming day was. today 6 may 2018 , lot happened past 3 months , ’ learned much ! sql , stats , python , power bi , presentation skills… [ still learning , never stops : ) ] . grateful selected one top 100 , ’ appreciative opportunity , thank edsa making possible me. lab data scientists making , greatly enjoy lab , environment condusive learning , group data science enthusiasts one room feel receipe success. training limited online courses edsa staff students learn lot , different backrounds , skills experience making lab interesting fun place grow academically socially. teamwork water crisis project : team liquidata must say initially concerned amount individual work deliver team well. realized importance effective team , working well team reduced work load stress also helpful individual tasks. roses regards team dynamics , working different individuals perceptions opinions challenging first , frustrating decide certain matters , helped lot grounded problem statement kept us focused needed done. worked well ’ result driven given task , completing task successfully required relevant info important , refering back problem statement got stuck great tool us. emotional moments team due personality clashes end worked well contributed success team , awesome team ! rsa stock market project : wolves woodstock accepted ’ work new people , really problem , hope wish strong programmer team , wish realised statistician programmer team , lovely ! great team , felt like outspoken person ( ) bit much , still worked well though delivered. prefer working team different ideas , creativity different skills ’ huge advantage eveyone team result driven , absolutelty appreciate that. ’ good experience teams far .  year huge one change , involving many aspects : leaving stable job , moving cities , moving new field , going back student , long distance taking chance brand new data science academy. quite excited meet new data science students despite usual shyness. people taking similar plunges shared interest field. enjoyed ( slowly ) meeting new people learning backgrounds , interesting varied stories are. working lab new experience. complete self learning tough journey , glad academy experience supervisors help guide us way becoming data scientists. bit unusual area ’ computer lab learning , also sort office , also people socialise. ’ sure nailed exactly be. feel work better home quiet comfortable ( apparently line personality profile ) . group work element year probably one biggest pain points. see intent setting us way , ’ think quite worked yet . group work challenging. throw meeting new people different backgrounds , huge amount new content learned fairly significant stakes ( internship opportunity ) difficult situation. think resulted frustrations running high poor quality work delivered , individual group would wanted. think worrying getting project done sometimes gotten way actually trying learn content. terms decision making groups far , think one positive definitely logic valued field students. state reasoning behind plan solidly ’ problem. think challenge group member interpret instructions priorities differently , sorting get get everyone page take valuable time. personal level noticed always looking clear , decisive decisions plans , deal well “ seeing goes ” . think something need look team setting , expect manager well. overall , still pretty excited year regrets decision join. think academy still bit rough around edges needs refinement , part iterative process right ?  first 4 months edsa filled many new experiences ; challenges , others joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people everyone met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates within edsa , provided insight gained experience tools like python & sql. also cherish able build network data scientists tomorrow well established industry professionals.. -- -working together lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious con would noise distraction - least say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience would say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes full responsibility part ) members first group noobs sql & python , gather analyse phases really exposed technical deficiencies resulted lot frustration came time delivering tasks. experience took comfort zone important lesson patience us all. second group high level experience , strong team co-ordinator team members took full responsibility tasks. even though work complex comfortable team dynamic. -- -how decisions teams made consistent make decisions -- - team decisions groups made deliberation. decisions easier make person presented clear points opinion was. group dynamic strong decision making felt natural , similiar make decision own. -- -how rational emotional decision making process -- - somebody technical knowledge could provide interpretation best solution. problem required research would take place would rely rational logical thinking arrive solution. regarding emotions , used open communication channels facilitators supervisors discuss emotions opposed letting emotions making project decisions. -- -overall -- - previous role electrical designer smitten data analytics within electricity metering sector. edsa provided platform pursue passion broader scope data using market leading tools importantly upgrade skill-set thus opening new opportunities within workplace .  joined academy crossroads sorts life. academy offered opportunity pivot career engineering data science unfairly dismissed struggling find employment result. meeting 100+ ( faculty students ) new people would part life whole year something get used , even though worked multinational companies. ability deal people different backgrounds , beliefs , etc. tested group work. needless say lead tension one 's life team mates pulling weight. team dynamics always learning process/curve , thankfully bad experience till now. needless say , people try cheat system , act dishonestly sometimes plain lazy. many opinionated thus rendered incapable making great contribution. way things always consider everyone 's well-being , opinions contributions. somehow persons think ok cynical unhelpful. end day , supposed learn become data scientists. work , improve daily. .  far experience positive definitely see value course. experience working different people valuable me. worked well attitude everyone team deliver take responsibility. difficult step back trust team work , however cases team members exceeded expectations. felt bit frustrated team members applied taken substandard lazy approach problems. like get stuck something get immediate results improve there. think approach worked well however also learnt allot team members structured task based approach. allot time spent tangents add value projects however tangents either paid results learnings took them. decision making approach generate ideas create structured work flow around ideas distribute work among individuals take interested work flow. conflicting ideas typically took democratic approach combined healthy debate. find build relationships people 'm teams with. get know best. strongest connections team members also people academy learn from. took team bit data collecting tangent last sprint necessarily productive use time could avoided understood problem better. one case probably assertive team , however way like make decisions get stuck workflow stream adapt go n't supervisor direction point. find environment stimulative enjoy going around teams see working on. good relationship people happy share working great. also found environment distracting times , would like help people get satisfaction , however 'm also that. working different people mixing regularly something im excited everyone different learnings experience theres something learn everyone .  fortunate opportunity join academy year. sure career choice , academy provided great chance change career paths. help choosing new path allow eventually reach larger goals life. moving cape town joining academy challenging , already knew two people academy joining. however made bit difficult making new friends. typically reserved , easily make new friends , especially already good friends , case see reason make new friends. working teams academy great challenge me. normally prefer work pace working team forced cooperate people adjust skill levels. sometimes meant taking work allocated me. addition , found difficult working individuals invested projects aiming learning much experience intend to. within teams , leadership quite lacking. typically like forcing onto people , circumstances , forced take lead projects. taking lead tried best lead behind , making sure consensus within team deciding. generally due lack experience one teams , meant think plan , execution thereof , followed team implementing without challenging ideas laid out. approach worked well , would liked greater input team , unsustainable left would render team incapable functioning properly. team , people much competent four members , decisions much easier take. good idea , still elected leader , role much organic time , much balanced consensual way making decisions. however , completion tasks effectively still perfect left. teams struggled making decisions , never experienced conflict generally emotional making decisions .  looking back one got academy right , confidently say grown intrapersonally interpersonally optimistic perception working also gradually shifted realistic end spectrum. came nothing positive outlook journey gotten opportunity embark , however , learned couple valuable lessons - one stand value understanding teammates expecting one understand you. put far put cap assertive nature gotten lot f pressure given opportunity even sharpen softer skills tends help quite lot even outside academy good vibe generator - one deflects negative energy positive energies whatever spheres happen find in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. learned focus making sure ’ expected delivered timely miss beauty process focussing relatively small hurdles ’ vast amount captivating material engage in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. 've also learned never bring emotions workplace people tend step sensitive things without noticing- example someone staring shoulder commenting work 're quite unpleasant kills moral detaching focusing 's required best coping mechanisms .  attending explore data science academy already proven , continues amazing experience. ’ quite astonishing realize much ’ learned mastered past couple months , chief among would ( subject essay ) , teamwork. meeting new people daunting task ( including ) . however academy different start. sheer excitement students especially explore team ( supervisors facilitators ) acted social catalyst , smoothing social awkwardness , knew getting along like house fire. working teams people completely different backgrounds personalities challenging throughout first project , said , also interesting see individual adapts accommodate team help build team dynamic. starting cape town water crisis project honestly idea expect , certainly ’ expect team dynamic play huge role success deliverables. team overcome different levels knowledge , skills , self motivation. couple late nights flared tempers managed get work done , realised overlooked crucial part teamwork. one need self motivation , also create environment team motivate comfortable enough ask help ’ feeling overwhelmed struggling get grips difficult concept. team definitely lacking aspect , take chin regards deliverables , walked away learning priceless lesson moving next sprint. assigned new teams rsa stock market sprint , refreshing hear ’ team struggled certain aspects team dynamics. get go approached structured interactive way things. decided stand meetings every morning report previous day ’ goals well setting new goals day ahead. already made huge difference motivation team. another part success dividing work ’ get overwhelmed individuals rather sharing workload one-on-ones relevant team members gain deeper understanding section work. approach even advantages thought. structure communication team members lead lower stress levels ( even though still ever present ) , resulting cooler tempers clear headed decisions less emotionally-driven first project. goes show “ soft ” skills important technical skills working teams. ’ wait learn brilliant diverse people academy. '' individual commitment group effort -- makes team work , company work , society work , civilization work. '' -- vince lombardi  essay discussing personal experience regards academy like meeting new people especially introverted person am. speak different teams , different different felt , learned different characters. comparing decision making done teams versus personal decision making. getting plane cape town without place stay friends city big risk , certain plan academy kept praying want scam plot get specific building get kidnapped. fairly surprised get meet people walks life , also kind questioning legit , mean hard question , sounded way good true good things learned ’ happen people like us , guess time shine receive blessings. ’ think academy , blessing. said , experience academy ups downs like always ends up.i learned lot , fallen love data science , things small mindedness used consider geek ’ s. learned push , always try learn , practice computers always evolving. since met people different personalities academy , personality clashes occurred 1 biggest lessons , able work different people different personalities. working teams ’ easy thought would first , everyone idea assumed supposed done different opinions could dismissed everyone 's opinion valuable working team , opinions somehow seemed clash. working teams taught patient , quality possess whatsoever. second time changed teams though seemed though everyone gotten idea working teams ’ like , basically dipping toes working teams thing first sprint , second sprint bit confident meant worked better second teams , improving go. teams , split work equally amongst , making sure everyone gets chance tackle important lessons like coding dashboard. group coordinator state views , he/she would speak supervisor decision would made way , supervisor would guide us right direction hearing thought right way.its almost concept use guess making decisions , weigh options , see one best one worst outcome obviously go best outcome. conclusion working teams skill gaining developing everyday , ’ mastered , heck difficult master many personalities , getting .  coming academy best thing ever happened grateful opportunity. learned academy going encourage teamwork excited meet new people different spheres life exchange ideas network it. working teams thus far ok. things like team commitment work work ethic thereof. comes decision making n't enjoy part felt sense dictatorship member would defy group 's decision still want impose best team start delegating work teammates based vision. felt defied purpose brainstorming planning objectives. role played team always conscientize group imperativeness working within scope prioritizing required us form basis going structure project would help decision making. supervisor meetings reporting based `` '' , `` '' . sign individualism collective project perpetrated unnecessary competition. enjoyed networking variety people academy also participate win hackathon hosted city capetown one best networks team collaborations went well me. like random conversations technology trying link 'd connect inspire other. brief summary work prefer plan try much possible stick plan always room changes. consolidation key helps deciding conflicting ideas .  past months one challenging thrilling experience life. coming , expected engage people knew introvert , experience academy would force engaging people. something totally outside comfort zone. ready ; much prepared , ready challenged understood growth reached uncomfortable spaces. experience taught could imagined. learnt fearless embrace greatness within me. come across shy , willingness push things enabled speak room full people made realize get nervous things actual fact get nervous things capable also fail. way scared inabilities potential. understand still lot learn fully comfortable know feel still lot knowledge gained. coming bit theoretical statistical background , interesting see techniques learnt university applied real world scenarios. coming space , space full well equipped individuals , felt bit intimidated idea career going lacked confidence really good at. experience far definitely given better understanding skill set career take , importantly come learn alone. able meet lots people also similar situation , simply shown worries worth worrying things fall place. environment quite relaxed , yet taught behave professional workplace. support peers also tremendous. observing everyday events supervisors taught teamwork , people come together get things done . although sometimes remind use inside voice , feel adapting environment relatively well. going forward , expect learn develop skills .  experiences explore data science academy experience meeting lots new people great one , months already learned lot , continue daily level. terms meeting new people much gain , especially terms offered different perspectives , challenged way thinking. also taken comfort zone turn taught lot handle different personalities. experience working together lab teams much appreciated close real world experience , 've already similar environment throughout university always found enjoyable interact people. level productivity performance exponentially increased , mainly deadlines needed meet wanting disappoint teammates , generally working others helps achieving individual. say interpersonal skills increased , example speaking listening others. come realize collaborating others helped identify strengths weaknesses , basically self-aware. experience decisions made teams was/am consistent like make decisions. lot people considerate want attain goal , appreciated , far people worked take time listen person say try take consideration relevant concise , decisions made depending everyone felt. experience rational emotional decision-making process been. decisions made rational group took time take people ’ emotions consideration possible. ’ obvious always agree completely certain things , occured , time taken explain person things certain way beneficial way disagreeing member thought. accepting possibility incorrect important role , especially comes working others. conclusion , experience positive anything academy .  attending academy first time like first year , however difference everyone age. idea meeting people really cool however issue forced work people things tend bit shaky everyone standards. make matters interesting grades directly dependent interaction team. varsity must admit best experiences working group tend find similar roles characters. however bit optimistic given diverse ages academy assumption folk work experience hopefully mature. furthermore , come realize establishing solid authentic team dynamics 's also lot stressful actually may seem without trying fake sake team trying accommodating. real issue trust decisions tends made around dominant personalities perspective may particularly best interest team whole. although always agree approach would agree fundamentally understanding skill set equally distributed also end day delivery project. overall happy rational decision making process. divide trust conquer works best teams something come learn. within teams lucky may find someone work ethic dedicated case would likely help could find someone understands work complements style. think initially everyone problem admitting n't get something deliverables due. journey emotional especially towards end tend give less less input become concerned delivery .  good fun experience though 's often case pressure. lab allowed enough time practice get used coding , improved efficiency coding working computer. coming academy n't used idea working team , along year got better , knowing someone count , rely go need help. also got know personal level makes easy understand , strength weaknesses allows us form strong team. sometimes disagreements think right , effective solving , lucky enough co-operative team members since beginning year. though n't got know many colleagues ones got know would say pleasure meeting  experience explore data science academy great far. goals explore year develop & master skills data science well improve leadership skills. felt empowered content ’ taught. four months , believe already learnt skills would allow enter & succeed job market. next 7 months plan spend refining data science skills using 1 hour evenings revising content learnt class. regards second goal ( improve leadership skills ) , ’ fortune team great coordinator & excellent team dynamics. also great team coordinator team. ’ taken many lessons experiences read book. given practical lessons improve leadership skills. ’ two teams far , worked first 1. coordinator nudged us always making progress towards bigger goal. 2. teammates voluntarily picked segment project would responsible for. 3. loose deadlines agreed , deadlines also allowances members equip skills required fulfil parts project. 4. team met twice week discuss progress. 5. time member grew comfortable asking help others well given feedback. team dynamic encouraged member give part project all. inevitably , team members technically advanced others ended taking advanced work rest , however time felt like overburden. given positive experience eager also try coordinator next sprint came around. new teams assigned & volunteered coordinate. aspects believe could done better. 1. ’ commonly accepted bigger goal. 2. ’ involve team identifying small goals were. worked work plan overnight & presented team. team loved plan , individual responsibilities work assigned. 3. led situation members team clear responsibilities were. 4. result power leadership , members team ended bulk work. able pull together & present satisfactory project , however believe could led better & helped team deliver far better project. member team would also proud contributed fully project. learnt great deal experience & would like give coordinating another try , like skill believe leadership learnt practice .  explore data science academy , road never thought would take lifetime. love passion cars , accepted data science internship one last thing never saw coming. love travelling exploring new avenues `` let 's '' tip. first day coming explore data science academy ( edsa ) beyond expectations , visually appealing facilities provided us. first could believe , cape town , comfort zone , pushing something involve anything studied. came expectations , however , told something need give order leave experience back future. speech supervisors gave us 22nd january reassured make mistake coming here. first week given work , install softwares would using throughout year , thought would pre-installed arrived. first , introvert side steering wheel interact lot people besides housemate. started interacting people water crisis sprint started. team people different background ; bioinformatics sciences , computer science , economic statistics mix , different parts south africa ( mostly gauteng ) . one things stood type people would start conversations loud side come order make everybody feel comfortable. first week gather sprint took team start making progress , thursday already figured system worked everyone order use something show supervisor. put weaknesses strengths table , started communicating better team. results shown supervisor told us think would deliver delivered , however , delivered end fundamental , faith would best one best. love good music , adventure socializing led connecting people group. time fundamentals spent helping people , building friendships would carry edsa year even greater , learning languages like python sql , channeling mind data scientist mode. weekends colleagues would tour cape town going beach , hike mountains could see edsa lab , visiting malls theme parks around cape town. python , one complex languages learn. well , thought since programming background. two teammates experience working languages , changing one language another , fairly doable. advantage since two people could consult whenever ran problem python. presentation fundamental one laid back fundamentals face since preplanned wanted add slides clear storyline. fact split fundamentals tore us apart , however , vowed exit whatsapp group created. review week , weekend , people never talked friends edsa lab suggested go theme park near canal walk mall take minds water crisis sprint. character part outting shy , yet interesting backgrounds. still friends them. stock market sprint came flew by. really feeling team since lot judgemental people felt like knew better. lacked communication , managed pull find sprint one piece. events bcx hosted highlights first months edsa , amount inpirational people engaged events. felt like meant every time spoke lucky waiting us greater wildest dreams. week takes cup. never expected go home least july. honestly grateful spending year edsa lab , could chose anything else .  academy amazing , love petry dish talent ! handful genius characters here.. backgrounds unique stories , explore data science academy best place develop data analysis skills. love time ! far , 've experienced whole lot pressure change something better. spend time studying , better feel , comfortable become closer get quality employee entrepreneur. everyday come office , feel home , learn something new.. something directly use business application. look around entire building get inspired start business 've learned here. speak among business academy helping us realize ideas clearly. without information back ideas , 's difficult starting. availability information outstanding. really appreciate nest. last two teams great work with. achieved bunch , , thanks resources available. first sprint tough though. team , realized strengths taught other. n't problem sql python used technologies before. understanding statistics applications data given difficult. worst , water crisis data n't collected well. n't worst experience data taught things career perfect. perfect skills data cleaning. realize limited number solutions data analysis. perfect outcomes data science , could breeze work future. believe hyper prepared things.. game theory data science would prove beneficial. emotionally , 'm great. academy brings best worst skills provides controlled environment mend holes analytical judgement. relief , emotionally physically becomes process bettering daily , bit bit without forcing stress .  experience edsa far combination challenging , fascinating , unpredictable. coming academy , excited would get meet new people learn new skills ; anticipated might somewhat difficult engaging new people ( since introverted ) coming grips new technical ideas ( creative technophile ) , since loved good challenge , still excited immerse experience. surprise , challenges faced none anticipated generally able interact cordially new people understand technicalities program. greatest challenge far working lab. environment many people ( different personalities , habits , learning techniques preferences ) forced natural inclination isolate working. easily distracted , think carefully navigate space simulate environment conducive learning. also found team consider optimally ration work , challenging , especially beginning n't yet good understanding 's strengths weaknesses. said , gotten know fairly well , able make decisions believed would amplify strengths diminish weaknesses could produce results best possible quality. sense , much decision-making teams rational rather emotional. would apply logic reason wanted achieve thought might able , found always room empathetic thoughtful ( thereby engaging emotions rather reason ) , sake team member , alter original plan. somewhat consistent personally make decisions : think times base decisions think yield best results terms trying achieve , even happens inconvenient taxing process ; times , especially decision made great consequence , make decisions based emotions believe 'rational ' 'logical ' decision likely hurt someone 's feelings cast bad light. , learned awful lot : preferences , strengths weaknesses , character ... found learning curve steep emotions rather turbulent throughout process , enjoyed every part challenge could mean future -- prospect holds together worst days exhilarates best days .  experience edsa interesting one thus far. pushed get shell interact people. since academy realised smarter think lot valuable knowledge insight share. biggest fear coming academy social aspect interact different people , since able overcome fear. first placed groups literally sweating , heart rate high felt sick stomach idea group strangers. second time split new groups ’ nervous. become social enjoy meeting working new people now. working groups challenge me. frustrating everyone different everyone different approach learning process project management. ’ learned lot working others. took backseat things even knew better way things. spoke saw helped team steer team better direction. become assertive. thought passive person ’ assertive save life , think assertive ’ find daunting. biggest frustration working teams dealing people think know everything ’ willing open learning something new fellow teammates. person like previous group. person would dictate team ’ want collaborate team , dominating personality rest us shy. realised none us ’ speak ’ fail sprint forced breakout shyness. think still fairly shy comes work fight shyness. decisions previous group initially made one person. prefer problems discussed everyone team different possible solutions explored come collaborative solution. found current team. still issues , people letting pride emotions get way work , people distracted ’ committed project. working labs challenge. lot distractions noise focus easily , prefer work home , get much done home. people come academy socialise anything distracting .  past 5 months explore academy challenging. make great adjustments order here. positive negative experiences. one positive experiences much learned. met lot people highly intelligent interests. worked 2 different teams. prefer work alone working team makes sense due sheer volume work. first team experience incredibly frustrating. team members highly introverted technically strong made discussions assigning tasks difficult. even tasks assigned accountability meet deadlines. prefer chip away steadily work opposed rushing close deadline finish. working second team much better. met regularly , communicated lot. tasks assigned everyone accountable. team members helpful towards .  experiences academy far life-changing say least. meeting smart , passionate , hardworking ambitious people different backgrounds happen many similar interests mine interesting journey. introvert prefers working alone pace within space , quickly find way adapt polish social skills order socialize 99 strangers going learning data science with. understood word go need socialize form meaningful relationships fellow interns could see forming strong network may helpful many years come career , long internship. academy ’ emphasis importance team work concept took warm to. teams exposing different individuals varying academic , social , work geographical background different personalities : introverts , extroverts , intuitive , artistic even really funny characters. teammates phenomenal experienced fields previous exposure to. learned lot them. equally ’ also encountered independent disagreeeable teammates required adjust levels patience , tolerance understanding may necessarily share similar beliefs me. need patient tolerant people ’ points view approaches solving completing team tasks. teams , due differing views , opinions approaches use democracy ( voting ) order reach decisions within groups. although easy one resort taking emotionally-driven decisions due one ’ firm beliefs ways things high level emotional intelligence critical favourable team dynamics. academy based cape town , city ’ visited holidays before. moving cape town challenges required solid ‘ family ’ away family fortunately found fellow interns resourceful many ways even outside ‘ labs ’ . example group share information accomodation , transport around city , social events extramural activities. chance test team-building skills virtues regular group sessions nuri believe helpful giving idea relate people less formal settings. credit due supervisors , facilitators oprerations officer ; helpful making internship experience manageable definitely one memorable years come. commitment programme makes easier people personalities settle nicely within diverse small community .  experience academy exceptional. meeting new people amazing , specifically working them. much challenging first inception relate , time things gets better begin know get use one another. odds events one two group member want participate ( eg. missing meetings ) thing way oppose agreed group , really problem participating also productive , especially meet group expectations group works , members end work. means group take collaborative decision planning expect everyone deliver agreed planned. lab one best way work , especially comes group work , find collaborative useful. 's unfortunate group members choose work home leave early come late. found behavior hinder progress , especially tasks divided sub groups among group members. critical aspect group always communication. members usually communicate inability complete task late work due. members get emotional approach , end day always encourage one another see right side approach task project hand. working many different people taught rely others , means trust member work , even better me. fun leaning since day one .  social perspective , experiences academy refreshing impact personal development. prefer things myself. reflected interest gaming , reading fan-fiction writing. however , willing interact others necessary. enjoy paintball friends far sitting lounge rather making casual small talk. applies workplace well essentially things together. even talking , inevitably get topics related coding such. generally people ( outside academy known years ) seem identify introvert give misconception poor communication skills n't always case. many issues communicating team mates others class regarding various topics ranging work-related hobbies ( especially sport , gaming anime ) . prefer finish roleplaying video game lounge talk also like talking science nerdy things. words , need socialise , problems so. intuition observation valuable traits workplace data scientist team-based project. working teams allowed utilise both. found often analysis others n't done apply intuition. managed thing. different academics would essentially everything myself. regard , team-based nature course done well. often think good one working others stimulate us potentially become good trait previously thought n't naturally suited to. case , working teams sort force us find ways overcome weakness make strength instead. said , team dynamics sometimes involved lack coordinated planning among team members. good keep things open , applied plan rather plan. decide certain approach solving problem involves coding , dashboard , n't mean ca n't improvise. agree improvisation ultimate way things. make plan rely open-minded thinking plan n't appear working , improvise. generally experience academy , emotion seems rule roost logic seen potential barrier team work cooperation. mean students end becoming dependent affect teamwork negatively long term. overall course excelled bringing personality traits relevance workplace , providing opportunities overcome weaknesses utilise strengths instead .  academy challenging experience. expected prior coding knowledge help minimal help , greatest challenge statistics. slowing getting point n't actually try understand concept thoroughly rather get basic theory focus application. overall , insightful journey , learning interact different personalities group dynamics. really excited leaning nlp regression incorporate app development monetize it. genuinely regret applying academy although days doubt hurdles remembering greatness opportunity always great motivator .  big part joining academy year meeting working lots new people. unexpectedly positive experience. despite everyone different backgrounds , sense camaraderie interns friendly. even though everyone tested graded environment non-competitive people willing help other. made enjoyable work lab. however , noise bustle lab sometimes overwhelming draining find productive sometimes spend hour two day lab working home quiet. challenging aspect year far teamwork. first project ( cape town water crisis ) times conflict team due team member contributing much therefore shifting workload team members. led resentment team members , expected. team projects work rarely equally divided team managed complete project despite difficulties. reflecting experience grateful done big share work gave opportunity learn lot would otherwise. second project ( jse stock market sprint ) much shorter time span therefore team approached project different. team divide work time everyone involved deliverables. hard meant trusting people work without directly involved dependant complete it. process decision making teams democratic. team discusses problems possible approaches usually obvious decision everyone agrees on. decision making far tended rational team members emotional. emotional part projects stress faced final deadlines neared. , experience academy far positive learnt lot. hope improve way approach team work upcoming sprints .  ‘ greatest opportunity part 100 chosen individuals wonderful programme. explore data science academy ( edsa ) environment makes feel welcome perfect terms collaboration networking different individuals interesting views personalities. ’ hard distinguish edsa work environment. type environment prepare real job. edsa provides one opportunity learn work diverse group people terms career background , culture religion. working groups different individuals project exciting. throughout time , ’ learned lot working teams. first ’ learned team work achieve terms results learning new things. secondly , come decision making , everyone opinion coming final decision another thing. learned ’ voicing opinion , convincing fellow team members indeed opinion fruitful. comes team strategies embark particular project , ’ one size fits situations. team come strategy going solve assigned problem. difficult , especially serve team coordinator serve , time complete different team whole set different personalities characters. instance learned strategy based skills , capabilities mostly team ’ attitude towards project hand. knowing strengths weaknesses team members good start. far learned lot. ’ still intrigued eager learn lot more. feel like becoming real data scientist day passes thanks edsa giving wonderful opportunity .  explore data science academy 19 weeks ( counted ) ; let ’ simplify 3 months shall ? exciting working group challenging problems , feel like world possibilities opened us. exciting overwhelming met many new people. right away seem made name myself. proud , even excited people feel someone come help - practical emotional. helping others role comfortable since typically excelled academic environment , ’ admit disappointment people ’ ask help more. like thought expert ( even like feel like expert , find happens less grow older ( experienced ? wiser ? one old greeks said something wisdom knowing ’ know ) . aspect larger group find overwhelming trying remember everyone , especially since many people seem know am. enjoy learning people ’ backgrounds amazed great selection people attending college. experience working intensely type team entirely new. worked teams school , university work , never amount diversity team. never sitting next team members everyday ; majority one ’ efforts focussed team work. immediate challenge first team working people skills ways operating different own. realised career worked people , background training. made less adaptable less tolerant different approaches problem. trying learn relax give people ’ method approaching problems chance. felt comfortable group leader first sprint , uncomfortable giving reigns second sprint onwards. second project usurped actual leader number occasions seldom pulled back. formed real bond first team , despite ( ) struggles. second team relationship seems formal , although things went much smoothly team arguably effective. found frustrating deal reserved people. hard read difficult reserve judgement ’ produce tangible outputs. still trying figure best deal people , draw shell patience hear trying say. also need consciously tell offer may valuable ( remain unconvinced evidence right table ) . experience also underlined much rely external encouragement praise , played interactions students staff. enjoy teaching students , specifically one one small groups , alot immediate feedback. first weeks felt really supported facilitators , later weeks felt like abandoned newer , exciting projects. similarly felt much secure first mentor ’ hands-on , available , approach second facilitator ’ reserved , hands-off approach .  young age always sheltered , reserved terrified getting comfort zone. knew year would challenge ’ challenged break shell. first day got real knew one person hundred. push open interact crowd ( struggle considering ’ shy standing front people makes extremely nervous ) . lucky people academy friendly understanding. working teams never problem , issue able fully center attention speak thoughts. taught importance communicate ideas without shy. one biggest challenge working team dealing teammates ’ fully committed tasks assigned them. personally prefer get things done certain way bit challenge team timelines ’ met team members ’ putting much effort everyone else. lab exposes us different personalities one needs learn work together effectively many personalities many conflicting personalities could recipe disaster. could say lucky teams assigned ’ difficult. understood importance contribution communication. misunderstanding ’ address becomes conflict , way able prevent it. teams decision-making process pretty much same. generally , challenge put team , communicate strengths weaknesses , able distribute tasks together based strengths weaknesses. decisions need made , every member given opportunity give opinion decide together team taking consideration every member 's contribution. overall , experience far great. learning new things everyday. workload get challenging time time managed. slowly coming shell , talking , interacting opening .  experience academy thus far potent insightful year , changed life become person really enjoys importance cooperating others. brought together amazing minds space really helped acquire soft hard skills short space time , indeed helped elevate way learning new technical subjects efficiently effectively. also , management team large especially supervisors instrumental always improvising make sure led way help us walk away well informed junior data scientists end year , hence work tirelessly making sure break obstacle attractive easily comprehended possible us order put hard work. addition , nuri formidable way helping us cope psychological illnesses caused intensity program , always go extra mile making sure one feels solitary. supporting structure throughout program importance , help us understand essence behind excruciating vulnerability management enable see work not. last words , want close saying means life , 'm grateful opportunity , best thing ever happened life besides varsity , 'm pretty sure set life greater things achievements .  time academy far challenging , yet extremely rewarding self-fulfilling - especially regards working people different backgrounds domains expertise. although meeting different people interesting , concerned bring different people together towards achieving common goal. two teams worked , often one take lead , drawing ambition past experience relation data science related fields. challenging , since used - comfortable - working alone. hold personal contributions team-based projects highest standards , therefore difficult team members show dedication team-based projects do. working teams challenged personality positive negative ways. like control , therefore difficult allow people take lead different aspects projects , especially objectively feel experience completing particular objective. however , learned impossible control aspects project therefore welcomed aided contributions team members shown willingness aptitude contribute. contrast , difficult form motivate team members show apathy towards achieving common goal. although somewhat frustrating , ensured never derailed team following initial plan objectives. felt comfortable role coordinator , relished opportunity deliver high-quality deliverables high pressure situations. drive work ethic good , always productive worker. over-analysed spent excessive amounts time subjects interesting challenging personal level , instead focusing time pressing matters. generally tend spend time challenging interesting concepts , tend postpone often neglect routine administrative tasks. creates frantic dash line time runs , creates unnecessary anxiety. conscious shortcoming , actively try remind prioritise complete pressing matters regardless personal interest. know often regarded serious fellow team mates , since always focused driven comes task hand. little time chit-chat tend engage conversations related intriguing philosophical subjects. feel comfortable responsive. however , learned casual conversation regarding sports hobbies helps learn team members , forges professional , understanding relationships team members. overall , work experience academy enlightening one , far perfect. lots learn , eager improve aspect career benefit team mates .  hardest moments life learning new things , matter simple easy might be. difficulty met new people academy. truth talkative nature , ’ always found difficult start conversations students anyone academy. however , amazing experience discover learn different students ’ cultural backgrounds diversities , interest data science life whole. academy , realized people learn discover interaction others. new environments nervous , yet exciting ones. explore data science academy best example scenarios. academy provided great exposure working environment challenges ought face work teams. able work team prerequisite skill needed data scientist. learning curve allowed self disciplined , self-motivated introspection little effort group work affect entire team ’ progress thus , results poor performance entire team work. people different cultural , educational programming backgrounds , learning difference right , right right became key determining factor understanding work teams. months academy , feel close becoming data scientist before. hunger efforts put understand data science seem rewarding now. great feeling among first group given opportunity learn data science. although pressure much times , hope believe everything designed meant prepare us real world working environments .  22nd january 2018 , time 7:00. seemed like normal day time. hit , difference location. 'm western cape , cape town precise. moved away home embark journey become data scientist explore data science academy ( edsa ) .today 8th may 2018 lot happened since ... first week got meet students different family background different level education. met explore team , included supervisors , facilitators nuri. told would working groups fives. nervous got used everything own. actually terrified. first group called optimize prime ( still cracks ) . made nervous everyone post matric qualification. beginning thought underdog , realised competition. mean must 've reason teamed guys , learn contribute wherever can. next group , named wolves woodstock ( named movie wolves wallstreet ) , confident engaging. developed previous group. even headed one sub projects. experience amazing great. even sister n't believe me. first group given opportunity work cape town water crisis , investigated demand supply factors affected cape town water crisis. project got use sql powerbi first time. saw another use python n't know capability , analysing data. second group got explore rsa stock market data. used data predict target return individual companies. interesting project accuracy predictions , limitations models. group assigned supervisors. got chance mentored two different supervisors. wish could create table comparison , two different , effective. times stats , python others gets much. group sessions nuri , refresh .  experience edsa great far. massive learning curve academically also personally. took myers-briggs test twice : , last night earlier today got slightly different results. first time came letter combination infp second time isfj. though slightly different , feel accurate overall representation various attributes exhibit. seen diplomat-mediator well ( second time taking test ) defender sentinel. naturally reserved nature , unfriendly , prefer working alone comfortable enough sit alone group people. however , approached , come across friendly. able read various personality types groups fairly quickly , allows quickly adapt different situations needed group times. n't usually take charge , know end goal something achieve , prefer clear cut plan achieve , one challenging parts working group , made need step comfort zone recently make decisions voice myself. times experienced stress due fact felt frustrated able comprehend certain concepts quick enough significant work project , deadlines much issue. overall , feel thoroughly enjoyed learning group dynamics far edsa .  nan  like meeting new people similar interests , many new people , connecting people time goes assigned new group. n't spoken students merely strong introverts , recent group nominated group coordinator take lead every meeting , every interaction acknowledge personal level group members feeling day , something never really thought experienced , even grew every mondays share weekend , really appreciated put position challenge grow personal academically. like make decisions based merits decision efficient come attention efficient time take account people 's feelings , made decisions past group people feel included left , worth noting individuals contribute project way , barely showed academy , never written one single line code left member take care making dashboard , presentation ultimately present , two us took uncomfortable route diplomacy , pleaded every morning attend briefings supervisor us , went group training sessions made impact , next group , understood academy meant worked together well. stallions , magnificent , sudden paradigm changed , enjoyed one idea best things , originally typed minutes meetings another member suggested trello effective , learnt something new , excited , day , well every day went bed knowing something ’ know woke ’ data scientist ask for. one important lesson got working two different groups leave time guide team work flow everyone miserable end project , however everything planned trello ( ’ trello ) ‘ tees ’ crossed ‘ eyes ’ dotted , pleasant experience bring best everyone involved , team creativity grow levels one planned beginning. , irrespective put group work , sit maybe second session “ getting know ” map strategy roles assigned , significant insignificant .  nan  edsa reflections 24 november 2017 , filled joy , excitement anxious time opened email edsa titled “ congratulations ! ’ accepted explore data science academy ! ” fast forward 22 january 2018 , ready meet new people ( possibly form new friendships ) , meet edsa team learn lies ahead 2018 , overwelming day was. today 6 may 2018 , lot happened past 3 months , ’ learned much ! sql , stats , python , power bi , presentation skills… [ still learning , never stops : ) ] . grateful selected one top 100 , ’ appreciative opportunity , thank edsa making possible me. lab data scientists making , greatly enjoy lab , environment condusive learning , group data science enthusiasts one room feel receipe success. training limited online courses edsa staff students learn lot , different backrounds , skills experience making lab interesting fun place grow academically socially. teamwork water crisis project : team liquidata must say initially concerned amount individual work deliver team well. realized importance effective team , working well team reduced work load stress also helpful individual tasks. roses regards team dynamics , working different individuals perceptions opinions challenging first , frustrating decide certain matters , helped lot grounded problem statement kept us focused needed done. worked well ’ result driven given task , completing task successfully required relevant info important , refering back problem statement got stuck great tool us. emotional moments team due personality clashes end worked well contributed success team , awesome team ! rsa stock market project : wolves woodstock accepted ’ work new people , really problem , hope wish strong programmer team , wish realised statistician programmer team , lovely ! great team , felt like outspoken person ( ) bit much , still worked well though delivered. prefer working team different ideas , creativity different skills ’ huge advantage eveyone team result driven , absolutelty appreciate that. ’ good experience teams far .  year huge one change , involving many aspects : leaving stable job , moving cities , moving new field , going back student , long distance taking chance brand new data science academy. quite excited meet new data science students despite usual shyness. people taking similar plunges shared interest field. enjoyed ( slowly ) meeting new people learning backgrounds , interesting varied stories are. working lab new experience. complete self learning tough journey , glad academy experience supervisors help guide us way becoming data scientists. bit unusual area ’ computer lab learning , also sort office , also people socialise. ’ sure nailed exactly be. feel work better home quiet comfortable ( apparently line personality profile ) . group work element year probably one biggest pain points. see intent setting us way , ’ think quite worked yet . group work challenging. throw meeting new people different backgrounds , huge amount new content learned fairly significant stakes ( internship opportunity ) difficult situation. think resulted frustrations running high poor quality work delivered , individual group would wanted. think worrying getting project done sometimes gotten way actually trying learn content. terms decision making groups far , think one positive definitely logic valued field students. state reasoning behind plan solidly ’ problem. think challenge group member interpret instructions priorities differently , sorting get get everyone page take valuable time. personal level noticed always looking clear , decisive decisions plans , deal well “ seeing goes ” . think something need look team setting , expect manager well. overall , still pretty excited year regrets decision join. think academy still bit rough around edges needs refinement , part iterative process right ?"
JtXt

"  first 4 months edsa filled many new experiences ; challenges , others joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people everyone met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates within edsa , provided insight gained experience tools like python & sql. also cherish able build network data scientists tomorrow well established industry professionals.. -- -working together lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious con would noise distraction - least say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience would say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes full responsibility part ) members first group n

In [49]:
ItXt = "  first 4 months edsa filled many new experiences ; challenges , others joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people everyone met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates within edsa , provided insight gained experience tools like python & sql. also cherish able build network data scientists tomorrow well established industry professionals.. -- -working together lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious con would noise distraction - least say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience would say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes full responsibility part ) members first group noobs sql & python , gather analyse phases really exposed technical deficiencies resulted lot frustration came time delivering tasks. experience took comfort zone important lesson patience us all. second group high level experience , strong team co-ordinator team members took full responsibility tasks. even though work complex comfortable team dynamic. -- -how decisions teams made consistent make decisions -- - team decisions groups made deliberation. decisions easier make person presented clear points opinion was. group dynamic strong decision making felt natural , similiar make decision own. -- -how rational emotional decision making process -- - somebody technical knowledge could provide interpretation best solution. problem required research would take place would rely rational logical thinking arrive solution. regarding emotions , used open communication channels facilitators supervisors discuss emotions opposed letting emotions making project decisions. -- -overall -- - previous role electrical designer smitten data analytics within electricity metering sector. edsa provided platform pursue passion broader scope data using market leading tools importantly upgrade skill-set thus opening new opportunities within workplace .  joined academy crossroads sorts life. academy offered opportunity pivot career engineering data science unfairly dismissed struggling find employment result. meeting 100+ ( faculty students ) new people would part life whole year something get used , even though worked multinational companies. ability deal people different backgrounds , beliefs , etc. tested group work. needless say lead tension one 's life team mates pulling weight. team dynamics always learning process/curve , thankfully bad experience till now. needless say , people try cheat system , act dishonestly sometimes plain lazy. many opinionated thus rendered incapable making great contribution. way things always consider everyone 's well-being , opinions contributions. somehow persons think ok cynical unhelpful. end day , supposed learn become data scientists. work , improve daily. .  far experience positive definitely see value course. experience working different people valuable me. worked well attitude everyone team deliver take responsibility. difficult step back trust team work , however cases team members exceeded expectations. felt bit frustrated team members applied taken substandard lazy approach problems. like get stuck something get immediate results improve there. think approach worked well however also learnt allot team members structured task based approach. allot time spent tangents add value projects however tangents either paid results learnings took them. decision making approach generate ideas create structured work flow around ideas distribute work among individuals take interested work flow. conflicting ideas typically took democratic approach combined healthy debate. find build relationships people 'm teams with. get know best. strongest connections team members also people academy learn from. took team bit data collecting tangent last sprint necessarily productive use time could avoided understood problem better. one case probably assertive team , however way like make decisions get stuck workflow stream adapt go n't supervisor direction point. find environment stimulative enjoy going around teams see working on. good relationship people happy share working great. also found environment distracting times , would like help people get satisfaction , however 'm also that. working different people mixing regularly something im excited everyone different learnings experience theres something learn everyone .  fortunate opportunity join academy year. sure career choice , academy provided great chance change career paths. help choosing new path allow eventually reach larger goals life. moving cape town joining academy challenging , already knew two people academy joining. however made bit difficult making new friends. typically reserved , easily make new friends , especially already good friends , case see reason make new friends. working teams academy great challenge me. normally prefer work pace working team forced cooperate people adjust skill levels. sometimes meant taking work allocated me. addition , found difficult working individuals invested projects aiming learning much experience intend to. within teams , leadership quite lacking. typically like forcing onto people , circumstances , forced take lead projects. taking lead tried best lead behind , making sure consensus within team deciding. generally due lack experience one teams , meant think plan , execution thereof , followed team implementing without challenging ideas laid out. approach worked well , would liked greater input team , unsustainable left would render team incapable functioning properly. team , people much competent four members , decisions much easier take. good idea , still elected leader , role much organic time , much balanced consensual way making decisions. however , completion tasks effectively still perfect left. teams struggled making decisions , never experienced conflict generally emotional making decisions .  looking back one got academy right , confidently say grown intrapersonally interpersonally optimistic perception working also gradually shifted realistic end spectrum. came nothing positive outlook journey gotten opportunity embark , however , learned couple valuable lessons - one stand value understanding teammates expecting one understand you. put far put cap assertive nature gotten lot f pressure given opportunity even sharpen softer skills tends help quite lot even outside academy good vibe generator - one deflects negative energy positive energies whatever spheres happen find in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. learned focus making sure ’ expected delivered timely miss beauty process focussing relatively small hurdles ’ vast amount captivating material engage in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. 've also learned never bring emotions workplace people tend step sensitive things without noticing- example someone staring shoulder commenting work 're quite unpleasant kills moral detaching focusing 's required best coping mechanisms .  attending explore data science academy already proven , continues amazing experience. ’ quite astonishing realize much ’ learned mastered past couple months , chief among would ( subject essay ) , teamwork. meeting new people daunting task ( including ) . however academy different start. sheer excitement students especially explore team ( supervisors facilitators ) acted social catalyst , smoothing social awkwardness , knew getting along like house fire. working teams people completely different backgrounds personalities challenging throughout first project , said , also interesting see individual adapts accommodate team help build team dynamic. starting cape town water crisis project honestly idea expect , certainly ’ expect team dynamic play huge role success deliverables. team overcome different levels knowledge , skills , self motivation. couple late nights flared tempers managed get work done , realised overlooked crucial part teamwork. one need self motivation , also create environment team motivate comfortable enough ask help ’ feeling overwhelmed struggling get grips difficult concept. team definitely lacking aspect , take chin regards deliverables , walked away learning priceless lesson moving next sprint. assigned new teams rsa stock market sprint , refreshing hear ’ team struggled certain aspects team dynamics. get go approached structured interactive way things. decided stand meetings every morning report previous day ’ goals well setting new goals day ahead. already made huge difference motivation team. another part success dividing work ’ get overwhelmed individuals rather sharing workload one-on-ones relevant team members gain deeper understanding section work. approach even advantages thought. structure communication team members lead lower stress levels ( even though still ever present ) , resulting cooler tempers clear headed decisions less emotionally-driven first project. goes show “ soft ” skills important technical skills working teams. ’ wait learn brilliant diverse people academy. '' individual commitment group effort -- makes team work , company work , society work , civilization work. '' -- vince lombardi  essay discussing personal experience regards academy like meeting new people especially introverted person am. speak different teams , different different felt , learned different characters. comparing decision making done teams versus personal decision making. getting plane cape town without place stay friends city big risk , certain plan academy kept praying want scam plot get specific building get kidnapped. fairly surprised get meet people walks life , also kind questioning legit , mean hard question , sounded way good true good things learned ’ happen people like us , guess time shine receive blessings. ’ think academy , blessing. said , experience academy ups downs like always ends up.i learned lot , fallen love data science , things small mindedness used consider geek ’ s. learned push , always try learn , practice computers always evolving. since met people different personalities academy , personality clashes occurred 1 biggest lessons , able work different people different personalities. working teams ’ easy thought would first , everyone idea assumed supposed done different opinions could dismissed everyone 's opinion valuable working team , opinions somehow seemed clash. working teams taught patient , quality possess whatsoever. second time changed teams though seemed though everyone gotten idea working teams ’ like , basically dipping toes working teams thing first sprint , second sprint bit confident meant worked better second teams , improving go. teams , split work equally amongst , making sure everyone gets chance tackle important lessons like coding dashboard. group coordinator state views , he/she would speak supervisor decision would made way , supervisor would guide us right direction hearing thought right way.its almost concept use guess making decisions , weigh options , see one best one worst outcome obviously go best outcome. conclusion working teams skill gaining developing everyday , ’ mastered , heck difficult master many personalities , getting .  coming academy best thing ever happened grateful opportunity. learned academy going encourage teamwork excited meet new people different spheres life exchange ideas network it. working teams thus far ok. things like team commitment work work ethic thereof. comes decision making n't enjoy part felt sense dictatorship member would defy group 's decision still want impose best team start delegating work teammates based vision. felt defied purpose brainstorming planning objectives. role played team always conscientize group imperativeness working within scope prioritizing required us form basis going structure project would help decision making. supervisor meetings reporting based `` '' , `` '' . sign individualism collective project perpetrated unnecessary competition. enjoyed networking variety people academy also participate win hackathon hosted city capetown one best networks team collaborations went well me. like random conversations technology trying link 'd connect inspire other. brief summary work prefer plan try much possible stick plan always room changes. consolidation key helps deciding conflicting ideas .  past months one challenging thrilling experience life. coming , expected engage people knew introvert , experience academy would force engaging people. something totally outside comfort zone. ready ; much prepared , ready challenged understood growth reached uncomfortable spaces. experience taught could imagined. learnt fearless embrace greatness within me. come across shy , willingness push things enabled speak room full people made realize get nervous things actual fact get nervous things capable also fail. way scared inabilities potential. understand still lot learn fully comfortable know feel still lot knowledge gained. coming bit theoretical statistical background , interesting see techniques learnt university applied real world scenarios. coming space , space full well equipped individuals , felt bit intimidated idea career going lacked confidence really good at. experience far definitely given better understanding skill set career take , importantly come learn alone. able meet lots people also similar situation , simply shown worries worth worrying things fall place. environment quite relaxed , yet taught behave professional workplace. support peers also tremendous. observing everyday events supervisors taught teamwork , people come together get things done . although sometimes remind use inside voice , feel adapting environment relatively well. going forward , expect learn develop skills .  experiences explore data science academy experience meeting lots new people great one , months already learned lot , continue daily level. terms meeting new people much gain , especially terms offered different perspectives , challenged way thinking. also taken comfort zone turn taught lot handle different personalities. experience working together lab teams much appreciated close real world experience , 've already similar environment throughout university always found enjoyable interact people. level productivity performance exponentially increased , mainly deadlines needed meet wanting disappoint teammates , generally working others helps achieving individual. say interpersonal skills increased , example speaking listening others. come realize collaborating others helped identify strengths weaknesses , basically self-aware. experience decisions made teams was/am consistent like make decisions. lot people considerate want attain goal , appreciated , far people worked take time listen person say try take consideration relevant concise , decisions made depending everyone felt. experience rational emotional decision-making process been. decisions made rational group took time take people ’ emotions consideration possible. ’ obvious always agree completely certain things , occured , time taken explain person things certain way beneficial way disagreeing member thought. accepting possibility incorrect important role , especially comes working others. conclusion , experience positive anything academy .  attending academy first time like first year , however difference everyone age. idea meeting people really cool however issue forced work people things tend bit shaky everyone standards. make matters interesting grades directly dependent interaction team. varsity must admit best experiences working group tend find similar roles characters. however bit optimistic given diverse ages academy assumption folk work experience hopefully mature. furthermore , come realize establishing solid authentic team dynamics 's also lot stressful actually may seem without trying fake sake team trying accommodating. real issue trust decisions tends made around dominant personalities perspective may particularly best interest team whole. although always agree approach would agree fundamentally understanding skill set equally distributed also end day delivery project. overall happy rational decision making process. divide trust conquer works best teams something come learn. within teams lucky may find someone work ethic dedicated case would likely help could find someone understands work complements style. think initially everyone problem admitting n't get something deliverables due. journey emotional especially towards end tend give less less input become concerned delivery .  good fun experience though 's often case pressure. lab allowed enough time practice get used coding , improved efficiency coding working computer. coming academy n't used idea working team , along year got better , knowing someone count , rely go need help. also got know personal level makes easy understand , strength weaknesses allows us form strong team. sometimes disagreements think right , effective solving , lucky enough co-operative team members since beginning year. though n't got know many colleagues ones got know would say pleasure meeting  experience explore data science academy great far. goals explore year develop & master skills data science well improve leadership skills. felt empowered content ’ taught. four months , believe already learnt skills would allow enter & succeed job market. next 7 months plan spend refining data science skills using 1 hour evenings revising content learnt class. regards second goal ( improve leadership skills ) , ’ fortune team great coordinator & excellent team dynamics. also great team coordinator team. ’ taken many lessons experiences read book. given practical lessons improve leadership skills. ’ two teams far , worked first 1. coordinator nudged us always making progress towards bigger goal. 2. teammates voluntarily picked segment project would responsible for. 3. loose deadlines agreed , deadlines also allowances members equip skills required fulfil parts project. 4. team met twice week discuss progress. 5. time member grew comfortable asking help others well given feedback. team dynamic encouraged member give part project all. inevitably , team members technically advanced others ended taking advanced work rest , however time felt like overburden. given positive experience eager also try coordinator next sprint came around. new teams assigned & volunteered coordinate. aspects believe could done better. 1. ’ commonly accepted bigger goal. 2. ’ involve team identifying small goals were. worked work plan overnight & presented team. team loved plan , individual responsibilities work assigned. 3. led situation members team clear responsibilities were. 4. result power leadership , members team ended bulk work. able pull together & present satisfactory project , however believe could led better & helped team deliver far better project. member team would also proud contributed fully project. learnt great deal experience & would like give coordinating another try , like skill believe leadership learnt practice .  explore data science academy , road never thought would take lifetime. love passion cars , accepted data science internship one last thing never saw coming. love travelling exploring new avenues `` let 's '' tip. first day coming explore data science academy ( edsa ) beyond expectations , visually appealing facilities provided us. first could believe , cape town , comfort zone , pushing something involve anything studied. came expectations , however , told something need give order leave experience back future. speech supervisors gave us 22nd january reassured make mistake coming here. first week given work , install softwares would using throughout year , thought would pre-installed arrived. first , introvert side steering wheel interact lot people besides housemate. started interacting people water crisis sprint started. team people different background ; bioinformatics sciences , computer science , economic statistics mix , different parts south africa ( mostly gauteng ) . one things stood type people would start conversations loud side come order make everybody feel comfortable. first week gather sprint took team start making progress , thursday already figured system worked everyone order use something show supervisor. put weaknesses strengths table , started communicating better team. results shown supervisor told us think would deliver delivered , however , delivered end fundamental , faith would best one best. love good music , adventure socializing led connecting people group. time fundamentals spent helping people , building friendships would carry edsa year even greater , learning languages like python sql , channeling mind data scientist mode. weekends colleagues would tour cape town going beach , hike mountains could see edsa lab , visiting malls theme parks around cape town. python , one complex languages learn. well , thought since programming background. two teammates experience working languages , changing one language another , fairly doable. advantage since two people could consult whenever ran problem python. presentation fundamental one laid back fundamentals face since preplanned wanted add slides clear storyline. fact split fundamentals tore us apart , however , vowed exit whatsapp group created. review week , weekend , people never talked friends edsa lab suggested go theme park near canal walk mall take minds water crisis sprint. character part outting shy , yet interesting backgrounds. still friends them. stock market sprint came flew by. really feeling team since lot judgemental people felt like knew better. lacked communication , managed pull find sprint one piece. events bcx hosted highlights first months edsa , amount inpirational people engaged events. felt like meant every time spoke lucky waiting us greater wildest dreams. week takes cup. never expected go home least july. honestly grateful spending year edsa lab , could chose anything else .  academy amazing , love petry dish talent ! handful genius characters here.. backgrounds unique stories , explore data science academy best place develop data analysis skills. love time ! far , 've experienced whole lot pressure change something better. spend time studying , better feel , comfortable become closer get quality employee entrepreneur. everyday come office , feel home , learn something new.. something directly use business application. look around entire building get inspired start business 've learned here. speak among business academy helping us realize ideas clearly. without information back ideas , 's difficult starting. availability information outstanding. really appreciate nest. last two teams great work with. achieved bunch , , thanks resources available. first sprint tough though. team , realized strengths taught other. n't problem sql python used technologies before. understanding statistics applications data given difficult. worst , water crisis data n't collected well. n't worst experience data taught things career perfect. perfect skills data cleaning. realize limited number solutions data analysis. perfect outcomes data science , could breeze work future. believe hyper prepared things.. game theory data science would prove beneficial. emotionally , 'm great. academy brings best worst skills provides controlled environment mend holes analytical judgement. relief , emotionally physically becomes process bettering daily , bit bit without forcing stress .  experience edsa far combination challenging , fascinating , unpredictable. coming academy , excited would get meet new people learn new skills ; anticipated might somewhat difficult engaging new people ( since introverted ) coming grips new technical ideas ( creative technophile ) , since loved good challenge , still excited immerse experience. surprise , challenges faced none anticipated generally able interact cordially new people understand technicalities program. greatest challenge far working lab. environment many people ( different personalities , habits , learning techniques preferences ) forced natural inclination isolate working. easily distracted , think carefully navigate space simulate environment conducive learning. also found team consider optimally ration work , challenging , especially beginning n't yet good understanding 's strengths weaknesses. said , gotten know fairly well , able make decisions believed would amplify strengths diminish weaknesses could produce results best possible quality. sense , much decision-making teams rational rather emotional. would apply logic reason wanted achieve thought might able , found always room empathetic thoughtful ( thereby engaging emotions rather reason ) , sake team member , alter original plan. somewhat consistent personally make decisions : think times base decisions think yield best results terms trying achieve , even happens inconvenient taxing process ; times , especially decision made great consequence , make decisions based emotions believe 'rational ' 'logical ' decision likely hurt someone 's feelings cast bad light. , learned awful lot : preferences , strengths weaknesses , character ... found learning curve steep emotions rather turbulent throughout process , enjoyed every part challenge could mean future -- prospect holds together worst days exhilarates best days .  experience edsa interesting one thus far. pushed get shell interact people. since academy realised smarter think lot valuable knowledge insight share. biggest fear coming academy social aspect interact different people , since able overcome fear. first placed groups literally sweating , heart rate high felt sick stomach idea group strangers. second time split new groups ’ nervous. become social enjoy meeting working new people now. working groups challenge me. frustrating everyone different everyone different approach learning process project management. ’ learned lot working others. took backseat things even knew better way things. spoke saw helped team steer team better direction. become assertive. thought passive person ’ assertive save life , think assertive ’ find daunting. biggest frustration working teams dealing people think know everything ’ willing open learning something new fellow teammates. person like previous group. person would dictate team ’ want collaborate team , dominating personality rest us shy. realised none us ’ speak ’ fail sprint forced breakout shyness. think still fairly shy comes work fight shyness. decisions previous group initially made one person. prefer problems discussed everyone team different possible solutions explored come collaborative solution. found current team. still issues , people letting pride emotions get way work , people distracted ’ committed project. working labs challenge. lot distractions noise focus easily , prefer work home , get much done home. people come academy socialise anything distracting .  past 5 months explore academy challenging. make great adjustments order here. positive negative experiences. one positive experiences much learned. met lot people highly intelligent interests. worked 2 different teams. prefer work alone working team makes sense due sheer volume work. first team experience incredibly frustrating. team members highly introverted technically strong made discussions assigning tasks difficult. even tasks assigned accountability meet deadlines. prefer chip away steadily work opposed rushing close deadline finish. working second team much better. met regularly , communicated lot. tasks assigned everyone accountable. team members helpful towards .  experiences academy far life-changing say least. meeting smart , passionate , hardworking ambitious people different backgrounds happen many similar interests mine interesting journey. introvert prefers working alone pace within space , quickly find way adapt polish social skills order socialize 99 strangers going learning data science with. understood word go need socialize form meaningful relationships fellow interns could see forming strong network may helpful many years come career , long internship. academy ’ emphasis importance team work concept took warm to. teams exposing different individuals varying academic , social , work geographical background different personalities : introverts , extroverts , intuitive , artistic even really funny characters. teammates phenomenal experienced fields previous exposure to. learned lot them. equally ’ also encountered independent disagreeeable teammates required adjust levels patience , tolerance understanding may necessarily share similar beliefs me. need patient tolerant people ’ points view approaches solving completing team tasks. teams , due differing views , opinions approaches use democracy ( voting ) order reach decisions within groups. although easy one resort taking emotionally-driven decisions due one ’ firm beliefs ways things high level emotional intelligence critical favourable team dynamics. academy based cape town , city ’ visited holidays before. moving cape town challenges required solid ‘ family ’ away family fortunately found fellow interns resourceful many ways even outside ‘ labs ’ . example group share information accomodation , transport around city , social events extramural activities. chance test team-building skills virtues regular group sessions nuri believe helpful giving idea relate people less formal settings. credit due supervisors , facilitators oprerations officer ; helpful making internship experience manageable definitely one memorable years come. commitment programme makes easier people personalities settle nicely within diverse small community .  experience academy exceptional. meeting new people amazing , specifically working them. much challenging first inception relate , time things gets better begin know get use one another. odds events one two group member want participate ( eg. missing meetings ) thing way oppose agreed group , really problem participating also productive , especially meet group expectations group works , members end work. means group take collaborative decision planning expect everyone deliver agreed planned. lab one best way work , especially comes group work , find collaborative useful. 's unfortunate group members choose work home leave early come late. found behavior hinder progress , especially tasks divided sub groups among group members. critical aspect group always communication. members usually communicate inability complete task late work due. members get emotional approach , end day always encourage one another see right side approach task project hand. working many different people taught rely others , means trust member work , even better me. fun leaning since day one .  social perspective , experiences academy refreshing impact personal development. prefer things myself. reflected interest gaming , reading fan-fiction writing. however , willing interact others necessary. enjoy paintball friends far sitting lounge rather making casual small talk. applies workplace well essentially things together. even talking , inevitably get topics related coding such. generally people ( outside academy known years ) seem identify introvert give misconception poor communication skills n't always case. many issues communicating team mates others class regarding various topics ranging work-related hobbies ( especially sport , gaming anime ) . prefer finish roleplaying video game lounge talk also like talking science nerdy things. words , need socialise , problems so. intuition observation valuable traits workplace data scientist team-based project. working teams allowed utilise both. found often analysis others n't done apply intuition. managed thing. different academics would essentially everything myself. regard , team-based nature course done well. often think good one working others stimulate us potentially become good trait previously thought n't naturally suited to. case , working teams sort force us find ways overcome weakness make strength instead. said , team dynamics sometimes involved lack coordinated planning among team members. good keep things open , applied plan rather plan. decide certain approach solving problem involves coding , dashboard , n't mean ca n't improvise. agree improvisation ultimate way things. make plan rely open-minded thinking plan n't appear working , improvise. generally experience academy , emotion seems rule roost logic seen potential barrier team work cooperation. mean students end becoming dependent affect teamwork negatively long term. overall course excelled bringing personality traits relevance workplace , providing opportunities overcome weaknesses utilise strengths instead .  academy challenging experience. expected prior coding knowledge help minimal help , greatest challenge statistics. slowing getting point n't actually try understand concept thoroughly rather get basic theory focus application. overall , insightful journey , learning interact different personalities group dynamics. really excited leaning nlp regression incorporate app development monetize it. genuinely regret applying academy although days doubt hurdles remembering greatness opportunity always great motivator .  big part joining academy year meeting working lots new people. unexpectedly positive experience. despite everyone different backgrounds , sense camaraderie interns friendly. even though everyone tested graded environment non-competitive people willing help other. made enjoyable work lab. however , noise bustle lab sometimes overwhelming draining find productive sometimes spend hour two day lab working home quiet. challenging aspect year far teamwork. first project ( cape town water crisis ) times conflict team due team member contributing much therefore shifting workload team members. led resentment team members , expected. team projects work rarely equally divided team managed complete project despite difficulties. reflecting experience grateful done big share work gave opportunity learn lot would otherwise. second project ( jse stock market sprint ) much shorter time span therefore team approached project different. team divide work time everyone involved deliverables. hard meant trusting people work without directly involved dependant complete it. process decision making teams democratic. team discusses problems possible approaches usually obvious decision everyone agrees on. decision making far tended rational team members emotional. emotional part projects stress faced final deadlines neared. , experience academy far positive learnt lot. hope improve way approach team work upcoming sprints .  ‘ greatest opportunity part 100 chosen individuals wonderful programme. explore data science academy ( edsa ) environment makes feel welcome perfect terms collaboration networking different individuals interesting views personalities. ’ hard distinguish edsa work environment. type environment prepare real job. edsa provides one opportunity learn work diverse group people terms career background , culture religion. working groups different individuals project exciting. throughout time , ’ learned lot working teams. first ’ learned team work achieve terms results learning new things. secondly , come decision making , everyone opinion coming final decision another thing. learned ’ voicing opinion , convincing fellow team members indeed opinion fruitful. comes team strategies embark particular project , ’ one size fits situations. team come strategy going solve assigned problem. difficult , especially serve team coordinator serve , time complete different team whole set different personalities characters. instance learned strategy based skills , capabilities mostly team ’ attitude towards project hand. knowing strengths weaknesses team members good start. far learned lot. ’ still intrigued eager learn lot more. feel like becoming real data scientist day passes thanks edsa giving wonderful opportunity .  explore data science academy 19 weeks ( counted ) ; let ’ simplify 3 months shall ? exciting working group challenging problems , feel like world possibilities opened us. exciting overwhelming met many new people. right away seem made name myself. proud , even excited people feel someone come help - practical emotional. helping others role comfortable since typically excelled academic environment , ’ admit disappointment people ’ ask help more. like thought expert ( even like feel like expert , find happens less grow older ( experienced ? wiser ? one old greeks said something wisdom knowing ’ know ) . aspect larger group find overwhelming trying remember everyone , especially since many people seem know am. enjoy learning people ’ backgrounds amazed great selection people attending college. experience working intensely type team entirely new. worked teams school , university work , never amount diversity team. never sitting next team members everyday ; majority one ’ efforts focussed team work. immediate challenge first team working people skills ways operating different own. realised career worked people , background training. made less adaptable less tolerant different approaches problem. trying learn relax give people ’ method approaching problems chance. felt comfortable group leader first sprint , uncomfortable giving reigns second sprint onwards. second project usurped actual leader number occasions seldom pulled back. formed real bond first team , despite ( ) struggles. second team relationship seems formal , although things went much smoothly team arguably effective. found frustrating deal reserved people. hard read difficult reserve judgement ’ produce tangible outputs. still trying figure best deal people , draw shell patience hear trying say. also need consciously tell offer may valuable ( remain unconvinced evidence right table ) . experience also underlined much rely external encouragement praise , played interactions students staff. enjoy teaching students , specifically one one small groups , alot immediate feedback. first weeks felt really supported facilitators , later weeks felt like abandoned newer , exciting projects. similarly felt much secure first mentor ’ hands-on , available , approach second facilitator ’ reserved , hands-off approach .  young age always sheltered , reserved terrified getting comfort zone. knew year would challenge ’ challenged break shell. first day got real knew one person hundred. push open interact crowd ( struggle considering ’ shy standing front people makes extremely nervous ) . lucky people academy friendly understanding. working teams never problem , issue able fully center attention speak thoughts. taught importance communicate ideas without shy. one biggest challenge working team dealing teammates ’ fully committed tasks assigned them. personally prefer get things done certain way bit challenge team timelines ’ met team members ’ putting much effort everyone else. lab exposes us different personalities one needs learn work together effectively many personalities many conflicting personalities could recipe disaster. could say lucky teams assigned ’ difficult. understood importance contribution communication. misunderstanding ’ address becomes conflict , way able prevent it. teams decision-making process pretty much same. generally , challenge put team , communicate strengths weaknesses , able distribute tasks together based strengths weaknesses. decisions need made , every member given opportunity give opinion decide together team taking consideration every member 's contribution. overall , experience far great. learning new things everyday. workload get challenging time time managed. slowly coming shell , talking , interacting opening .  experience academy thus far potent insightful year , changed life become person really enjoys importance cooperating others. brought together amazing minds space really helped acquire soft hard skills short space time , indeed helped elevate way learning new technical subjects efficiently effectively. also , management team large especially supervisors instrumental always improvising make sure led way help us walk away well informed junior data scientists end year , hence work tirelessly making sure break obstacle attractive easily comprehended possible us order put hard work. addition , nuri formidable way helping us cope psychological illnesses caused intensity program , always go extra mile making sure one feels solitary. supporting structure throughout program importance , help us understand essence behind excruciating vulnerability management enable see work not. last words , want close saying means life , 'm grateful opportunity , best thing ever happened life besides varsity , 'm pretty sure set life greater things achievements .  time academy far challenging , yet extremely rewarding self-fulfilling - especially regards working people different backgrounds domains expertise. although meeting different people interesting , concerned bring different people together towards achieving common goal. two teams worked , often one take lead , drawing ambition past experience relation data science related fields. challenging , since used - comfortable - working alone. hold personal contributions team-based projects highest standards , therefore difficult team members show dedication team-based projects do. working teams challenged personality positive negative ways. like control , therefore difficult allow people take lead different aspects projects , especially objectively feel experience completing particular objective. however , learned impossible control aspects project therefore welcomed aided contributions team members shown willingness aptitude contribute. contrast , difficult form motivate team members show apathy towards achieving common goal. although somewhat frustrating , ensured never derailed team following initial plan objectives. felt comfortable role coordinator , relished opportunity deliver high-quality deliverables high pressure situations. drive work ethic good , always productive worker. over-analysed spent excessive amounts time subjects interesting challenging personal level , instead focusing time pressing matters. generally tend spend time challenging interesting concepts , tend postpone often neglect routine administrative tasks. creates frantic dash line time runs , creates unnecessary anxiety. conscious shortcoming , actively try remind prioritise complete pressing matters regardless personal interest. know often regarded serious fellow team mates , since always focused driven comes task hand. little time chit-chat tend engage conversations related intriguing philosophical subjects. feel comfortable responsive. however , learned casual conversation regarding sports hobbies helps learn team members , forges professional , understanding relationships team members. overall , work experience academy enlightening one , far perfect. lots learn , eager improve aspect career benefit team mates .  hardest moments life learning new things , matter simple easy might be. difficulty met new people academy. truth talkative nature , ’ always found difficult start conversations students anyone academy. however , amazing experience discover learn different students ’ cultural backgrounds diversities , interest data science life whole. academy , realized people learn discover interaction others. new environments nervous , yet exciting ones. explore data science academy best example scenarios. academy provided great exposure working environment challenges ought face work teams. able work team prerequisite skill needed data scientist. learning curve allowed self disciplined , self-motivated introspection little effort group work affect entire team ’ progress thus , results poor performance entire team work. people different cultural , educational programming backgrounds , learning difference right , right right became key determining factor understanding work teams. months academy , feel close becoming data scientist before. hunger efforts put understand data science seem rewarding now. great feeling among first group given opportunity learn data science. although pressure much times , hope believe everything designed meant prepare us real world working environments .  22nd january 2018 , time 7:00. seemed like normal day time. hit , difference location. 'm western cape , cape town precise. moved away home embark journey become data scientist explore data science academy ( edsa ) .today 8th may 2018 lot happened since ... first week got meet students different family background different level education. met explore team , included supervisors , facilitators nuri. told would working groups fives. nervous got used everything own. actually terrified. first group called optimize prime ( still cracks ) . made nervous everyone post matric qualification. beginning thought underdog , realised competition. mean must 've reason teamed guys , learn contribute wherever can. next group , named wolves woodstock ( named movie wolves wallstreet ) , confident engaging. developed previous group. even headed one sub projects. experience amazing great. even sister n't believe me. first group given opportunity work cape town water crisis , investigated demand supply factors affected cape town water crisis. project got use sql powerbi first time. saw another use python n't know capability , analysing data. second group got explore rsa stock market data. used data predict target return individual companies. interesting project accuracy predictions , limitations models. group assigned supervisors. got chance mentored two different supervisors. wish could create table comparison , two different , effective. times stats , python others gets much. group sessions nuri , refresh .  experience edsa great far. massive learning curve academically also personally. took myers-briggs test twice : , last night earlier today got slightly different results. first time came letter combination infp second time isfj. though slightly different , feel accurate overall representation various attributes exhibit. seen diplomat-mediator well ( second time taking test ) defender sentinel. naturally reserved nature , unfriendly , prefer working alone comfortable enough sit alone group people. however , approached , come across friendly. able read various personality types groups fairly quickly , allows quickly adapt different situations needed group times. n't usually take charge , know end goal something achieve , prefer clear cut plan achieve , one challenging parts working group , made need step comfort zone recently make decisions voice myself. times experienced stress due fact felt frustrated able comprehend certain concepts quick enough significant work project , deadlines much issue. overall , feel thoroughly enjoyed learning group dynamics far edsa .  nan  like meeting new people similar interests , many new people , connecting people time goes assigned new group. n't spoken students merely strong introverts , recent group nominated group coordinator take lead every meeting , every interaction acknowledge personal level group members feeling day , something never really thought experienced , even grew every mondays share weekend , really appreciated put position challenge grow personal academically. like make decisions based merits decision efficient come attention efficient time take account people 's feelings , made decisions past group people feel included left , worth noting individuals contribute project way , barely showed academy , never written one single line code left member take care making dashboard , presentation ultimately present , two us took uncomfortable route diplomacy , pleaded every morning attend briefings supervisor us , went group training sessions made impact , next group , understood academy meant worked together well. stallions , magnificent , sudden paradigm changed , enjoyed one idea best things , originally typed minutes meetings another member suggested trello effective , learnt something new , excited , day , well every day went bed knowing something ’ know woke ’ data scientist ask for. one important lesson got working two different groups leave time guide team work flow everyone miserable end project , however everything planned trello ( ’ trello ) ‘ tees ’ crossed ‘ eyes ’ dotted , pleasant experience bring best everyone involved , team creativity grow levels one planned beginning. , irrespective put group work , sit maybe second session “ getting know ” map strategy roles assigned , significant insignificant .  nan  edsa reflections 24 november 2017 , filled joy , excitement anxious time opened email edsa titled “ congratulations ! ’ accepted explore data science academy ! ” fast forward 22 january 2018 , ready meet new people ( possibly form new friendships ) , meet edsa team learn lies ahead 2018 , overwelming day was. today 6 may 2018 , lot happened past 3 months , ’ learned much ! sql , stats , python , power bi , presentation skills… [ still learning , never stops : ) ] . grateful selected one top 100 , ’ appreciative opportunity , thank edsa making possible me. lab data scientists making , greatly enjoy lab , environment condusive learning , group data science enthusiasts one room feel receipe success. training limited online courses edsa staff students learn lot , different backrounds , skills experience making lab interesting fun place grow academically socially. teamwork water crisis project : team liquidata must say initially concerned amount individual work deliver team well. realized importance effective team , working well team reduced work load stress also helpful individual tasks. roses regards team dynamics , working different individuals perceptions opinions challenging first , frustrating decide certain matters , helped lot grounded problem statement kept us focused needed done. worked well ’ result driven given task , completing task successfully required relevant info important , refering back problem statement got stuck great tool us. emotional moments team due personality clashes end worked well contributed success team , awesome team ! rsa stock market project : wolves woodstock accepted ’ work new people , really problem , hope wish strong programmer team , wish realised statistician programmer team , lovely ! great team , felt like outspoken person ( ) bit much , still worked well though delivered. prefer working team different ideas , creativity different skills ’ huge advantage eveyone team result driven , absolutelty appreciate that. ’ good experience teams far .  year huge one change , involving many aspects : leaving stable job , moving cities , moving new field , going back student , long distance taking chance brand new data science academy. quite excited meet new data science students despite usual shyness. people taking similar plunges shared interest field. enjoyed ( slowly ) meeting new people learning backgrounds , interesting varied stories are. working lab new experience. complete self learning tough journey , glad academy experience supervisors help guide us way becoming data scientists. bit unusual area ’ computer lab learning , also sort office , also people socialise. ’ sure nailed exactly be. feel work better home quiet comfortable ( apparently line personality profile ) . group work element year probably one biggest pain points. see intent setting us way , ’ think quite worked yet . group work challenging. throw meeting new people different backgrounds , huge amount new content learned fairly significant stakes ( internship opportunity ) difficult situation. think resulted frustrations running high poor quality work delivered , individual group would wanted. think worrying getting project done sometimes gotten way actually trying learn content. terms decision making groups far , think one positive definitely logic valued field students. state reasoning behind plan solidly ’ problem. think challenge group member interpret instructions priorities differently , sorting get get everyone page take valuable time. personal level noticed always looking clear , decisive decisions plans , deal well “ seeing goes ” . think something need look team setting , expect manager well. overall , still pretty excited year regrets decision join. think academy still bit rough around edges needs refinement , part iterative process right ?"


In [60]:
EtXt = "  first 4 months edsa filled many new experiences ; challenges , others joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people everyone met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates within edsa , provided insight gained experience tools like python & sql. also cherish able build network data scientists tomorrow well established industry professionals.. -- -working together lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious con would noise distraction - least say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience would say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes full responsibility part ) members first group noobs sql & python , gather analyse phases really exposed technical deficiencies resulted lot frustration came time delivering tasks. experience took comfort zone important lesson patience us all. second group high level experience , strong team co-ordinator team members took full responsibility tasks. even though work complex comfortable team dynamic. -- -how decisions teams made consistent make decisions -- - team decisions groups made deliberation. decisions easier make person presented clear points opinion was. group dynamic strong decision making felt natural , similiar make decision own. -- -how rational emotional decision making process -- - somebody technical knowledge could provide interpretation best solution. problem required research would take place would rely rational logical thinking arrive solution. regarding emotions , used open communication channels facilitators supervisors discuss emotions opposed letting emotions making project decisions. -- -overall -- - previous role electrical designer smitten data analytics within electricity metering sector. edsa provided platform pursue passion broader scope data using market leading tools importantly upgrade skill-set thus opening new opportunities within workplace .  looking back one got academy right , confidently say grown intrapersonally interpersonally optimistic perception working also gradually shifted realistic end spectrum. came nothing positive outlook journey gotten opportunity embark , however , learned couple valuable lessons - one stand value understanding teammates expecting one understand you. put far put cap assertive nature gotten lot f pressure given opportunity even sharpen softer skills tends help quite lot even outside academy good vibe generator - one deflects negative energy positive energies whatever spheres happen find in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. learned focus making sure ’ expected delivered timely miss beauty process focussing relatively small hurdles ’ vast amount captivating material engage in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. 've also learned never bring emotions workplace people tend step sensitive things without noticing- example someone staring shoulder commenting work 're quite unpleasant kills moral detaching focusing 's required best coping mechanisms .  attending explore data science academy already proven , continues amazing experience. ’ quite astonishing realize much ’ learned mastered past couple months , chief among would ( subject essay ) , teamwork. meeting new people daunting task ( including ) . however academy different start. sheer excitement students especially explore team ( supervisors facilitators ) acted social catalyst , smoothing social awkwardness , knew getting along like house fire. working teams people completely different backgrounds personalities challenging throughout first project , said , also interesting see individual adapts accommodate team help build team dynamic. starting cape town water crisis project honestly idea expect , certainly ’ expect team dynamic play huge role success deliverables. team overcome different levels knowledge , skills , self motivation. couple late nights flared tempers managed get work done , realised overlooked crucial part teamwork. one need self motivation , also create environment team motivate comfortable enough ask help ’ feeling overwhelmed struggling get grips difficult concept. team definitely lacking aspect , take chin regards deliverables , walked away learning priceless lesson moving next sprint. assigned new teams rsa stock market sprint , refreshing hear ’ team struggled certain aspects team dynamics. get go approached structured interactive way things. decided stand meetings every morning report previous day ’ goals well setting new goals day ahead. already made huge difference motivation team. another part success dividing work ’ get overwhelmed individuals rather sharing workload one-on-ones relevant team members gain deeper understanding section work. approach even advantages thought. structure communication team members lead lower stress levels ( even though still ever present ) , resulting cooler tempers clear headed decisions less emotionally-driven first project. goes show “ soft ” skills important technical skills working teams. ’ wait learn brilliant diverse people academy. '' individual commitment group effort -- makes team work , company work , society work , civilization work. '' -- vince lombardi  wonderful place , place waking morning easy task great desire learning something new everyday. meeting new people great thing learn work team also socialize since person well crowd. actually first time working lab like reason experience labs quite instrumentals labs testing experiments computers , n't computer labs first time coding lab reason coming edsa first choice . skills data scientist one drove apply wan na learn coding. learned lot lab also working team. one main problem faced working team group rather , groups perfect might wish change disaster.sometimes set time slots group meet discuss progress deliverable surprised one member pitch thought lot work , disappointing would mostly prefer working alone future. working team sometimes emotional draining decision divide conquer could work together well. hoping great team smart people need told since every sprint `` lists '' .  essay discussing personal experience regards academy like meeting new people especially introverted person am. speak different teams , different different felt , learned different characters. comparing decision making done teams versus personal decision making. getting plane cape town without place stay friends city big risk , certain plan academy kept praying want scam plot get specific building get kidnapped. fairly surprised get meet people walks life , also kind questioning legit , mean hard question , sounded way good true good things learned ’ happen people like us , guess time shine receive blessings. ’ think academy , blessing. said , experience academy ups downs like always ends up.i learned lot , fallen love data science , things small mindedness used consider geek ’ s. learned push , always try learn , practice computers always evolving. since met people different personalities academy , personality clashes occurred 1 biggest lessons , able work different people different personalities. working teams ’ easy thought would first , everyone idea assumed supposed done different opinions could dismissed everyone 's opinion valuable working team , opinions somehow seemed clash. working teams taught patient , quality possess whatsoever. second time changed teams though seemed though everyone gotten idea working teams ’ like , basically dipping toes working teams thing first sprint , second sprint bit confident meant worked better second teams , improving go. teams , split work equally amongst , making sure everyone gets chance tackle important lessons like coding dashboard. group coordinator state views , he/she would speak supervisor decision would made way , supervisor would guide us right direction hearing thought right way.its almost concept use guess making decisions , weigh options , see one best one worst outcome obviously go best outcome. conclusion working teams skill gaining developing everyday , ’ mastered , heck difficult master many personalities , getting .  experience academy far stressful yet fun. 'm one enjoys working groups , 'd working times. starting first sprint reminded prefer working alone. working group certain people carry weight , even frustrating. however , blame speak let concerns heard. end us saw way tackle task hand work towards finishing assignments given. last group complete opposite. three weeks complete sprint therefore knew work collective. tasks split one accountable part project. ended working well together got along outside work well. work previous group ? know , look forward working different people different dynamics .  experience academy enjoyable far. learning lot interesting valuable skills meeting many intelligent interesting people. meeting working new people lot fun rewarding experience. working lab challenging. open plan allows many diverse unusual conversions easily started joined also prevent work done people interrupt prevent others entering staying state flow necessary learning. especially destructive slower learners like myself. working teams also challenging insightful. sometimes tricky setting understanding team dynamics new teams especially people ’ know well , practice working teams always appreciated team dynamics always easy decipher understand. decisions made group ( least feeling ) without one person charge giving orders. could also part still learning try make effort person group feels opinion asked heard decisions made group decisions everyone opportunity raise differing opinions votes support. consistent way enjoy making decisions think allows everyone feel involved dedicated outcome everyone feeling others group think feel decisions made. say decision making rational emotional know accurately determine that. maybe mix , though slightly leaning towards rational .  intimidating meeting many new people , especially wildly different personalities backgrounds. sharing work space sometimes challenging everyone shares views communal space treated terms cleanliness keeping noise levels down. interacted lot people , small number individuals like background temperament. team dynamics far challenging part course. particularly struggle team-members lack self-motivation work ethic. found often instead insisting team member pulling weight would rather task myself. tend feel someone lacks motivation something unlikely perform task best abilities. also seem little confidence others completing task way would like done. generally , decision making teams easy mostly due apathy uncertainty surrounding decision behalf team members. found frustrating since often lead one-sided discussion resulted unspoken dissatisfaction resultant product. decisions also often ignored later , dont mind changing decisions believe communicated completing project still paramount. feel often good sense required complete project communicate steps team members , however rarely self-assurance properly delegate tasks hold people accountable meet deadlines complete task .  past months one challenging thrilling experience life. coming , expected engage people knew introvert , experience academy would force engaging people. something totally outside comfort zone. ready ; much prepared , ready challenged understood growth reached uncomfortable spaces. experience taught could imagined. learnt fearless embrace greatness within me. come across shy , willingness push things enabled speak room full people made realize get nervous things actual fact get nervous things capable also fail. way scared inabilities potential. understand still lot learn fully comfortable know feel still lot knowledge gained. coming bit theoretical statistical background , interesting see techniques learnt university applied real world scenarios. coming space , space full well equipped individuals , felt bit intimidated idea career going lacked confidence really good at. experience far definitely given better understanding skill set career take , importantly come learn alone. able meet lots people also similar situation , simply shown worries worth worrying things fall place. environment quite relaxed , yet taught behave professional workplace. support peers also tremendous. observing everyday events supervisors taught teamwork , people come together get things done . although sometimes remind use inside voice , feel adapting environment relatively well. going forward , expect learn develop skills .  data science academy amazing path far , one room people ambitious love learning got excited work also supervisors facilitators determined love job motivates even keeps thrilled like one day. thankful part academy , much learn everyday proven need specific field certain level education data scientist , way academy structured students fields various level education think great idea , would see someone never studied computer science statistics knowing concept better degrees , brief say need order survive academy focused know expected everyday. fortunate enough placed teams people determined never struggled get group mate deliver expected , hence said people academy focused know for. think selection team good job placing us academy. appreciate also fact much supervisors facilitators busy , always accessible help us work passion , see getting better better day become data scientist certain end year level corporate expects be. academy coaching sessions assists come across obstacles program , work related personal problems , someone talk share predicaments going , find helpful workload much might loose track things , especially something distracting going personal life , deviate attention work , think service useful personally used load shoulders shared problems someone trust judge share problems anyone. randomly placed teams believe help us come across different personalities people learn work around , although people hoped , believe prepares us fit enough deal people meet corporate world. also learn assist patient people might struggle certain concept also learn others .  attending academy first time like first year , however difference everyone age. idea meeting people really cool however issue forced work people things tend bit shaky everyone standards. make matters interesting grades directly dependent interaction team. varsity must admit best experiences working group tend find similar roles characters. however bit optimistic given diverse ages academy assumption folk work experience hopefully mature. furthermore , come realize establishing solid authentic team dynamics 's also lot stressful actually may seem without trying fake sake team trying accommodating. real issue trust decisions tends made around dominant personalities perspective may particularly best interest team whole. although always agree approach would agree fundamentally understanding skill set equally distributed also end day delivery project. overall happy rational decision making process. divide trust conquer works best teams something come learn. within teams lucky may find someone work ethic dedicated case would likely help could find someone understands work complements style. think initially everyone problem admitting n't get something deliverables due. journey emotional especially towards end tend give less less input become concerned delivery .  good day , really n't know start essay besides good day. anyway 's way lets start business. moments academy mostly up. let start day one meeting students south africa great , meeting people different personalities coming different background. people different races , culture tribes ( enjoy stuck cape town since day born. ) . getting first team , part crazy , fun team. guys n't look one another open , strength. first weeks everyone working well getting close end became frustrated team members started lose focus ( assume got tired sprint long ) . following team great came work bonding n't all. two members liked working home ( productive ) two quit time. came decision making , meter bouncing ideas around choosing one move stayed one teams. second team split two teams two teams thing 's believe new way things great. submitting individual team one learnt skill taught sprint. thanks .  good fun experience though 's often case pressure. lab allowed enough time practice get used coding , improved efficiency coding working computer. coming academy n't used idea working team , along year got better , knowing someone count , rely go need help. also got know personal level makes easy understand , strength weaknesses allows us form strong team. sometimes disagreements think right , effective solving , lucky enough co-operative team members since beginning year. though n't got know many colleagues ones got know would say pleasure meeting  academy amazing , love petry dish talent ! handful genius characters here.. backgrounds unique stories , explore data science academy best place develop data analysis skills. love time ! far , 've experienced whole lot pressure change something better. spend time studying , better feel , comfortable become closer get quality employee entrepreneur. everyday come office , feel home , learn something new.. something directly use business application. look around entire building get inspired start business 've learned here. speak among business academy helping us realize ideas clearly. without information back ideas , 's difficult starting. availability information outstanding. really appreciate nest. last two teams great work with. achieved bunch , , thanks resources available. first sprint tough though. team , realized strengths taught other. n't problem sql python used technologies before. understanding statistics applications data given difficult. worst , water crisis data n't collected well. n't worst experience data taught things career perfect. perfect skills data cleaning. realize limited number solutions data analysis. perfect outcomes data science , could breeze work future. believe hyper prepared things.. game theory data science would prove beneficial. emotionally , 'm great. academy brings best worst skills provides controlled environment mend holes analytical judgement. relief , emotionally physically becomes process bettering daily , bit bit without forcing stress .  imagine excited finally meet people like minded. maybe could rephrased meeting people like me. however , instead , meet small sample different types people different types personalities. yet find stuck world different types people different , different interest ambitions , translates make decisions every day. decisions affect , especially working teams. hand , decisions make affect team processes. according personality test , personality defined five letters entp-t , according classification debater. true debater , convinced , play along suck test discuss personality affected general experience academy. lab allowed us access well collaborate , although working effectively lab challenge , many distractions. found easier get know people type strengths , knowledge background passions general. type knowledge allows efficient knowing ask help different things. prefer asking facilitator. always done past prefer working challenge peer , along process learn would otherwise. comes working teams , always , first one speak give opinion came variety ideas , ’ extroverted side me. therefore , end leading , team. role prefer police people , therefore either allowed people lead would make sure project requirements completed end. last sprint dedicate responsibilities according people ’ backgrounds strengths interests. one person leading deliverable project end would ensure requirements submission met. employed similar strategies past , working tangibly people bring table. framed decision making teams worked in. someone shows work work have. approach made easier focus objective deliver projects. decision making rational time little emotive times , irrational judgements employed make assumption people attitudes toward project work. creates barrier people pulling weight past immediately , integrate teams later. however team dynamics forced finisher .  past 5 months explore academy challenging. make great adjustments order here. positive negative experiences. one positive experiences much learned. met lot people highly intelligent interests. worked 2 different teams. prefer work alone working team makes sense due sheer volume work. first team experience incredibly frustrating. team members highly introverted technically strong made discussions assigning tasks difficult. even tasks assigned accountability meet deadlines. prefer chip away steadily work opposed rushing close deadline finish. working second team much better. met regularly , communicated lot. tasks assigned everyone accountable. team members helpful towards .  experiences academy far life-changing say least. meeting smart , passionate , hardworking ambitious people different backgrounds happen many similar interests mine interesting journey. introvert prefers working alone pace within space , quickly find way adapt polish social skills order socialize 99 strangers going learning data science with. understood word go need socialize form meaningful relationships fellow interns could see forming strong network may helpful many years come career , long internship. academy ’ emphasis importance team work concept took warm to. teams exposing different individuals varying academic , social , work geographical background different personalities : introverts , extroverts , intuitive , artistic even really funny characters. teammates phenomenal experienced fields previous exposure to. learned lot them. equally ’ also encountered independent disagreeeable teammates required adjust levels patience , tolerance understanding may necessarily share similar beliefs me. need patient tolerant people ’ points view approaches solving completing team tasks. teams , due differing views , opinions approaches use democracy ( voting ) order reach decisions within groups. although easy one resort taking emotionally-driven decisions due one ’ firm beliefs ways things high level emotional intelligence critical favourable team dynamics. academy based cape town , city ’ visited holidays before. moving cape town challenges required solid ‘ family ’ away family fortunately found fellow interns resourceful many ways even outside ‘ labs ’ . example group share information accomodation , transport around city , social events extramural activities. chance test team-building skills virtues regular group sessions nuri believe helpful giving idea relate people less formal settings. credit due supervisors , facilitators oprerations officer ; helpful making internship experience manageable definitely one memorable years come. commitment programme makes easier people personalities settle nicely within diverse small community .  experience academy far interesting 4 months although apprehensive meeting work new people , still enjoyed work looking forward comes next. group work team dynamics particular challenge although ’ really socialised colleagues since arrived cape town , friendships formed mostly students live house me. team dynamics challenge fundamentals sprint ’ fair division responsibility well accountability team. group members would slack part hard hold anyone accountable concerned nice. decision-making group fundamentals sprint mixture rational emotional , things right things badly one wanted take responsibility , much passing buck. team also beset laziness unwillingness really get learn needed deliver project. rsa stock market sprint , group somewhat better people hard-workers divided responsibilities early based perceived strengths individual. really serious problem group communication different areas functional responsibility , clear group members prepared presentation fully understand analysis analysis team performed. decision-making group rational. working labs , nerve-wracking experience significant proportion people simply go socialise distracting us , prefer working environment absolute silence ( hence prefer working midnight ) simply think problem allow creative possible solving problem. people assist thought-processes terms thinking problem solutions irritating distraction. around many people long eventually lose focus problem focus people , would much better silence mandated enforced labs. also need individual work instead group-work. suggests ways could contribute group signs learnt lessons emerged previous group , still lot trepidation comes next. hope meet group members committed project , otherwise , everything end tears .  honesty enjoying time regret choices made. really educational love structure , environment one force work push much facilitator really helpful bounce ideas see far able explore imagination. meeting new people 's always nice stories people far go life , combine different cultural group things got learn people 's culture stories tell. work together 's really tricky fun times tricky , matter many times think always tricky talk wont enough words left. making decisions 's simple everyone agreeing goals deadline arrived people stories , 'm unable relate excuses , honestly wish everyone could adhere words , really would like decisions made like explore lot 'm working something due time constrain sometimes 's best expected enough time explore more. team members sensitive way sensitive leave part there. love academy wish time learn bit , enough resources even conduct maybe research , instance know write good machine learning algorithms n't understand work might even able improve .  time academy positive. 've unpleasant interactions person far n't believe 've cultivated many deep relationships aside perhaps one. two groups 've assigned socially pleasant. first group n't ideally organised second group better regard. n't particularly assertive presence first group unlike second positive dynamic present both. prefer make decisions group try best pull weight burden .  starting academy wonderful experience. cape town beautiful city location academy could better. meeting different characters really experience.in first weeks meeting new people daily basis sometimes intimidating. teamwork never really strongest skill always something would like improve upon , heard academy strong teamwork emphasis happy.the academy working lab strong social element , even though work learning somewhat technical. first sprint , quickly discovered diverse group was. vastly differing amounts experience fresh high-school graduate someone working decade. also differing skill sets. working groups challenging rewarding time. cape town sprint primary downfall lack planning what. resulted us almost late deadlines. communication within group could lot better. tend keep flexible approach completing tasks , flexibility ended costing establish proper deadlines .  social perspective , experiences academy refreshing impact personal development. prefer things myself. reflected interest gaming , reading fan-fiction writing. however , willing interact others necessary. enjoy paintball friends far sitting lounge rather making casual small talk. applies workplace well essentially things together. even talking , inevitably get topics related coding such. generally people ( outside academy known years ) seem identify introvert give misconception poor communication skills n't always case. many issues communicating team mates others class regarding various topics ranging work-related hobbies ( especially sport , gaming anime ) . prefer finish roleplaying video game lounge talk also like talking science nerdy things. words , need socialise , problems so. intuition observation valuable traits workplace data scientist team-based project. working teams allowed utilise both. found often analysis others n't done apply intuition. managed thing. different academics would essentially everything myself. regard , team-based nature course done well. often think good one working others stimulate us potentially become good trait previously thought n't naturally suited to. case , working teams sort force us find ways overcome weakness make strength instead. said , team dynamics sometimes involved lack coordinated planning among team members. good keep things open , applied plan rather plan. decide certain approach solving problem involves coding , dashboard , n't mean ca n't improvise. agree improvisation ultimate way things. make plan rely open-minded thinking plan n't appear working , improvise. generally experience academy , emotion seems rule roost logic seen potential barrier team work cooperation. mean students end becoming dependent affect teamwork negatively long term. overall course excelled bringing personality traits relevance workplace , providing opportunities overcome weaknesses utilise strengths instead .  academy challenging experience. expected prior coding knowledge help minimal help , greatest challenge statistics. slowing getting point n't actually try understand concept thoroughly rather get basic theory focus application. overall , insightful journey , learning interact different personalities group dynamics. really excited leaning nlp regression incorporate app development monetize it. genuinely regret applying academy although days doubt hurdles remembering greatness opportunity always great motivator .  four months since decided embark journey. packed bag , moved new city took first step living best moments life. love meeting new souls. always fascinating finding someone else ’ journey began , similar different footsteps left are. amongst 99 brilliant minds incredible , getting know refreshing. people stand , everyone unique. always good working , taking reins. ability work effectively team acquired skill , understand different personalities , people deal problems communicate. delegating seemed help terms getting work done teams , person taking piece , sure finish elephants. decision-making process structured , benefited whole team. guidelines rubric supervisors helped keep us right track. decision-making mostly based impulse , following process ideal getting results. great team members pay attention detail since mostly focus bigger picture. made sure work appealing. quite experience , constant learning. growing skill set , including soft skills. beginning , lot learn .  ‘ greatest opportunity part 100 chosen individuals wonderful programme. explore data science academy ( edsa ) environment makes feel welcome perfect terms collaboration networking different individuals interesting views personalities. ’ hard distinguish edsa work environment. type environment prepare real job. edsa provides one opportunity learn work diverse group people terms career background , culture religion. working groups different individuals project exciting. throughout time , ’ learned lot working teams. first ’ learned team work achieve terms results learning new things. secondly , come decision making , everyone opinion coming final decision another thing. learned ’ voicing opinion , convincing fellow team members indeed opinion fruitful. comes team strategies embark particular project , ’ one size fits situations. team come strategy going solve assigned problem. difficult , especially serve team coordinator serve , time complete different team whole set different personalities characters. instance learned strategy based skills , capabilities mostly team ’ attitude towards project hand. knowing strengths weaknesses team members good start. far learned lot. ’ still intrigued eager learn lot more. feel like becoming real data scientist day passes thanks edsa giving wonderful opportunity .  explore data science academy 19 weeks ( counted ) ; let ’ simplify 3 months shall ? exciting working group challenging problems , feel like world possibilities opened us. exciting overwhelming met many new people. right away seem made name myself. proud , even excited people feel someone come help - practical emotional. helping others role comfortable since typically excelled academic environment , ’ admit disappointment people ’ ask help more. like thought expert ( even like feel like expert , find happens less grow older ( experienced ? wiser ? one old greeks said something wisdom knowing ’ know ) . aspect larger group find overwhelming trying remember everyone , especially since many people seem know am. enjoy learning people ’ backgrounds amazed great selection people attending college. experience working intensely type team entirely new. worked teams school , university work , never amount diversity team. never sitting next team members everyday ; majority one ’ efforts focussed team work. immediate challenge first team working people skills ways operating different own. realised career worked people , background training. made less adaptable less tolerant different approaches problem. trying learn relax give people ’ method approaching problems chance. felt comfortable group leader first sprint , uncomfortable giving reigns second sprint onwards. second project usurped actual leader number occasions seldom pulled back. formed real bond first team , despite ( ) struggles. second team relationship seems formal , although things went much smoothly team arguably effective. found frustrating deal reserved people. hard read difficult reserve judgement ’ produce tangible outputs. still trying figure best deal people , draw shell patience hear trying say. also need consciously tell offer may valuable ( remain unconvinced evidence right table ) . experience also underlined much rely external encouragement praise , played interactions students staff. enjoy teaching students , specifically one one small groups , alot immediate feedback. first weeks felt really supported facilitators , later weeks felt like abandoned newer , exciting projects. similarly felt much secure first mentor ’ hands-on , available , approach second facilitator ’ reserved , hands-off approach .  young age always sheltered , reserved terrified getting comfort zone. knew year would challenge ’ challenged break shell. first day got real knew one person hundred. push open interact crowd ( struggle considering ’ shy standing front people makes extremely nervous ) . lucky people academy friendly understanding. working teams never problem , issue able fully center attention speak thoughts. taught importance communicate ideas without shy. one biggest challenge working team dealing teammates ’ fully committed tasks assigned them. personally prefer get things done certain way bit challenge team timelines ’ met team members ’ putting much effort everyone else. lab exposes us different personalities one needs learn work together effectively many personalities many conflicting personalities could recipe disaster. could say lucky teams assigned ’ difficult. understood importance contribution communication. misunderstanding ’ address becomes conflict , way able prevent it. teams decision-making process pretty much same. generally , challenge put team , communicate strengths weaknesses , able distribute tasks together based strengths weaknesses. decisions need made , every member given opportunity give opinion decide together team taking consideration every member 's contribution. overall , experience far great. learning new things everyday. workload get challenging time time managed. slowly coming shell , talking , interacting opening .  experience academy really mixed bag far. initially `` '' whatever decision making process needed order get ahead , new group , 've participate others made feel `` follower '' leader times. 's really nice working people myriad personalities. n't really think 've come across situation tempered emotionally although think quite emotional person. think far decision making way prefer ; logical straightforward also room change improve personal knowledge  find experiences academy interesting fulfilling. really like working smart people explore gives abundance. personality clashes character development hurdles point need growth adjustment. one real challenges find balance zoning focus work teaming discuss ideas others help others. choose start would rather acted like focused introvert would give time learn without harming learning curve. honestly contributed negatively group members know , well , deliver more. working together lab thing made real progress growth us individuals. crucial team decisions easier make. working lab also downsides feel still take full advantage privilege. missed couple key submissions already , messed two tests failed badly. made aware team dynamics past negatively affected catch-up work cost team process. honestly think hard also says high expectations myself. decision-making process usually one based voting. people seen getting things knowledgeable automatically obtain voting power. past experience work usually drive person chooses group role willing play. future , would prefer , rules engagement given thorough attention prior commencement group work. way , best obtained everyone. personally believe need place comfortable enough work done tools available. accomplished. achieve contribute .  experience academy thus far potent insightful year , changed life become person really enjoys importance cooperating others. brought together amazing minds space really helped acquire soft hard skills short space time , indeed helped elevate way learning new technical subjects efficiently effectively. also , management team large especially supervisors instrumental always improvising make sure led way help us walk away well informed junior data scientists end year , hence work tirelessly making sure break obstacle attractive easily comprehended possible us order put hard work. addition , nuri formidable way helping us cope psychological illnesses caused intensity program , always go extra mile making sure one feels solitary. supporting structure throughout program importance , help us understand essence behind excruciating vulnerability management enable see work not. last words , want close saying means life , 'm grateful opportunity , best thing ever happened life besides varsity , 'm pretty sure set life greater things achievements .  time academy far challenging , yet extremely rewarding self-fulfilling - especially regards working people different backgrounds domains expertise. although meeting different people interesting , concerned bring different people together towards achieving common goal. two teams worked , often one take lead , drawing ambition past experience relation data science related fields. challenging , since used - comfortable - working alone. hold personal contributions team-based projects highest standards , therefore difficult team members show dedication team-based projects do. working teams challenged personality positive negative ways. like control , therefore difficult allow people take lead different aspects projects , especially objectively feel experience completing particular objective. however , learned impossible control aspects project therefore welcomed aided contributions team members shown willingness aptitude contribute. contrast , difficult form motivate team members show apathy towards achieving common goal. although somewhat frustrating , ensured never derailed team following initial plan objectives. felt comfortable role coordinator , relished opportunity deliver high-quality deliverables high pressure situations. drive work ethic good , always productive worker. over-analysed spent excessive amounts time subjects interesting challenging personal level , instead focusing time pressing matters. generally tend spend time challenging interesting concepts , tend postpone often neglect routine administrative tasks. creates frantic dash line time runs , creates unnecessary anxiety. conscious shortcoming , actively try remind prioritise complete pressing matters regardless personal interest. know often regarded serious fellow team mates , since always focused driven comes task hand. little time chit-chat tend engage conversations related intriguing philosophical subjects. feel comfortable responsive. however , learned casual conversation regarding sports hobbies helps learn team members , forges professional , understanding relationships team members. overall , work experience academy enlightening one , far perfect. lots learn , eager improve aspect career benefit team mates .  positive experience academy far. enjoyed meeting people interest data science also made unusual choice learning institution. fascinating getting know people class. people incredibly wide range personalities backgrounds. generally enjoyed working lab though get rowdy lunch people 's work motivation decreases motivation socialise increases. found first team members rather like-able , felt team dynamic failed dismally properly tackling project. success beginning ultimately disappointed team ’ performance. lacked cohesiveness n't project manage effectively. people focused exclusively individual learning rather group work. second group considerably better. clear strategy allocated tasks effectively. regular meetings , took minutes , wrote tasks sub deadlines. member motivated pulled weight. also communicated frequently balanced individual group tasks much better previous group. decision making initial group rational however rarely executed decisions intended. lot procrastination members arrived late class left early. one group member spent time away group chatting friends. claimed working group tasks , ended contributing absolutely nothing group tasks. contrast , individual second group went beyond call duty. regular meetings ensured tasks divided properly. progress closely monitored , problem occurred , rallied together solve adapt strategy. one member group already quite experienced particular topic project tremendous advantage us. thought carefully team workflow set appropriate software working folders. analysed project marking rubric created gantt chart schedule tasks , understand dependencies. also manage group member availability member group required periods absence project. also scheduled tasks around class tests aware time allocated individual efforts. initial group unhealthy diffusion responsibility member assumed someone else would pick slack. lead group members becoming despondent refusing carry group members. also faced issue one group member ’ feel sufficient programming skills make decent contribution group effort. despite shortcomings group got reasonable marks. credit go two five group members. sure second group get good marks. credit also given equally group members one member particularly strong. however think efforts equal previous group everyone felt satisfied group effort end .  bit loner times id say experience really taught engage people talk especially student kind goal. based academic level think got experience programming fact familiar sql python shows grown academic level. knowing wake everyday see new faces lab thought able manage knowing loner told working teams thought end much talker people thought going express opinions , participate happened best experience even started releasing need talk order opinion heard. last team really understood much teams challenges valued opinions decision making part even pimplier , first team agreed everybody gon na take part everything look strength weaknesses. assign one another activities concerning project still forgetting help strengthen academy weaknesses basically created fridays helping other. comes decision coming acadamy one hard decisions take cape town far home choose completing honors degree academy. ended choosing academy honors degree felt like needed experience data science marketing world academy would best place start fact ought learn everything beginning great computing skills wanted stipend also contributed decision coming here. would conclude regret coming instead happy here.its 5 months already feel like year going bit faster want even programming skills think would got much experience programming languages skills honors.the experience got far best. helped growing academically also helped gain confidence individual .  like meeting new people similar interests , many new people , connecting people time goes assigned new group. n't spoken students merely strong introverts , recent group nominated group coordinator take lead every meeting , every interaction acknowledge personal level group members feeling day , something never really thought experienced , even grew every mondays share weekend , really appreciated put position challenge grow personal academically. like make decisions based merits decision efficient come attention efficient time take account people 's feelings , made decisions past group people feel included left , worth noting individuals contribute project way , barely showed academy , never written one single line code left member take care making dashboard , presentation ultimately present , two us took uncomfortable route diplomacy , pleaded every morning attend briefings supervisor us , went group training sessions made impact , next group , understood academy meant worked together well. stallions , magnificent , sudden paradigm changed , enjoyed one idea best things , originally typed minutes meetings another member suggested trello effective , learnt something new , excited , day , well every day went bed knowing something ’ know woke ’ data scientist ask for. one important lesson got working two different groups leave time guide team work flow everyone miserable end project , however everything planned trello ( ’ trello ) ‘ tees ’ crossed ‘ eyes ’ dotted , pleasant experience bring best everyone involved , team creativity grow levels one planned beginning. , irrespective put group work , sit maybe second session “ getting know ” map strategy roles assigned , significant insignificant .  edsa reflections 24 november 2017 , filled joy , excitement anxious time opened email edsa titled “ congratulations ! ’ accepted explore data science academy ! ” fast forward 22 january 2018 , ready meet new people ( possibly form new friendships ) , meet edsa team learn lies ahead 2018 , overwelming day was. today 6 may 2018 , lot happened past 3 months , ’ learned much ! sql , stats , python , power bi , presentation skills… [ still learning , never stops : ) ] . grateful selected one top 100 , ’ appreciative opportunity , thank edsa making possible me. lab data scientists making , greatly enjoy lab , environment condusive learning , group data science enthusiasts one room feel receipe success. training limited online courses edsa staff students learn lot , different backrounds , skills experience making lab interesting fun place grow academically socially. teamwork water crisis project : team liquidata must say initially concerned amount individual work deliver team well. realized importance effective team , working well team reduced work load stress also helpful individual tasks. roses regards team dynamics , working different individuals perceptions opinions challenging first , frustrating decide certain matters , helped lot grounded problem statement kept us focused needed done. worked well ’ result driven given task , completing task successfully required relevant info important , refering back problem statement got stuck great tool us. emotional moments team due personality clashes end worked well contributed success team , awesome team ! rsa stock market project : wolves woodstock accepted ’ work new people , really problem , hope wish strong programmer team , wish realised statistician programmer team , lovely ! great team , felt like outspoken person ( ) bit much , still worked well though delivered. prefer working team different ideas , creativity different skills ’ huge advantage eveyone team result driven , absolutelty appreciate that. ’ good experience teams far .  first day first looked familiar faces. started looking people would like associate self with. meeting new people fine long n't expect keep conversation going. found nice , guys hang with. love working together labs , could share idea 's , findings ways certain things. enjoy working teams. believe 2 minds better one. also another line validity checking. one also learns lot new things working groups. team dynamics like , everyone knows need achieve bigger picture. long term though ones ideas change clearer picture , completed task. ( problems change unforeseen problems come up. ) decisions made done still needs happen. times felt team members know exactly needed next ( floated around ) . decision making based possible without overworking everyone .  4 months edsa filled new experiences ; challenges , joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates edsa , provided insight gained experience tools like python & sql. cherish able build network data scientists tomorrow established industry professionals.. -- -working lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious noise distraction - say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes responsibility ) members group noobs sql & python , gather analyse phases really exposed technical deficiencies resulted lot frustration came time delivering tasks. experience took comfort zone important lesson patience all. second group high level experience , strong team co-ordinator team members took responsibility tasks. work complex comfortable team dynamic. -- -how decisions teams consistent make decisions -- - team decisions groups deliberation. decisions easier make person presented clear points opinion was. group dynamic strong decision making felt natural , similiar make decision own. -- -how rational emotional decision making process -- - somebody technical knowledge provide interpretation best solution. problem required research place rely rational logical thinking arrive solution. regarding emotions , used open communication channels facilitators supervisors discuss emotions opposed letting emotions making project decisions. -- -overall -- - previous role electrical designer smitten data analytics electricity metering sector. edsa provided platform pursue passion broader scope data using market leading tools importantly upgrade skill-set opening new opportunities workplace .  looking got academy right , confidently say grown intrapersonally interpersonally optimistic perception working gradually shifted realistic end spectrum. came positive outlook journey gotten opportunity embark , , learned couple valuable lessons - stand value understanding teammates expecting understand you. far cap assertive nature gotten lot f pressure given opportunity sharpen softer skills tends o help quite lot outside academy good vibe generator - deflects negative energy positive energies spheres happen in. getting know colleagues exciting initially thought best know knowing underlying power networking tend ripe rewards expect it. , various factors including time constraints , n't gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed - 've connects colleagues duties better understanding really drives bed morning tends relative ease synergies working people group. foundation does good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities n't easiest tasks , things far role duty allocations bit messy groups. mind team makes decisions usually voting process duties tend easily misallocated does accord effectiveness leverage ( exploiting team member unique abilities motivate play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've decision-making techniques bit irrational central tend especially group nice people express feel doing so. learned focus making sure ’ s expected delivered timely miss beauty process focussing relatively small hurdles ’ s vast captivating material engage in. getting know colleagues exciting initially thought best know knowing underlying power networking tend ripe rewards expect it. , various factors including time constraints , n't gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed - 've connects colleagues duties better understanding really drives bed morning tends relative ease synergies working people group. foundation does good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities n't easiest tasks , things far role duty allocations bit messy groups. mind team makes decisions usually voting process duties tend easily misallocated does accord effectiveness leverage ( exploiting team member unique abilities motivate play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've decision-making techniques bit irrational central tend especially group nice people express feel doing so. 've learned bring emotions workplace people tend step sensitive things noticing- example having staring shoulder commenting work 're doing quite unpleasant kills moral detaching focusing 's required best coping mechanisms .  attending explore data science academy proven , continues amazing experience. ’ s quite astonishing realize ’ ve learned mastered past couple months , chief ( subject essay ) , teamwork. meeting new people daunting task ( including ) . academy different start. sheer excitement students especially explore team ( supervisors facilitators ) acted social catalyst , smoothing social awkwardness , knew getting like house fire. working teams people completely different backgrounds personalities challenging project , said , interesting individual adapts accommodate team help build team dynamic. starting cape town water crisis project honestly idea expect , certainly didn ’ t expect team dynamic play huge role success deliverables. team overcome different levels knowledge , skills , self motivation. couple late nights flared tempers managed work , realised overlooked crucial teamwork. does need self motivation , create environment team motivate comfortable ask help ’ feeling overwhelmed struggling grips difficult concept. team definitely lacking aspect , chin regards deliverables , walked away learning priceless lesson moving sprint. assigned new teams rsa stock market sprint , refreshing hear weren ’ t team struggled certain aspects team dynamics. approached structured interactive way doing things. decided stand meetings morning report previous day ’ s goals setting new goals day ahead. huge difference motivation team. success dividing work didn ’ t overwhelmed individuals sharing workload having one-on-ones relevant team members gain deeper understanding section work. approach advantages thought. having structure communication team members lead lower stress levels ( present ) , resulting cooler tempers clear headed decisions emotionally-driven project. just goes “ soft ” skills just important technical skills working teams. ’ t wait learn brilliant diverse people academy. '' individual commitment group effort -- makes team work , company work , society work , civilization work. '' -- vince lombardi  wonderful place , place waking morning easy task great desire learning new everyday. meeting new people great thing learn work team socialize person crowd. actually time working lab like reason experience labs quite instrumentals labs testing experiments computers , did n't computer labs time coding lab reason coming edsa choice . skills data scientist drove apply wan na learn coding. learned lot lab working team. main problem faced working team group , groups perfect wish change disaster.sometimes set time slots group meet discuss progress deliverable surprised member pitch having thought lot work , disappointing prefer working future. working team emotional draining decision divide conquer work well. hoping great team smart people need told sprint `` lists '' .  essay discussing personal experience regards academy like meeting new people especially introverted person am. speak different teams , different different felt , learned different characters. comparing decision making teams versus personal decision making. getting plane cape town place stay friends city big risk , certain plan academy kept praying want scam plot specific building kidnapped. fairly surprised meet people walks life , kind questioning legit , mean hard question , sounded way good true good things learned don ’ t happen people like , guess time shine receive blessings. ’ s think academy , blessing. said , experience academy ups downs like ends up.i learned lot , fallen love data science , doing things small mindedness used consider geek ’ s. learned push , try learn , practice computers evolving. met people different personalities academy , personality clashes occurred 1 biggest lessons , able work different people different personalities. working teams wasn ’ t easy thought , idea assumed supposed different opinions dismissed 's opinion valuable working team , opinions clash. working teams taught patient , quality did possess whatsoever. second time changed teams gotten idea working teams shouldn ’ t like , basically just dipping toes working teams thing sprint , second sprint bit confident meant worked better second teams , improving go. teams , split work equally , making sure gets chance tackle important lessons like coding dashboard. group coordinator state views , he/she speak supervisor decision way , supervisor guide right direction hearing thought right way.its concept use guess making decisions , weigh options , best worst outcome obviously best outcome. conclusion working teams skill gaining developing everyday , haven ’ t mastered , heck difficult master personalities , getting .  experience academy far stressful fun. 'm enjoys working groups , 'd working times. starting sprint reminded prefer working alone. working group certain people did carry weight , frustrating. , blame did speak let concerns heard. end saw way tackle task hand work finishing assignments given. group complete opposite. weeks complete sprint knew work collective. tasks split accountable project. ended working got outside work well. did work previous group ? know , look forward working different people different dynamics .  experience academy enjoyable far. learning lot interesting valuable skills meeting intelligent interesting people. meeting working new people lot fun rewarding experience. working lab challenging. open plan allows diverse unusual conversions easily started joined prevent work people interrupt prevent entering staying state flow necessary learning. especially destructive slower learners like myself. working teams challenging insightful. tricky setting understanding team dynamics new teams especially people don ’ t know , practice working teams appreciated team dynamics easy decipher understand. decisions group ( feeling ) person charge giving orders. learning try make effort person group feels opinion asked heard decisions group decisions opportunity raise differing opinions votes support. consistent way enjoy making decisions think allows feel involved dedicated outcome feeling group think feel decisions made. say decision making rational emotional know accurately determine that. maybe mix , slightly leaning rational .  intimidating meeting new people , especially wildly different personalities backgrounds. sharing work space challenging shares views communal space treated terms cleanliness keeping noise levels down. interacted lot people , small number individuals like background temperament. team dynamics far challenging course. particularly struggle team-members lack self-motivation work ethic. instead insisting team member pulling weight just task myself. tend feel lacks motivation unlikely perform task best abilities. little confidence completing task way like done. generally , decision making teams easy apathy uncertainty surrounding decision behalf team members. frustrating lead one-sided discussion resulted unspoken dissatisfaction resultant product. decisions ignored later , dont mind changing decisions believe communicated completing project paramount. feel good sense required complete project communicate steps team members , rarely self-assurance properly delegate tasks hold people accountable meet deadlines complete task .  past months challenging thrilling experience life. coming , expected engage people knew introvert , experience academy force engaging people. totally outside comfort zone. ready ; prepared , ready challenged understood growth reached uncomfortable spaces. experience taught imagined. learnt fearless embrace greatness me. come shy , willingness push things enabled speak room people realize nervous things actual fact nervous things capable fail. way scared inabilities potential. understand lot learn fully comfortable know feel lot knowledge gained. coming bit theoretical statistical background , interesting techniques learnt university applied real world scenarios. coming space , space equipped individuals , felt bit intimidated idea career going lacked confidence really good at. experience far definitely given better understanding skill set career , importantly come learn alone. able meet lots people similar situation , simply shown worries worth worrying things fall place. environment quite relaxed , taught behave professional workplace. support peers tremendous. just observing everyday events supervisors taught teamwork , people come things . remind use inside voice , feel adapting environment relatively well. going forward , expect learn develop skills .  data science academy amazing path far , room people ambitious love learning got excited work doing having supervisors facilitators determined love job motivates keeps thrilled like day. thankful academy , learn everyday proven need specific field certain level education data scientist , way academy structured students fields various level education think great idea , studied computer science statistics knowing concept better degrees , brief say need order survive academy focused know expected everyday. fortunate placed teams people determined struggled group mate deliver expected , said people academy focused know for. think selection team did good job placing academy. appreciate fact supervisors facilitators busy , accessible help work passion , getting better better day data scientist certain end year level corporate expects be. academy coaching sessions assists come obstacles program , work related personal problems , talk share predicaments going , helpful workload loose track things , especially distracting going personal life , deviate attention work , think service useful personally used load shoulders shared problems trust judge share problems anyone. randomly placed teams believe help come different personalities people learn work , people hoped , believe prepares fit deal people meet corporate world. learn assist patient people struggle certain concept learn .  attending academy time like year , difference age. idea meeting people really cool issue forced work people things tend bit shaky standards. make matters interesting grades directly dependent interaction team. varsity admit best experiences working group tend similar roles characters. bit optimistic given diverse ages academy assumption folk work experience hopefully mature. furthermore , come realize establishing solid authentic team dynamics 's lot stressful actually trying fake sake team trying accommodating. real issue trust decisions tends dominant personalities perspective particularly best team whole. agree approach agree fundamentally understanding skill set equally distributed end day delivery project. overall happy rational decision making process. divide trust conquer works best teams come learn. teams lucky work ethic dedicated case likely help understands work complements style. think initially problem admitting n't deliverables due. journey emotional especially end tend input concerned delivery .  good day , really n't know start essay good day. 's way lets start business. moments academy up. let start day meeting students south africa great , meeting people different personalities coming different background. people different races , culture tribes ( enjoy stuck cape town day born. ) . getting team , crazy , fun team. guys did n't look open , strength. weeks working getting close end frustrated team members started lose focus ( assume got tired sprint long ) . following team great came work bonding n't all. members liked working home ( productive ) quit time. came decision making , meter bouncing ideas choosing stayed teams. second team split teams teams did thing 's believe new way doing things great. submitting individual team learnt skill taught sprint. thanks .  good fun experience 's case pressure. lab allowed time practice used coding , improved efficiency coding working computer. coming academy n't used idea working team , year got better , knowing count , rely need help. got know personal level makes easy understand , strength weaknesses allows form strong team. disagreements think right , effective solving , lucky co-operative team members beginning year. n't got know colleagues ones got know say pleasure meeting  academy amazing , love petry dish talent ! handful genius characters here.. backgrounds unique stories , explore data science academy best place develop data analysis skills. love time ! far , 've experienced lot pressure change better. spend time studying , better feel , comfortable closer quality employee entrepreneur. everyday come office , feel home , learn new.. directly use business application. look entire building inspired start business 've learned here. speak business academy helping realize ideas clearly. information ideas , 's difficult starting. availability information outstanding. really appreciate nest. teams great work with. achieved bunch , , thanks resources available. sprint tough though. team , realized strengths taught other. did n't problem sql python used technologies before. understanding statistics applications data given difficult. worst , water crisis data n't collected well. n't worst experience data taught things career perfect. perfect skills data cleaning. realize limited number solutions data analysis. perfect outcomes data science , breeze work future. believe hyper prepared things.. game theory data science prove beneficial. emotionally , 'm great. academy brings best worst skills provides controlled environment mend holes analytical judgement. relief , emotionally physically process bettering daily , bit bit forcing stress .  imagine excited finally meet people like minded. maybe rephrased meeting people like me. , instead , meet small sample different types people different types personalities. stuck world different types people different , different ambitions , translates make decisions day. decisions affect , especially working teams. hand , decisions make affect team processes. according personality test , personality defined letters entp-t , according classification debater. true debater , convinced , play suck test discuss personality affected general experience academy. lab allowed access collaborate , working effectively lab challenge , distractions. easier know people type strengths , knowledge background passions general. type knowledge allows efficient knowing ask help different things. prefer doing asking facilitator. past prefer working challenge peer , process learn otherwise. comes working teams , , speak opinion came variety ideas , ’ s extroverted me. , end leading , team. role prefer police people , allowed people lead make sure project requirements completed end. sprint dedicate responsibilities according people ’ s backgrounds strengths interests. person leading deliverable project end ensure requirements submission met. employed similar strategies past , working tangibly people bring table. framed decision making teams worked in. shows work work have. approach easier focus objective deliver projects. decision making rational time little emotive times , irrational judgements employed make assumption people attitudes project work. creates barrier people pulling weight past immediately , integrate teams later. team dynamics forced finisher .  past 5 months explore academy challenging. make great adjustments order here. positive negative experiences. positive experiences learned. met lot people highly intelligent interests. worked 2 different teams. prefer work working team makes sense sheer volume work. team experience incredibly frustrating. team members highly introverted technically strong discussions assigning tasks difficult. tasks assigned accountability meet deadlines. prefer chip away steadily work opposed rushing close deadline finish. working second team better. met regularly , communicated lot. tasks assigned accountable. team members helpful .  experiences academy far life-changing say least. meeting smart , passionate , hardworking ambitious people different backgrounds happen similar interests interesting journey. introvert prefers working pace space , quickly way adapt polish social skills order socialize 99 strangers going learning data science with. understood word need socialize form meaningful relationships fellow interns forming strong network helpful years come career , long internship. academy ’ s emphasis importance team work concept took warm to. teams exposing different individuals varying academic , social , work geographical background different personalities : introverts , extroverts , intuitive , artistic really funny characters. teammates phenomenal experienced fields previous exposure to. learned lot them. equally ’ ve encountered independent disagreeeable teammates required adjust levels patience , tolerance understanding necessarily share similar beliefs me. need patient tolerant people ’ s points view approaches solving completing team tasks. teams , differing views , opinions approaches use democracy ( voting ) order reach decisions groups. easy resort taking emotionally-driven decisions ’ s firm beliefs ways doing things high level emotional intelligence critical favourable team dynamics. academy based cape town , city ’ ve visited holidays before. moving cape town challenges required solid ‘ family ’ away family fortunately fellow interns resourceful ways outside ‘ labs ’ . example group share information accomodation , transport city , social events extramural activities. chance test team-building skills virtues regular group sessions nuri believe helpful giving idea relate people formal settings. credit supervisors , facilitators oprerations officer ; helpful making internship experience manageable definitely memorable years come. commitment programme makes easier people personalities settle nicely diverse small community .  experience academy far interesting 4 months apprehensive meeting having work new people , enjoyed work looking forward comes next. group work team dynamics particular challenge haven ’ t really socialised colleagues arrived cape town , friendships formed students live house me. team dynamics challenge fundamentals sprint didn ’ t fair division responsibility accountability team. group members slack hard hold accountable concerned nice. decision-making group fundamentals sprint mixture rational emotional , did things right things badly wanted responsibility , passing buck. team beset laziness unwillingness really learn needed deliver project. rsa stock market sprint , group somewhat better people hard-workers divided responsibilities early based perceived strengths individual. really problem group communication different areas functional responsibility , clear group members prepared presentation did fully understand analysis analysis team performed. decision-making group rational. working labs , nerve-wracking experience significant proportion people simply socialise distracting , prefer working environment absolute silence ( prefer working midnight ) simply think problem allow creative possible solving problem. people assist thought-processes terms thinking problem solutions irritating distraction. people long eventually lose focus problem focus people , better silence mandated enforced labs. need individual work instead group-work. suggests ways contribute group signs learnt lessons emerged previous group , lot trepidation comes next. hope meet group members committed project , , end tears .  honesty enjoying time regret choices made. really educational love structure , environment force work just push facilitator really helpful bounce ideas far able explore imagination. meeting new people 's nice stories people far life , combine different cultural group things got learn people 's culture stories tell. work 's really tricky fun times tricky , does matter times think tricky talk wont words left. making decisions 's simple agreeing goals deadline arrived people stories , 'm unable relate excuses , honestly wish adhere words , really like decisions like explore lot 'm working time constrain 's just best expected time explore more. team members sensitive way sensitive leave there. love academy wish time learn bit , resources conduct maybe research , instance know write good machine learning algorithms n't understand work able improve .  time academy positive. 've unpleasant interactions person far n't believe 've cultivated deep relationships aside one. groups 've assigned socially pleasant. group n't ideally organised second group better regard. n't particularly assertive presence group unlike second positive dynamic present both. prefer make decisions group try best pull weight burden .  starting academy wonderful experience. cape town beautiful city location academy better. meeting different characters really experience.in weeks meeting new people daily basis intimidating. teamwork really strongest skill like improve , heard academy strong teamwork emphasis happy.the academy working lab strong social element , work learning somewhat technical. sprint , quickly discovered diverse group was. vastly differing amounts experience fresh high-school graduate working decade. differing skill sets. working groups challenging rewarding time. cape town sprint primary downfall lack planning does what. resulted late deadlines. communication group lot better. tend flexible approach completing tasks , flexibility ended costing did establish proper deadlines .  social perspective , experiences academy refreshing impact personal development. prefer things myself. reflected gaming , reading fan-fiction writing. , willing interact necessary. enjoy paintball friends far sitting lounge making casual small talk. applies workplace essentially doing things together. talking , inevitably topics related coding such. generally people ( outside academy known years ) identify introvert misconception poor communication skills n't case. issues communicating team mates class regarding various topics ranging work-related hobbies ( especially sport , gaming anime ) . prefer finish roleplaying video game lounge talk like talking science nerdy things. words , need socialise , problems doing so. intuition observation valuable traits workplace data scientist team-based project. working teams allowed utilise both. analysis n't apply intuition. managed thing. different academics essentially myself. regard , team-based nature course well. think good working stimulate potentially good trait previously thought n't naturally suited to. case , working teams sort force ways overcome weakness make strength instead. said , team dynamics involved lack coordinated planning team members. good things open , applied plan plan. decide certain approach solving problem involves coding , dashboard , does n't mean ca n't improvise. agree improvisation ultimate way things. make plan rely open-minded thinking plan does n't appear working , improvise. generally experience academy , emotion rule roost logic seen potential barrier team work cooperation. mean students end dependent affect teamwork negatively long term. overall course excelled bringing personality traits relevance workplace , providing opportunities overcome weaknesses utilise strengths instead .  academy challenging experience. expected prior coding knowledge help minimal help , greatest challenge statistics. slowing getting point n't actually try understand concept thoroughly basic theory focus application. overall , insightful journey , learning interact different personalities group dynamics. really excited leaning nlp regression incorporate app development monetize it. genuinely regret applying academy days doubt hurdles remembering greatness opportunity great motivator .  months decided embark journey. packed bag , moved new city took step living best moments life. love meeting new souls. fascinating finding ’ s journey began , similar different footsteps left are. 99 brilliant minds incredible , getting know refreshing. people stand , unique. good working , taking reins. ability work effectively team acquired skill , having understand different personalities , people deal problems communicate. delegating help terms getting work teams , person taking piece , sure finish elephants. decision-making process structured , benefited team. guidelines rubric supervisors helped right track. decision-making based impulse , following process ideal getting results. great having team members pay attention focus bigger picture. sure work appealing. quite experience , constant learning. growing skill set , including soft skills. just beginning , lot learn .  ‘ s greatest opportunity 100 chosen individuals wonderful programme. explore data science academy ( edsa ) environment makes feel welcome perfect terms collaboration networking different individuals interesting views personalities. ’ s hard distinguish edsa work environment. type environment prepare real job. edsa provides opportunity learn work diverse group people terms career background , culture religion. working groups different individuals project exciting. time , ’ ve learned lot working teams. ’ ve learned team work achieve terms results learning new things. secondly , come decision making , opinion coming final decision thing. learned ’ s voicing opinion , convincing fellow team members opinion fruitful. comes team strategies embark particular project , ’ s size fits situations. team come strategy going solve assigned problem. difficult , especially serve team coordinator serve , time complete different team set different personalities characters. instance learned strategy based skills , capabilities team ’ s attitude project hand. knowing strengths weaknesses team members good start. far learned lot. ’ m intrigued eager learn lot more. feel like real data scientist day passes thanks edsa giving wonderful opportunity .  explore data science academy 19 weeks ( counted ) ; let ’ s simplify 3 months shall ? exciting working group challenging problems , feel like world possibilities opened us. exciting overwhelming met new people. right away myself. proud , excited people feel come help - practical emotional. helping role comfortable typically excelled academic environment , ’ ll admit disappointment people don ’ t ask help more. like thought expert ( like feel like expert , happens grow older ( experienced ? wiser ? old greeks said wisdom knowing don ’ t know ) . aspect larger group overwhelming trying remember , especially people know am. enjoy learning people ’ s backgrounds amazed great selection people attending college. experience working intensely type team entirely new. worked teams school , university work , diversity team. sitting team members everyday ; majority ’ s efforts focussed team work. immediate challenge team working people skills ways operating different own. realised career worked people , background training. adaptable tolerant different approaches problem. trying learn relax people ’ s method approaching problems chance. felt comfortable group leader sprint , uncomfortable giving reigns second sprint onwards. second project usurped actual leader number occasions seldom pulled back. formed real bond team , despite ( ) struggles. second team relationship formal , things went smoothly team arguably effective. frustrating deal reserved people. hard read difficult reserve judgement don ’ t produce tangible outputs. trying figure best deal people , draw shell patience hear trying say. need consciously tell offer valuable ( remain unconvinced evidence right table ) . experience underlined rely external encouragement praise , played interactions students staff. enjoy teaching students , specifically small groups , alot immediate feedback. weeks felt really supported facilitators , later weeks felt like abandoned newer , exciting projects. similarly felt secure mentor ’ s hands-on , available , approach second facilitator ’ s reserved , hands-off approach .  young age sheltered , reserved terrified getting comfort zone. knew year challenge ’ d challenged break shell. day got real knew person hundred. push open interact crowd ( struggle considering ’ m shy standing people makes extremely nervous ) . lucky people academy friendly understanding. working teams problem , issue able fully center attention speak thoughts. taught importance communicate ideas shy. biggest challenge working team dealing teammates aren ’ t fully committed tasks assigned them. personally prefer things certain way bit challenge team timelines weren ’ t met team members weren ’ t putting effort else. lab exposes different personalities needs learn work effectively personalities conflicting personalities recipe disaster. say lucky teams assigned weren ’ t difficult. understood importance contribution communication. misunderstanding ’ d address conflict , way able prevent it. teams decision-making process pretty same. generally , challenge team , communicate strengths weaknesses , able distribute tasks based strengths weaknesses. decisions need , member given opportunity opinion decide team taking consideration member 's contribution. overall , experience far great. learning new things everyday. workload does challenging time time managed. slowly coming shell , talking , interacting opening .  experience academy really mixed bag far. initially `` '' decision making process needed order ahead , new group , 've participate feel `` follower '' leader times. 's really nice working people myriad personalities. n't really think 've come situation tempered emotionally think quite emotional person. think far decision making just way prefer ; logical straightforward room change improve personal knowledge  experiences academy interesting fulfilling. really like working smart people explore gives abundance. personality clashes character development hurdles point need growth adjustment. real challenges balance zoning focus work teaming discuss ideas help others. choose start acted like focused introvert time learn harming learning curve. honestly contributed negatively group members know , , deliver more. working lab thing real progress growth individuals. crucial team decisions easier make. working lab downsides feel advantage privilege. missed couple key submissions , messed tests failed badly. aware team dynamics past negatively affected catch-up work cost team process. honestly think hard says high expectations myself. decision-making process usually based voting. people seen getting things knowledgeable automatically obtain voting power. past experience work usually drive person chooses group role willing play. future , prefer , rules engagement given thorough attention prior commencement group work. way , best obtained everyone. personally believe need place comfortable work tools available. accomplished. achieve contribute .  experience academy far potent insightful year , changed life person really enjoys importance cooperating others. brought amazing minds space really helped acquire soft hard skills short space time , helped elevate way learning new technical subjects efficiently effectively. , management team large especially supervisors instrumental improvising just make sure led way help walk away informed junior data scientists end year , work tirelessly making sure break obstacle attractive easily comprehended possible order hard work. addition , nuri formidable way helping cope psychological illnesses caused intensity program , extra mile making sure feels solitary. supporting structure program importance , does help understand essence excruciating vulnerability management enable does work not. words , want close saying means life , 'm grateful opportunity , best thing happened life varsity , 'm pretty sure set life greater things achievements .  time academy far challenging , extremely rewarding self-fulfilling - especially regards working people different backgrounds domains expertise. meeting different people interesting , concerned bring different people achieving common goal. teams worked , lead , drawing ambition past experience relation data science related fields. challenging , used - comfortable - working alone. hold personal contributions team-based projects highest standards , difficult team members dedication team-based projects do. working teams challenged personality positive negative ways. like control , difficult allow people lead different aspects projects , especially objectively feel experience completing particular objective. , learned impossible control aspects project welcomed aided contributions team members shown willingness aptitude contribute. contrast , difficult form motivate team members apathy achieving common goal. somewhat frustrating , ensured derailed team following initial plan objectives. felt comfortable role coordinator , relished opportunity deliver high-quality deliverables high pressure situations. drive work ethic good , productive worker. over-analysed spent excessive amounts time subjects interesting challenging personal level , instead focusing time pressing matters. generally tend spend time challenging interesting concepts , tend postpone neglect routine administrative tasks. creates frantic dash line time runs , creates unnecessary anxiety. conscious shortcoming , actively try remind prioritise complete pressing matters regardless personal interest. know regarded fellow team mates , focused driven comes task hand. little time chit-chat tend engage conversations related intriguing philosophical subjects. feel comfortable responsive. , learned casual conversation regarding sports hobbies helps learn team members , forges professional , understanding relationships team members. overall , work experience academy enlightening , far perfect. lots learn , eager improve aspect career benefit team mates .  positive experience academy far. enjoyed meeting people data science unusual choice learning institution. fascinating getting know people class. people incredibly wide range personalities backgrounds. generally enjoyed working lab rowdy lunch people 's work motivation decreases motivation socialise increases. team members like-able , felt team dynamic failed dismally properly tackling project. success beginning t ultimately disappointed team ’ s performance. lacked cohesiveness did n't project manage effectively. people focused exclusively individual learning group work. second group considerably better. clear strategy allocated tasks effectively. regular meetings , took minutes , wrote tasks sub deadlines. member motivated pulled weight. communicated frequently balanced individual group tasks better previous group. decision making initial group rational rarely executed decisions intended. lot procrastination members arrived late class left early. group member spent time away group chatting friends. claimed working group tasks , ended contributing absolutely group tasks. contrast , individual second group went duty. regular meetings ensured tasks divided properly. progress closely monitored , problem occurred , rallied solve adapt strategy. member group quite experienced particular topic project tremendous advantage us. thought carefully team workflow set appropriate software working folders. analysed project marking rubric created gantt chart schedule tasks , understand dependencies. manage group member availability member group required periods absence project. scheduled tasks class tests aware time allocated individual efforts. initial group unhealthy diffusion responsibility member assumed pick slack. lead group members despondent refusing carry group members. faced issue group member didn ’ t feel sufficient programming skills make decent contribution group effort. despite shortcomings group got reasonable marks. credit group members. sure second group good marks. credit given equally group members member particularly strong. think efforts equal previous group felt satisfied group effort end .  bit loner times id say experience really taught engage people talk especially student kind goal. based academic level think o got experience programming fact familiar sql python shows grown academic level. knowing wake everyday new faces lab thought able manage knowing loner told working teams thought end m talker people thought m going express opinions , participate happened best experience started releasing need talk order opinion heard. team really understood did teams challenges valued opinions decision making pimplier , team agreed everybody gon na look strength weaknesses. assign activities concerning project forgetting help strengthen academy weaknesses basically created fridays helping other. comes decision coming acadamy hard decisions cape town far home having choose completing honors degree academy. ended choosing academy honors degree felt like needed experience data science marketing world academy best place start fact ought learn beginning great computing skills wanted stipend contributed decision coming here. conclude regret coming instead happy here.its 5 months feel like year just going bit faster want programming skills think got experience programming languages skills doing honors.the experience got far best. helped growing academically helped gain confidence individual .  like meeting new people similar interests , new people , connecting people time goes assigned new group. n't spoken students merely strong introverts , recent group nominated group coordinator lead meeting , interaction acknowledge personal level group members feeling day , really thought experienced , grew mondays share did weekend , really appreciated position challenge grow personal academically. like make decisions based merits decision efficient come attention efficient time does account people 's feelings , decisions past group people feel included left , worth noting individuals did contribute project way , barely showed academy , written single line code left member care making dashboard , presentation ultimately present , took uncomfortable route diplomacy , pleaded morning attend briefings supervisor , went group training sessions impact , group , understood academy meant worked well. stallions , magnificent , sudden paradigm changed , enjoyed having idea best things , originally typed minutes meetings member suggested trello effective , learnt new , excited , day , day went bed knowing didn ’ t know woke ’ s data scientist ask for. important lesson got working different groups leave time guide team work flow miserable end project , planned trello ( doesn ’ t trello ) ‘ tees ’ crossed ‘ eyes ’ dotted , pleasant experience bring best involved , team creativity grow levels planned beginning. , irrespective group work , sit maybe second session “ getting know ” map strategy roles assigned , significant insignificant .  edsa reflections 24 november 2017 , filled joy , excitement anxious time opened email edsa titled “ congratulations ! ’ ve accepted explore data science academy ! ” fast forward 22 january 2018 , ready meet new people ( possibly form new friendships ) , meet edsa team learn lies ahead 2018 , overwelming day was. today 6 2018 , lot happened past 3 months , ’ ve learned ! sql , stats , python , power bi , presentation skills… [ learning , stops : ) ] . grateful selected 100 , ’ m appreciative opportunity , thank edsa making possible me. lab data scientists making , greatly enjoy lab , environment condusive learning , group data science enthusiasts room feel receipe success. training limited online courses edsa staff students learn lot , different backrounds , skills experience making lab interesting fun place grow academically socially. teamwork water crisis project : team liquidata say initially concerned individual work having deliver team well. realized importance effective team , working team reduced work load stress helpful individual tasks. roses regards team dynamics , working different individuals perceptions opinions challenging , frustrating having decide certain matters , helped lot grounded problem statement kept focused needed done. worked ’ m result driven given task , completing task successfully required relevant info important , refering problem statement got stuck great tool us. emotional moments team personality clashes end worked contributed success team , awesome team ! rsa stock market project : wolves woodstock accepted ’ d work new people , really problem , hope wish strong programmer team , wish realised statistician programmer team , lovely ! great team , felt like outspoken person ( ) bit , worked delivered. prefer working team different ideas , creativity different skills ’ s huge advantage eveyone team result driven , absolutelty appreciate that. ’ ve good experience teams far .  day looked familiar faces. started looking people like associate self with. meeting new people fine long did n't expect conversation going. nice , guys hang with. love working labs , share idea 's , findings ways doing certain things. enjoy working teams. believe 2 minds better one. line validity checking. learns lot new things working groups. team dynamics like , knows need doing achieve bigger picture. long term ones ideas change clearer picture , having completed task. ( problems change unforeseen problems come up. ) decisions needs happen. times felt team members did know exactly needed ( just floated ) . decision making based possible overworking ."


In [50]:
NtXt = "  joined academy crossroads sorts life. academy offered opportunity pivot career engineering data science unfairly dismissed struggling find employment result. meeting 100+ ( faculty students ) new people would part life whole year something get used , even though worked multinational companies. ability deal people different backgrounds , beliefs , etc. tested group work. needless say lead tension one 's life team mates pulling weight. team dynamics always learning process/curve , thankfully bad experience till now. needless say , people try cheat system , act dishonestly sometimes plain lazy. many opinionated thus rendered incapable making great contribution. way things always consider everyone 's well-being , opinions contributions. somehow persons think ok cynical unhelpful. end day , supposed learn become data scientists. work , improve daily. .  far experience positive definitely see value course. experience working different people valuable me. worked well attitude everyone team deliver take responsibility. difficult step back trust team work , however cases team members exceeded expectations. felt bit frustrated team members applied taken substandard lazy approach problems. like get stuck something get immediate results improve there. think approach worked well however also learnt allot team members structured task based approach. allot time spent tangents add value projects however tangents either paid results learnings took them. decision making approach generate ideas create structured work flow around ideas distribute work among individuals take interested work flow. conflicting ideas typically took democratic approach combined healthy debate. find build relationships people 'm teams with. get know best. strongest connections team members also people academy learn from. took team bit data collecting tangent last sprint necessarily productive use time could avoided understood problem better. one case probably assertive team , however way like make decisions get stuck workflow stream adapt go n't supervisor direction point. find environment stimulative enjoy going around teams see working on. good relationship people happy share working great. also found environment distracting times , would like help people get satisfaction , however 'm also that. working different people mixing regularly something im excited everyone different learnings experience theres something learn everyone .  fortunate opportunity join academy year. sure career choice , academy provided great chance change career paths. help choosing new path allow eventually reach larger goals life. moving cape town joining academy challenging , already knew two people academy joining. however made bit difficult making new friends. typically reserved , easily make new friends , especially already good friends , case see reason make new friends. working teams academy great challenge me. normally prefer work pace working team forced cooperate people adjust skill levels. sometimes meant taking work allocated me. addition , found difficult working individuals invested projects aiming learning much experience intend to. within teams , leadership quite lacking. typically like forcing onto people , circumstances , forced take lead projects. taking lead tried best lead behind , making sure consensus within team deciding. generally due lack experience one teams , meant think plan , execution thereof , followed team implementing without challenging ideas laid out. approach worked well , would liked greater input team , unsustainable left would render team incapable functioning properly. team , people much competent four members , decisions much easier take. good idea , still elected leader , role much organic time , much balanced consensual way making decisions. however , completion tasks effectively still perfect left. teams struggled making decisions , never experienced conflict generally emotional making decisions .  looking back one got academy right , confidently say grown intrapersonally interpersonally optimistic perception working also gradually shifted realistic end spectrum. came nothing positive outlook journey gotten opportunity embark , however , learned couple valuable lessons - one stand value understanding teammates expecting one understand you. put far put cap assertive nature gotten lot f pressure given opportunity even sharpen softer skills tends help quite lot even outside academy good vibe generator - one deflects negative energy positive energies whatever spheres happen find in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. learned focus making sure ’ expected delivered timely miss beauty process focussing relatively small hurdles ’ vast amount captivating material engage in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. 've also learned never bring emotions workplace people tend step sensitive things without noticing- example someone staring shoulder commenting work 're quite unpleasant kills moral detaching focusing 's required best coping mechanisms .  got academy , felt like n't belong here. felt place. person n't like talk people own. people approached , got know them. getting know fellow classmates , made realise feeling similar emotions 'm feeling. 've met lots like minded people. personally like work alone , working group always difficult me. academy made easier me. forcing work teamwork skills. often , 's group member takes charge , keeping team focused shared end goal .  attending explore data science academy already proven , continues amazing experience. ’ quite astonishing realize much ’ learned mastered past couple months , chief among would ( subject essay ) , teamwork. meeting new people daunting task ( including ) . however academy different start. sheer excitement students especially explore team ( supervisors facilitators ) acted social catalyst , smoothing social awkwardness , knew getting along like house fire. working teams people completely different backgrounds personalities challenging throughout first project , said , also interesting see individual adapts accommodate team help build team dynamic. starting cape town water crisis project honestly idea expect , certainly ’ expect team dynamic play huge role success deliverables. team overcome different levels knowledge , skills , self motivation. couple late nights flared tempers managed get work done , realised overlooked crucial part teamwork. one need self motivation , also create environment team motivate comfortable enough ask help ’ feeling overwhelmed struggling get grips difficult concept. team definitely lacking aspect , take chin regards deliverables , walked away learning priceless lesson moving next sprint. assigned new teams rsa stock market sprint , refreshing hear ’ team struggled certain aspects team dynamics. get go approached structured interactive way things. decided stand meetings every morning report previous day ’ goals well setting new goals day ahead. already made huge difference motivation team. another part success dividing work ’ get overwhelmed individuals rather sharing workload one-on-ones relevant team members gain deeper understanding section work. approach even advantages thought. structure communication team members lead lower stress levels ( even though still ever present ) , resulting cooler tempers clear headed decisions less emotionally-driven first project. goes show “ soft ” skills important technical skills working teams. ’ wait learn brilliant diverse people academy. '' individual commitment group effort -- makes team work , company work , society work , civilization work. '' -- vince lombardi  wonderful place , place waking morning easy task great desire learning something new everyday. meeting new people great thing learn work team also socialize since person well crowd. actually first time working lab like reason experience labs quite instrumentals labs testing experiments computers , n't computer labs first time coding lab reason coming edsa first choice . skills data scientist one drove apply wan na learn coding. learned lot lab also working team. one main problem faced working team group rather , groups perfect might wish change disaster.sometimes set time slots group meet discuss progress deliverable surprised one member pitch thought lot work , disappointing would mostly prefer working alone future. working team sometimes emotional draining decision divide conquer could work together well. hoping great team smart people need told since every sprint `` lists '' .  essay discussing personal experience regards academy like meeting new people especially introverted person am. speak different teams , different different felt , learned different characters. comparing decision making done teams versus personal decision making. getting plane cape town without place stay friends city big risk , certain plan academy kept praying want scam plot get specific building get kidnapped. fairly surprised get meet people walks life , also kind questioning legit , mean hard question , sounded way good true good things learned ’ happen people like us , guess time shine receive blessings. ’ think academy , blessing. said , experience academy ups downs like always ends up.i learned lot , fallen love data science , things small mindedness used consider geek ’ s. learned push , always try learn , practice computers always evolving. since met people different personalities academy , personality clashes occurred 1 biggest lessons , able work different people different personalities. working teams ’ easy thought would first , everyone idea assumed supposed done different opinions could dismissed everyone 's opinion valuable working team , opinions somehow seemed clash. working teams taught patient , quality possess whatsoever. second time changed teams though seemed though everyone gotten idea working teams ’ like , basically dipping toes working teams thing first sprint , second sprint bit confident meant worked better second teams , improving go. teams , split work equally amongst , making sure everyone gets chance tackle important lessons like coding dashboard. group coordinator state views , he/she would speak supervisor decision would made way , supervisor would guide us right direction hearing thought right way.its almost concept use guess making decisions , weigh options , see one best one worst outcome obviously go best outcome. conclusion working teams skill gaining developing everyday , ’ mastered , heck difficult master many personalities , getting .  experience academy enjoyable far. learning lot interesting valuable skills meeting many intelligent interesting people. meeting working new people lot fun rewarding experience. working lab challenging. open plan allows many diverse unusual conversions easily started joined also prevent work done people interrupt prevent others entering staying state flow necessary learning. especially destructive slower learners like myself. working teams also challenging insightful. sometimes tricky setting understanding team dynamics new teams especially people ’ know well , practice working teams always appreciated team dynamics always easy decipher understand. decisions made group ( least feeling ) without one person charge giving orders. could also part still learning try make effort person group feels opinion asked heard decisions made group decisions everyone opportunity raise differing opinions votes support. consistent way enjoy making decisions think allows everyone feel involved dedicated outcome everyone feeling others group think feel decisions made. say decision making rational emotional know accurately determine that. maybe mix , though slightly leaning towards rational .  started academy workload easily manageable students able keep learning curve , whereas struggled either asked peers caught , n't got stuck behind pack. workload difficulty increased , able keep better programming foundation others course. although might helped lot , appeared blessing curse. found losing lot time could spent work , helping students ( bad thing ) , questions asked topic currently working found difficult get back line doing. decision teams need made , seemed myth. team needed carry lot workload rest team members order get scores wanted , trying teach team thing two long way. similar situation second sprint , severe , 2 members team got flying start , 2 spectating participating one eventually found feet ended bang , last member , felt , never contributed final `` prediction '' , speculations. quite frustrating like working teams , however members carries weight. might sound selfish , n't want affecting results much goal year 1 30 students going bcx. feel working hard enough spending enough time working , still happy ultimately hand present group. finally happy course content. course already taught lot data science thought knew ca n't wait come. awesome experience happy chose .  coming academy best thing ever happened grateful opportunity. learned academy going encourage teamwork excited meet new people different spheres life exchange ideas network it. working teams thus far ok. things like team commitment work work ethic thereof. comes decision making n't enjoy part felt sense dictatorship member would defy group 's decision still want impose best team start delegating work teammates based vision. felt defied purpose brainstorming planning objectives. role played team always conscientize group imperativeness working within scope prioritizing required us form basis going structure project would help decision making. supervisor meetings reporting based `` '' , `` '' . sign individualism collective project perpetrated unnecessary competition. enjoyed networking variety people academy also participate win hackathon hosted city capetown one best networks team collaborations went well me. like random conversations technology trying link 'd connect inspire other. brief summary work prefer plan try much possible stick plan always room changes. consolidation key helps deciding conflicting ideas .  past months one challenging thrilling experience life. coming , expected engage people knew introvert , experience academy would force engaging people. something totally outside comfort zone. ready ; much prepared , ready challenged understood growth reached uncomfortable spaces. experience taught could imagined. learnt fearless embrace greatness within me. come across shy , willingness push things enabled speak room full people made realize get nervous things actual fact get nervous things capable also fail. way scared inabilities potential. understand still lot learn fully comfortable know feel still lot knowledge gained. coming bit theoretical statistical background , interesting see techniques learnt university applied real world scenarios. coming space , space full well equipped individuals , felt bit intimidated idea career going lacked confidence really good at. experience far definitely given better understanding skill set career take , importantly come learn alone. able meet lots people also similar situation , simply shown worries worth worrying things fall place. environment quite relaxed , yet taught behave professional workplace. support peers also tremendous. observing everyday events supervisors taught teamwork , people come together get things done . although sometimes remind use inside voice , feel adapting environment relatively well. going forward , expect learn develop skills .  experiences explore data science academy experience meeting lots new people great one , months already learned lot , continue daily level. terms meeting new people much gain , especially terms offered different perspectives , challenged way thinking. also taken comfort zone turn taught lot handle different personalities. experience working together lab teams much appreciated close real world experience , 've already similar environment throughout university always found enjoyable interact people. level productivity performance exponentially increased , mainly deadlines needed meet wanting disappoint teammates , generally working others helps achieving individual. say interpersonal skills increased , example speaking listening others. come realize collaborating others helped identify strengths weaknesses , basically self-aware. experience decisions made teams was/am consistent like make decisions. lot people considerate want attain goal , appreciated , far people worked take time listen person say try take consideration relevant concise , decisions made depending everyone felt. experience rational emotional decision-making process been. decisions made rational group took time take people ’ emotions consideration possible. ’ obvious always agree completely certain things , occured , time taken explain person things certain way beneficial way disagreeing member thought. accepting possibility incorrect important role , especially comes working others. conclusion , experience positive anything academy .  experience far great though first bit anxious excited whole program , working teams challenging first still becoming less less challenging , far groups far completely different 's interesting terms knowing works n't , also helps experiencing different thinking processes ...  experiences edsa experience academy intense , interesting fun. even though met small subset people academy , great gotten know time. also particularly interesting , seeing different everyone ’ background perspective is. learning come want life chose apart academy. working lab great experience , reminds worked open plan office space feel like ’ great comfortable environment work in. also proven difficult times , slow computers noisy environment keeping focused sometimes tricky , views mountain really great ’ complain much. regards team dynamics , two groups part different terms worked , however overlap dynamics. one dynamic found two groups extroverted individuals groups took leadership role early team took cues two groups pushed leadership role. part groups introverted individuals extroverted ones. ’ major conflicts clashes dominance. first group differences personalities caused friction conflict. despite , got long worked together cohesively meet goals. natural personalities would create conflict never lost sight there. regards decision making two groups. first group lot uncertainty terms approach problems created environment aimlessness uncertainty times. second group came consensus based strengths team members would allocate roles accordingly. decisions first group made based high emotions think caused new environment learning new skills created stress , coupled trying meet deliverables made tensions high. whereas second group came decisions based logical approach. would say overall decision making rational tried deal problem statements thoughtful planned manner would allow us cope coordinate responsibilities effectively .  data science academy amazing path far , one room people ambitious love learning got excited work also supervisors facilitators determined love job motivates even keeps thrilled like one day. thankful part academy , much learn everyday proven need specific field certain level education data scientist , way academy structured students fields various level education think great idea , would see someone never studied computer science statistics knowing concept better degrees , brief say need order survive academy focused know expected everyday. fortunate enough placed teams people determined never struggled get group mate deliver expected , hence said people academy focused know for. think selection team good job placing us academy. appreciate also fact much supervisors facilitators busy , always accessible help us work passion , see getting better better day become data scientist certain end year level corporate expects be. academy coaching sessions assists come across obstacles program , work related personal problems , someone talk share predicaments going , find helpful workload much might loose track things , especially something distracting going personal life , deviate attention work , think service useful personally used load shoulders shared problems someone trust judge share problems anyone. randomly placed teams believe help us come across different personalities people learn work around , although people hoped , believe prepares us fit enough deal people meet corporate world. also learn assist patient people might struggle certain concept also learn others .  good day , really n't know start essay besides good day. anyway 's way lets start business. moments academy mostly up. let start day one meeting students south africa great , meeting people different personalities coming different background. people different races , culture tribes ( enjoy stuck cape town since day born. ) . getting first team , part crazy , fun team. guys n't look one another open , strength. first weeks everyone working well getting close end became frustrated team members started lose focus ( assume got tired sprint long ) . following team great came work bonding n't all. two members liked working home ( productive ) two quit time. came decision making , meter bouncing ideas around choosing one move stayed one teams. second team split two teams two teams thing 's believe new way things great. submitting individual team one learnt skill taught sprint. thanks .  good fun experience though 's often case pressure. lab allowed enough time practice get used coding , improved efficiency coding working computer. coming academy n't used idea working team , along year got better , knowing someone count , rely go need help. also got know personal level makes easy understand , strength weaknesses allows us form strong team. sometimes disagreements think right , effective solving , lucky enough co-operative team members since beginning year. though n't got know many colleagues ones got know would say pleasure meeting  experience explore data science academy great far. goals explore year develop & master skills data science well improve leadership skills. felt empowered content ’ taught. four months , believe already learnt skills would allow enter & succeed job market. next 7 months plan spend refining data science skills using 1 hour evenings revising content learnt class. regards second goal ( improve leadership skills ) , ’ fortune team great coordinator & excellent team dynamics. also great team coordinator team. ’ taken many lessons experiences read book. given practical lessons improve leadership skills. ’ two teams far , worked first 1. coordinator nudged us always making progress towards bigger goal. 2. teammates voluntarily picked segment project would responsible for. 3. loose deadlines agreed , deadlines also allowances members equip skills required fulfil parts project. 4. team met twice week discuss progress. 5. time member grew comfortable asking help others well given feedback. team dynamic encouraged member give part project all. inevitably , team members technically advanced others ended taking advanced work rest , however time felt like overburden. given positive experience eager also try coordinator next sprint came around. new teams assigned & volunteered coordinate. aspects believe could done better. 1. ’ commonly accepted bigger goal. 2. ’ involve team identifying small goals were. worked work plan overnight & presented team. team loved plan , individual responsibilities work assigned. 3. led situation members team clear responsibilities were. 4. result power leadership , members team ended bulk work. able pull together & present satisfactory project , however believe could led better & helped team deliver far better project. member team would also proud contributed fully project. learnt great deal experience & would like give coordinating another try , like skill believe leadership learnt practice .  academy amazing , love petry dish talent ! handful genius characters here.. backgrounds unique stories , explore data science academy best place develop data analysis skills. love time ! far , 've experienced whole lot pressure change something better. spend time studying , better feel , comfortable become closer get quality employee entrepreneur. everyday come office , feel home , learn something new.. something directly use business application. look around entire building get inspired start business 've learned here. speak among business academy helping us realize ideas clearly. without information back ideas , 's difficult starting. availability information outstanding. really appreciate nest. last two teams great work with. achieved bunch , , thanks resources available. first sprint tough though. team , realized strengths taught other. n't problem sql python used technologies before. understanding statistics applications data given difficult. worst , water crisis data n't collected well. n't worst experience data taught things career perfect. perfect skills data cleaning. realize limited number solutions data analysis. perfect outcomes data science , could breeze work future. believe hyper prepared things.. game theory data science would prove beneficial. emotionally , 'm great. academy brings best worst skills provides controlled environment mend holes analytical judgement. relief , emotionally physically becomes process bettering daily , bit bit without forcing stress .  experience edsa far combination challenging , fascinating , unpredictable. coming academy , excited would get meet new people learn new skills ; anticipated might somewhat difficult engaging new people ( since introverted ) coming grips new technical ideas ( creative technophile ) , since loved good challenge , still excited immerse experience. surprise , challenges faced none anticipated generally able interact cordially new people understand technicalities program. greatest challenge far working lab. environment many people ( different personalities , habits , learning techniques preferences ) forced natural inclination isolate working. easily distracted , think carefully navigate space simulate environment conducive learning. also found team consider optimally ration work , challenging , especially beginning n't yet good understanding 's strengths weaknesses. said , gotten know fairly well , able make decisions believed would amplify strengths diminish weaknesses could produce results best possible quality. sense , much decision-making teams rational rather emotional. would apply logic reason wanted achieve thought might able , found always room empathetic thoughtful ( thereby engaging emotions rather reason ) , sake team member , alter original plan. somewhat consistent personally make decisions : think times base decisions think yield best results terms trying achieve , even happens inconvenient taxing process ; times , especially decision made great consequence , make decisions based emotions believe 'rational ' 'logical ' decision likely hurt someone 's feelings cast bad light. , learned awful lot : preferences , strengths weaknesses , character ... found learning curve steep emotions rather turbulent throughout process , enjoyed every part challenge could mean future -- prospect holds together worst days exhilarates best days .  imagine excited finally meet people like minded. maybe could rephrased meeting people like me. however , instead , meet small sample different types people different types personalities. yet find stuck world different types people different , different interest ambitions , translates make decisions every day. decisions affect , especially working teams. hand , decisions make affect team processes. according personality test , personality defined five letters entp-t , according classification debater. true debater , convinced , play along suck test discuss personality affected general experience academy. lab allowed us access well collaborate , although working effectively lab challenge , many distractions. found easier get know people type strengths , knowledge background passions general. type knowledge allows efficient knowing ask help different things. prefer asking facilitator. always done past prefer working challenge peer , along process learn would otherwise. comes working teams , always , first one speak give opinion came variety ideas , ’ extroverted side me. therefore , end leading , team. role prefer police people , therefore either allowed people lead would make sure project requirements completed end. last sprint dedicate responsibilities according people ’ backgrounds strengths interests. one person leading deliverable project end would ensure requirements submission met. employed similar strategies past , working tangibly people bring table. framed decision making teams worked in. someone shows work work have. approach made easier focus objective deliver projects. decision making rational time little emotive times , irrational judgements employed make assumption people attitudes toward project work. creates barrier people pulling weight past immediately , integrate teams later. however team dynamics forced finisher .  experience edsa interesting one thus far. pushed get shell interact people. since academy realised smarter think lot valuable knowledge insight share. biggest fear coming academy social aspect interact different people , since able overcome fear. first placed groups literally sweating , heart rate high felt sick stomach idea group strangers. second time split new groups ’ nervous. become social enjoy meeting working new people now. working groups challenge me. frustrating everyone different everyone different approach learning process project management. ’ learned lot working others. took backseat things even knew better way things. spoke saw helped team steer team better direction. become assertive. thought passive person ’ assertive save life , think assertive ’ find daunting. biggest frustration working teams dealing people think know everything ’ willing open learning something new fellow teammates. person like previous group. person would dictate team ’ want collaborate team , dominating personality rest us shy. realised none us ’ speak ’ fail sprint forced breakout shyness. think still fairly shy comes work fight shyness. decisions previous group initially made one person. prefer problems discussed everyone team different possible solutions explored come collaborative solution. found current team. still issues , people letting pride emotions get way work , people distracted ’ committed project. working labs challenge. lot distractions noise focus easily , prefer work home , get much done home. people come academy socialise anything distracting .  past 5 months explore academy challenging. make great adjustments order here. positive negative experiences. one positive experiences much learned. met lot people highly intelligent interests. worked 2 different teams. prefer work alone working team makes sense due sheer volume work. first team experience incredibly frustrating. team members highly introverted technically strong made discussions assigning tasks difficult. even tasks assigned accountability meet deadlines. prefer chip away steadily work opposed rushing close deadline finish. working second team much better. met regularly , communicated lot. tasks assigned everyone accountable. team members helpful towards .  experiences academy far life-changing say least. meeting smart , passionate , hardworking ambitious people different backgrounds happen many similar interests mine interesting journey. introvert prefers working alone pace within space , quickly find way adapt polish social skills order socialize 99 strangers going learning data science with. understood word go need socialize form meaningful relationships fellow interns could see forming strong network may helpful many years come career , long internship. academy ’ emphasis importance team work concept took warm to. teams exposing different individuals varying academic , social , work geographical background different personalities : introverts , extroverts , intuitive , artistic even really funny characters. teammates phenomenal experienced fields previous exposure to. learned lot them. equally ’ also encountered independent disagreeeable teammates required adjust levels patience , tolerance understanding may necessarily share similar beliefs me. need patient tolerant people ’ points view approaches solving completing team tasks. teams , due differing views , opinions approaches use democracy ( voting ) order reach decisions within groups. although easy one resort taking emotionally-driven decisions due one ’ firm beliefs ways things high level emotional intelligence critical favourable team dynamics. academy based cape town , city ’ visited holidays before. moving cape town challenges required solid ‘ family ’ away family fortunately found fellow interns resourceful many ways even outside ‘ labs ’ . example group share information accomodation , transport around city , social events extramural activities. chance test team-building skills virtues regular group sessions nuri believe helpful giving idea relate people less formal settings. credit due supervisors , facilitators oprerations officer ; helpful making internship experience manageable definitely one memorable years come. commitment programme makes easier people personalities settle nicely within diverse small community .  experience academy far interesting 4 months although apprehensive meeting work new people , still enjoyed work looking forward comes next. group work team dynamics particular challenge although ’ really socialised colleagues since arrived cape town , friendships formed mostly students live house me. team dynamics challenge fundamentals sprint ’ fair division responsibility well accountability team. group members would slack part hard hold anyone accountable concerned nice. decision-making group fundamentals sprint mixture rational emotional , things right things badly one wanted take responsibility , much passing buck. team also beset laziness unwillingness really get learn needed deliver project. rsa stock market sprint , group somewhat better people hard-workers divided responsibilities early based perceived strengths individual. really serious problem group communication different areas functional responsibility , clear group members prepared presentation fully understand analysis analysis team performed. decision-making group rational. working labs , nerve-wracking experience significant proportion people simply go socialise distracting us , prefer working environment absolute silence ( hence prefer working midnight ) simply think problem allow creative possible solving problem. people assist thought-processes terms thinking problem solutions irritating distraction. around many people long eventually lose focus problem focus people , would much better silence mandated enforced labs. also need individual work instead group-work. suggests ways could contribute group signs learnt lessons emerged previous group , still lot trepidation comes next. hope meet group members committed project , otherwise , everything end tears .  enjoy meeting new people different walks life constant , everyday social interaction. slightly reserved comes approaching people much older me. also fear inadequate unqualified many people already worked professional environments qualification , undergraduate otherwise. large emphasis group work big learning curve always preferred work alone everything way want , although improved since time here. tend take charge unless one else better talking people helping find role. versatile n't mind taking least desired role despite preferences. one group decisive. general guideline try stick although hardly case. prefer versatile approach work. rather consistent method making decisions. little less organised group though. tend procrastinate make major decisions complete important tasks last minute. tend sway easily depending emotions. know let emotions run ahead ( since something known ) put foot , take step back examine things rational perspective. hard easy put emotions beside .  honesty enjoying time regret choices made. really educational love structure , environment one force work push much facilitator really helpful bounce ideas see far able explore imagination. meeting new people 's always nice stories people far go life , combine different cultural group things got learn people 's culture stories tell. work together 's really tricky fun times tricky , matter many times think always tricky talk wont enough words left. making decisions 's simple everyone agreeing goals deadline arrived people stories , 'm unable relate excuses , honestly wish everyone could adhere words , really would like decisions made like explore lot 'm working something due time constrain sometimes 's best expected enough time explore more. team members sensitive way sensitive leave part there. love academy wish time learn bit , enough resources even conduct maybe research , instance know write good machine learning algorithms n't understand work might even able improve .  starting academy wonderful experience. cape town beautiful city location academy could better. meeting different characters really experience.in first weeks meeting new people daily basis sometimes intimidating. teamwork never really strongest skill always something would like improve upon , heard academy strong teamwork emphasis happy.the academy working lab strong social element , even though work learning somewhat technical. first sprint , quickly discovered diverse group was. vastly differing amounts experience fresh high-school graduate someone working decade. also differing skill sets. working groups challenging rewarding time. cape town sprint primary downfall lack planning what. resulted us almost late deadlines. communication within group could lot better. tend keep flexible approach completing tasks , flexibility ended costing establish proper deadlines .  academy challenging experience. expected prior coding knowledge help minimal help , greatest challenge statistics. slowing getting point n't actually try understand concept thoroughly rather get basic theory focus application. overall , insightful journey , learning interact different personalities group dynamics. really excited leaning nlp regression incorporate app development monetize it. genuinely regret applying academy although days doubt hurdles remembering greatness opportunity always great motivator .  big part joining academy year meeting working lots new people. unexpectedly positive experience. despite everyone different backgrounds , sense camaraderie interns friendly. even though everyone tested graded environment non-competitive people willing help other. made enjoyable work lab. however , noise bustle lab sometimes overwhelming draining find productive sometimes spend hour two day lab working home quiet. challenging aspect year far teamwork. first project ( cape town water crisis ) times conflict team due team member contributing much therefore shifting workload team members. led resentment team members , expected. team projects work rarely equally divided team managed complete project despite difficulties. reflecting experience grateful done big share work gave opportunity learn lot would otherwise. second project ( jse stock market sprint ) much shorter time span therefore team approached project different. team divide work time everyone involved deliverables. hard meant trusting people work without directly involved dependant complete it. process decision making teams democratic. team discusses problems possible approaches usually obvious decision everyone agrees on. decision making far tended rational team members emotional. emotional part projects stress faced final deadlines neared. , experience academy far positive learnt lot. hope improve way approach team work upcoming sprints .  four months since decided embark journey. packed bag , moved new city took first step living best moments life. love meeting new souls. always fascinating finding someone else ’ journey began , similar different footsteps left are. amongst 99 brilliant minds incredible , getting know refreshing. people stand , everyone unique. always good working , taking reins. ability work effectively team acquired skill , understand different personalities , people deal problems communicate. delegating seemed help terms getting work done teams , person taking piece , sure finish elephants. decision-making process structured , benefited whole team. guidelines rubric supervisors helped keep us right track. decision-making mostly based impulse , following process ideal getting results. great team members pay attention detail since mostly focus bigger picture. made sure work appealing. quite experience , constant learning. growing skill set , including soft skills. beginning , lot learn .  explore data science academy 19 weeks ( counted ) ; let ’ simplify 3 months shall ? exciting working group challenging problems , feel like world possibilities opened us. exciting overwhelming met many new people. right away seem made name myself. proud , even excited people feel someone come help - practical emotional. helping others role comfortable since typically excelled academic environment , ’ admit disappointment people ’ ask help more. like thought expert ( even like feel like expert , find happens less grow older ( experienced ? wiser ? one old greeks said something wisdom knowing ’ know ) . aspect larger group find overwhelming trying remember everyone , especially since many people seem know am. enjoy learning people ’ backgrounds amazed great selection people attending college. experience working intensely type team entirely new. worked teams school , university work , never amount diversity team. never sitting next team members everyday ; majority one ’ efforts focussed team work. immediate challenge first team working people skills ways operating different own. realised career worked people , background training. made less adaptable less tolerant different approaches problem. trying learn relax give people ’ method approaching problems chance. felt comfortable group leader first sprint , uncomfortable giving reigns second sprint onwards. second project usurped actual leader number occasions seldom pulled back. formed real bond first team , despite ( ) struggles. second team relationship seems formal , although things went much smoothly team arguably effective. found frustrating deal reserved people. hard read difficult reserve judgement ’ produce tangible outputs. still trying figure best deal people , draw shell patience hear trying say. also need consciously tell offer may valuable ( remain unconvinced evidence right table ) . experience also underlined much rely external encouragement praise , played interactions students staff. enjoy teaching students , specifically one one small groups , alot immediate feedback. first weeks felt really supported facilitators , later weeks felt like abandoned newer , exciting projects. similarly felt much secure first mentor ’ hands-on , available , approach second facilitator ’ reserved , hands-off approach .  young age always sheltered , reserved terrified getting comfort zone. knew year would challenge ’ challenged break shell. first day got real knew one person hundred. push open interact crowd ( struggle considering ’ shy standing front people makes extremely nervous ) . lucky people academy friendly understanding. working teams never problem , issue able fully center attention speak thoughts. taught importance communicate ideas without shy. one biggest challenge working team dealing teammates ’ fully committed tasks assigned them. personally prefer get things done certain way bit challenge team timelines ’ met team members ’ putting much effort everyone else. lab exposes us different personalities one needs learn work together effectively many personalities many conflicting personalities could recipe disaster. could say lucky teams assigned ’ difficult. understood importance contribution communication. misunderstanding ’ address becomes conflict , way able prevent it. teams decision-making process pretty much same. generally , challenge put team , communicate strengths weaknesses , able distribute tasks together based strengths weaknesses. decisions need made , every member given opportunity give opinion decide together team taking consideration every member 's contribution. overall , experience far great. learning new things everyday. workload get challenging time time managed. slowly coming shell , talking , interacting opening .  experience academy really mixed bag far. initially `` '' whatever decision making process needed order get ahead , new group , 've participate others made feel `` follower '' leader times. 's really nice working people myriad personalities. n't really think 've come across situation tempered emotionally although think quite emotional person. think far decision making way prefer ; logical straightforward also room change improve personal knowledge  find experiences academy interesting fulfilling. really like working smart people explore gives abundance. personality clashes character development hurdles point need growth adjustment. one real challenges find balance zoning focus work teaming discuss ideas others help others. choose start would rather acted like focused introvert would give time learn without harming learning curve. honestly contributed negatively group members know , well , deliver more. working together lab thing made real progress growth us individuals. crucial team decisions easier make. working lab also downsides feel still take full advantage privilege. missed couple key submissions already , messed two tests failed badly. made aware team dynamics past negatively affected catch-up work cost team process. honestly think hard also says high expectations myself. decision-making process usually one based voting. people seen getting things knowledgeable automatically obtain voting power. past experience work usually drive person chooses group role willing play. future , would prefer , rules engagement given thorough attention prior commencement group work. way , best obtained everyone. personally believe need place comfortable enough work done tools available. accomplished. achieve contribute .  experience academy thus far potent insightful year , changed life become person really enjoys importance cooperating others. brought together amazing minds space really helped acquire soft hard skills short space time , indeed helped elevate way learning new technical subjects efficiently effectively. also , management team large especially supervisors instrumental always improvising make sure led way help us walk away well informed junior data scientists end year , hence work tirelessly making sure break obstacle attractive easily comprehended possible us order put hard work. addition , nuri formidable way helping us cope psychological illnesses caused intensity program , always go extra mile making sure one feels solitary. supporting structure throughout program importance , help us understand essence behind excruciating vulnerability management enable see work not. last words , want close saying means life , 'm grateful opportunity , best thing ever happened life besides varsity , 'm pretty sure set life greater things achievements .  time academy far challenging , yet extremely rewarding self-fulfilling - especially regards working people different backgrounds domains expertise. although meeting different people interesting , concerned bring different people together towards achieving common goal. two teams worked , often one take lead , drawing ambition past experience relation data science related fields. challenging , since used - comfortable - working alone. hold personal contributions team-based projects highest standards , therefore difficult team members show dedication team-based projects do. working teams challenged personality positive negative ways. like control , therefore difficult allow people take lead different aspects projects , especially objectively feel experience completing particular objective. however , learned impossible control aspects project therefore welcomed aided contributions team members shown willingness aptitude contribute. contrast , difficult form motivate team members show apathy towards achieving common goal. although somewhat frustrating , ensured never derailed team following initial plan objectives. felt comfortable role coordinator , relished opportunity deliver high-quality deliverables high pressure situations. drive work ethic good , always productive worker. over-analysed spent excessive amounts time subjects interesting challenging personal level , instead focusing time pressing matters. generally tend spend time challenging interesting concepts , tend postpone often neglect routine administrative tasks. creates frantic dash line time runs , creates unnecessary anxiety. conscious shortcoming , actively try remind prioritise complete pressing matters regardless personal interest. know often regarded serious fellow team mates , since always focused driven comes task hand. little time chit-chat tend engage conversations related intriguing philosophical subjects. feel comfortable responsive. however , learned casual conversation regarding sports hobbies helps learn team members , forges professional , understanding relationships team members. overall , work experience academy enlightening one , far perfect. lots learn , eager improve aspect career benefit team mates .  22nd january 2018 , time 7:00. seemed like normal day time. hit , difference location. 'm western cape , cape town precise. moved away home embark journey become data scientist explore data science academy ( edsa ) .today 8th may 2018 lot happened since ... first week got meet students different family background different level education. met explore team , included supervisors , facilitators nuri. told would working groups fives. nervous got used everything own. actually terrified. first group called optimize prime ( still cracks ) . made nervous everyone post matric qualification. beginning thought underdog , realised competition. mean must 've reason teamed guys , learn contribute wherever can. next group , named wolves woodstock ( named movie wolves wallstreet ) , confident engaging. developed previous group. even headed one sub projects. experience amazing great. even sister n't believe me. first group given opportunity work cape town water crisis , investigated demand supply factors affected cape town water crisis. project got use sql powerbi first time. saw another use python n't know capability , analysing data. second group got explore rsa stock market data. used data predict target return individual companies. interesting project accuracy predictions , limitations models. group assigned supervisors. got chance mentored two different supervisors. wish could create table comparison , two different , effective. times stats , python others gets much. group sessions nuri , refresh .  positive experience academy far. enjoyed meeting people interest data science also made unusual choice learning institution. fascinating getting know people class. people incredibly wide range personalities backgrounds. generally enjoyed working lab though get rowdy lunch people 's work motivation decreases motivation socialise increases. found first team members rather like-able , felt team dynamic failed dismally properly tackling project. success beginning ultimately disappointed team ’ performance. lacked cohesiveness n't project manage effectively. people focused exclusively individual learning rather group work. second group considerably better. clear strategy allocated tasks effectively. regular meetings , took minutes , wrote tasks sub deadlines. member motivated pulled weight. also communicated frequently balanced individual group tasks much better previous group. decision making initial group rational however rarely executed decisions intended. lot procrastination members arrived late class left early. one group member spent time away group chatting friends. claimed working group tasks , ended contributing absolutely nothing group tasks. contrast , individual second group went beyond call duty. regular meetings ensured tasks divided properly. progress closely monitored , problem occurred , rallied together solve adapt strategy. one member group already quite experienced particular topic project tremendous advantage us. thought carefully team workflow set appropriate software working folders. analysed project marking rubric created gantt chart schedule tasks , understand dependencies. also manage group member availability member group required periods absence project. also scheduled tasks around class tests aware time allocated individual efforts. initial group unhealthy diffusion responsibility member assumed someone else would pick slack. lead group members becoming despondent refusing carry group members. also faced issue one group member ’ feel sufficient programming skills make decent contribution group effort. despite shortcomings group got reasonable marks. credit go two five group members. sure second group get good marks. credit also given equally group members one member particularly strong. however think efforts equal previous group everyone felt satisfied group effort end .  nan  nan  first day first looked familiar faces. started looking people would like associate self with. meeting new people fine long n't expect keep conversation going. found nice , guys hang with. love working together labs , could share idea 's , findings ways certain things. enjoy working teams. believe 2 minds better one. also another line validity checking. one also learns lot new things working groups. team dynamics like , everyone knows need achieve bigger picture. long term though ones ideas change clearer picture , completed task. ( problems change unforeseen problems come up. ) decisions made done still needs happen. times felt team members know exactly needed next ( floated around ) . decision making based possible without overworking everyone .  enjoyed time edsa far. love meeting new people first time life talk people face-to-face programming topics around data science. previously strangers internet. great discussing algorithms coding challenges geeking people understand contribute , rather watching people 's eyes gloss describe trying achieve ( laymans terms ) . love prescribed uniform everyone comfortable naturally ease , opposed fashion parade. countless times struck conversation work , weekend even mountain. sometimes team members get distracted abit long guests talk abit loud , n't big thing. biggest challenge balancing studying implementation theory everyone page. team suffers individuals pulling weight put strain rest. luckily found colleagues kick around ideas discuss solutions current problems. decision making process voting best way forward. typically consensus reached roles , typically 2 3 members end bulk work. conflict resentment arise far real flare ups encountered , hard individuals argue decision made present put work. tend keep peace expect everyone act respectable adults. hate micromanagement love freedom given solve problem statement. glad able view teams ' presentations see went solving problem. continue learn lot data science well working time time constraint seeing people manage. even workload others offer time assist need help  experience edsa one sparked lot introspection. socially , made affirm fact extroverted person cherishes human connection , communication cares 's well-being. regards team-work , made realize quite assertive prefer logical structure comes completing tasks fulfilling set roles. made also realize end taking charge situation group , even n't initially bargained , mainly seems like n't much direction group time seems going waste. decision- making process rational one mostly times based emotion people would choose certain roles based believing good certain thing interest end n't end achieving/delivering much end burning out. 've also realized things n't go according plan comes activities part role , stress feel defeated easily perfectionist side creeps end completing needs done time. future teams wish branch attempt master every different skill best ability instead focusing much individual role , lot potential know capable ."


In [51]:
StXt = "  first 4 months edsa filled many new experiences ; challenges , others joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people everyone met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates within edsa , provided insight gained experience tools like python & sql. also cherish able build network data scientists tomorrow well established industry professionals.. -- -working together lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious con would noise distraction - least say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience would say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes full responsibility part ) members first group noobs sql & python , gather analyse phases really exposed technical deficiencies resulted lot frustration came time delivering tasks. experience took comfort zone important lesson patience us all. second group high level experience , strong team co-ordinator team members took full responsibility tasks. even though work complex comfortable team dynamic. -- -how decisions teams made consistent make decisions -- - team decisions groups made deliberation. decisions easier make person presented clear points opinion was. group dynamic strong decision making felt natural , similiar make decision own. -- -how rational emotional decision making process -- - somebody technical knowledge could provide interpretation best solution. problem required research would take place would rely rational logical thinking arrive solution. regarding emotions , used open communication channels facilitators supervisors discuss emotions opposed letting emotions making project decisions. -- -overall -- - previous role electrical designer smitten data analytics within electricity metering sector. edsa provided platform pursue passion broader scope data using market leading tools importantly upgrade skill-set thus opening new opportunities within workplace .  overall experience academy far great somewhat challenging. firstly team leader elected team ; tasks disturbed among members. disagreement team meetings. achieved aim , members satisfied. team achieved aim could done time. allocation work workloads decided team leader discussing team. arguments workload. team leader assumed everyone level. questions answered answered satisfy members. individual members needed extra research questions answered properly. team able work together , reliable tried level best punctual meetings. allocated different tasks person researched thoroughly co-ordinated findings together team asked support needed. personally helped individual confident , however bit strain keep mates vast experience programming statistics sometimes felt incompetent tasks given , helped gain knowledge pushed learn harder. role played team members team player , several suggestions made everyone team adopted. team selective adopting new ideas. team worked well , agreements reached almost issues little new information gained members. believe team work ideal method quick information gathering , learning oneself time consuming tiresome. believe contributed effectively team reliable although bit hesitant first stages adjust different characters communicate freely afraid judged believe group work essential would mind working team aim team achieved .  start academy quite tricking due meeting new people , unknown environment away home family proved tricky. first fundemnental sprint extrememely challenging memebers group failed help work assignment hence resulting lot friction stress. poor coomunication bad attendance critical times. resulting wasted time send message finding nessecary details perosn thus wasting time. problems poor team dymanics dealt memebers realise importanace good team dynamic order complete set task. next sprint alot better group organised everyone commited assigned task intially lacking prior sprint. group communicated effectiely term problems incountered. first sprint reveal emotional weakness team memebers unable keep instead moving foward broke away decided give instead still trying help team current assign task. second sprint clear cut division two teams helped left driven knew expected .  journey exciting. stress constructive sense forced work. everyday excited come academy interact vast personalities class offer. given nature work , days really tough days really good ( especially code running ) . academy taught , testing limits understanding. whole life 've always team player. although n't like charge , somehow found leadership positions based assertiveness became norm. shocking finding opposite position first sprint. zero communication whatsoever , made feel uncomfortable could n't share thoughts feelings , however found way work well collective. took forever wrap head around content work reason made feel futile. due lack communication , people felt right thing , would later put everything together. opinion , think structure , leader would keep us track. way decisions made , shared responsibilities based people 's strengths weaknesses. equally made decisions led productivity , sometimes would disagreements. times , would emotionally draining , every low , growth. throughout roller coaster journey , 's nothing growth , understanding learning. understanding workplace , understanding colleagues understanding .  intimidating meeting many new people , especially wildly different personalities backgrounds. sharing work space sometimes challenging everyone shares views communal space treated terms cleanliness keeping noise levels down. interacted lot people , small number individuals like background temperament. team dynamics far challenging part course. particularly struggle team-members lack self-motivation work ethic. found often instead insisting team member pulling weight would rather task myself. tend feel someone lacks motivation something unlikely perform task best abilities. also seem little confidence others completing task way would like done. generally , decision making teams easy mostly due apathy uncertainty surrounding decision behalf team members. found frustrating since often lead one-sided discussion resulted unspoken dissatisfaction resultant product. decisions also often ignored later , dont mind changing decisions believe communicated completing project still paramount. feel often good sense required complete project communicate steps team members , however rarely self-assurance properly delegate tasks hold people accountable meet deadlines complete task .  attending academy first time like first year , however difference everyone age. idea meeting people really cool however issue forced work people things tend bit shaky everyone standards. make matters interesting grades directly dependent interaction team. varsity must admit best experiences working group tend find similar roles characters. however bit optimistic given diverse ages academy assumption folk work experience hopefully mature. furthermore , come realize establishing solid authentic team dynamics 's also lot stressful actually may seem without trying fake sake team trying accommodating. real issue trust decisions tends made around dominant personalities perspective may particularly best interest team whole. although always agree approach would agree fundamentally understanding skill set equally distributed also end day delivery project. overall happy rational decision making process. divide trust conquer works best teams something come learn. within teams lucky may find someone work ethic dedicated case would likely help could find someone understands work complements style. think initially everyone problem admitting n't get something deliverables due. journey emotional especially towards end tend give less less input become concerned delivery .  explore data science academy , road never thought would take lifetime. love passion cars , accepted data science internship one last thing never saw coming. love travelling exploring new avenues `` let 's '' tip. first day coming explore data science academy ( edsa ) beyond expectations , visually appealing facilities provided us. first could believe , cape town , comfort zone , pushing something involve anything studied. came expectations , however , told something need give order leave experience back future. speech supervisors gave us 22nd january reassured make mistake coming here. first week given work , install softwares would using throughout year , thought would pre-installed arrived. first , introvert side steering wheel interact lot people besides housemate. started interacting people water crisis sprint started. team people different background ; bioinformatics sciences , computer science , economic statistics mix , different parts south africa ( mostly gauteng ) . one things stood type people would start conversations loud side come order make everybody feel comfortable. first week gather sprint took team start making progress , thursday already figured system worked everyone order use something show supervisor. put weaknesses strengths table , started communicating better team. results shown supervisor told us think would deliver delivered , however , delivered end fundamental , faith would best one best. love good music , adventure socializing led connecting people group. time fundamentals spent helping people , building friendships would carry edsa year even greater , learning languages like python sql , channeling mind data scientist mode. weekends colleagues would tour cape town going beach , hike mountains could see edsa lab , visiting malls theme parks around cape town. python , one complex languages learn. well , thought since programming background. two teammates experience working languages , changing one language another , fairly doable. advantage since two people could consult whenever ran problem python. presentation fundamental one laid back fundamentals face since preplanned wanted add slides clear storyline. fact split fundamentals tore us apart , however , vowed exit whatsapp group created. review week , weekend , people never talked friends edsa lab suggested go theme park near canal walk mall take minds water crisis sprint. character part outting shy , yet interesting backgrounds. still friends them. stock market sprint came flew by. really feeling team since lot judgemental people felt like knew better. lacked communication , managed pull find sprint one piece. events bcx hosted highlights first months edsa , amount inpirational people engaged events. felt like meant every time spoke lucky waiting us greater wildest dreams. week takes cup. never expected go home least july. honestly grateful spending year edsa lab , could chose anything else .  experience academy exceptional. meeting new people amazing , specifically working them. much challenging first inception relate , time things gets better begin know get use one another. odds events one two group member want participate ( eg. missing meetings ) thing way oppose agreed group , really problem participating also productive , especially meet group expectations group works , members end work. means group take collaborative decision planning expect everyone deliver agreed planned. lab one best way work , especially comes group work , find collaborative useful. 's unfortunate group members choose work home leave early come late. found behavior hinder progress , especially tasks divided sub groups among group members. critical aspect group always communication. members usually communicate inability complete task late work due. members get emotional approach , end day always encourage one another see right side approach task project hand. working many different people taught rely others , means trust member work , even better me. fun leaning since day one .  life academy. wonderful experience , joyous full many possibilities. 22nd january 2018 took tax kraainfontein heading woodstock , place never , told far capetown cbd. exact location going woodstock exchange building 's explore data science academy located. help asking people taxi help gps able find building got around 10:30 , yes late got presentation room aidan ( supervisor ) giving overview academy expected us ( student ) . room full 100 students different provinces , races backgrounds. listened long bearded white guy speaking eloquent profound right. one thing struck intention emphasis speech intelligent ( students ) apparently 8000 people applied program , 3000 people able finish whole selection process 250 people interviewed top 100 made program. aidan 's words full passion compassionate , inspiring encourages , gave birth new hope life knew labelled people 'failure ' knew opportunity beginning great journey ahead. promised things different time , work hard play less , would make friends pray give thanks heavenly father. week later ( students ) split group five people work capetown water crisis using skill learnt past week i.e sql , python power bi. group name bean bags , group four males one female , embark project , use divide conquer approach help assign work member , group members wonderful people work , jelled much fun together . bean bags often worked together , using one computer helped tasks assigned member. worked together two months present findings regarding capetown water crisis solutions suggestions sprint came end. split south african stock market project , working different people adjust new team went name stockoverflow , bit nervous previous team already made friends working together forged friendship amongst adjust. stockoverflow team three males two females , sprint month much done. learning different models predict stock prices fascinating challenging me. used divide conquer approach assigning tasks member consolidating member done google drive , 26th april presented model predictions stock price one week break came back another sprint. realized short space time learned many many skills apply solve real world problems. wonderful experience grateful .  time academy positive. 've unpleasant interactions person far n't believe 've cultivated many deep relationships aside perhaps one. two groups 've assigned socially pleasant. first group n't ideally organised second group better regard. n't particularly assertive presence first group unlike second positive dynamic present both. prefer make decisions group try best pull weight burden .  social perspective , experiences academy refreshing impact personal development. prefer things myself. reflected interest gaming , reading fan-fiction writing. however , willing interact others necessary. enjoy paintball friends far sitting lounge rather making casual small talk. applies workplace well essentially things together. even talking , inevitably get topics related coding such. generally people ( outside academy known years ) seem identify introvert give misconception poor communication skills n't always case. many issues communicating team mates others class regarding various topics ranging work-related hobbies ( especially sport , gaming anime ) . prefer finish roleplaying video game lounge talk also like talking science nerdy things. words , need socialise , problems so. intuition observation valuable traits workplace data scientist team-based project. working teams allowed utilise both. found often analysis others n't done apply intuition. managed thing. different academics would essentially everything myself. regard , team-based nature course done well. often think good one working others stimulate us potentially become good trait previously thought n't naturally suited to. case , working teams sort force us find ways overcome weakness make strength instead. said , team dynamics sometimes involved lack coordinated planning among team members. good keep things open , applied plan rather plan. decide certain approach solving problem involves coding , dashboard , n't mean ca n't improvise. agree improvisation ultimate way things. make plan rely open-minded thinking plan n't appear working , improvise. generally experience academy , emotion seems rule roost logic seen potential barrier team work cooperation. mean students end becoming dependent affect teamwork negatively long term. overall course excelled bringing personality traits relevance workplace , providing opportunities overcome weaknesses utilise strengths instead .  ‘ greatest opportunity part 100 chosen individuals wonderful programme. explore data science academy ( edsa ) environment makes feel welcome perfect terms collaboration networking different individuals interesting views personalities. ’ hard distinguish edsa work environment. type environment prepare real job. edsa provides one opportunity learn work diverse group people terms career background , culture religion. working groups different individuals project exciting. throughout time , ’ learned lot working teams. first ’ learned team work achieve terms results learning new things. secondly , come decision making , everyone opinion coming final decision another thing. learned ’ voicing opinion , convincing fellow team members indeed opinion fruitful. comes team strategies embark particular project , ’ one size fits situations. team come strategy going solve assigned problem. difficult , especially serve team coordinator serve , time complete different team whole set different personalities characters. instance learned strategy based skills , capabilities mostly team ’ attitude towards project hand. knowing strengths weaknesses team members good start. far learned lot. ’ still intrigued eager learn lot more. feel like becoming real data scientist day passes thanks edsa giving wonderful opportunity .  choosing spend year explore data science academy could easily best choice ever made. choice renewed feeling purpose educational/occupational direction. learnt past 3 months thought would entire year grateful presented opportunity. fellow students edsa staff added excellent experience encountered thus far. interacting brilliant minds results one , inevitably , attempting better experienced first hand. excited path edsa setting look forward future holds prospective data scientist : )  hardest moments life learning new things , matter simple easy might be. difficulty met new people academy. truth talkative nature , ’ always found difficult start conversations students anyone academy. however , amazing experience discover learn different students ’ cultural backgrounds diversities , interest data science life whole. academy , realized people learn discover interaction others. new environments nervous , yet exciting ones. explore data science academy best example scenarios. academy provided great exposure working environment challenges ought face work teams. able work team prerequisite skill needed data scientist. learning curve allowed self disciplined , self-motivated introspection little effort group work affect entire team ’ progress thus , results poor performance entire team work. people different cultural , educational programming backgrounds , learning difference right , right right became key determining factor understanding work teams. months academy , feel close becoming data scientist before. hunger efforts put understand data science seem rewarding now. great feeling among first group given opportunity learn data science. although pressure much times , hope believe everything designed meant prepare us real world working environments .  mixed emotions went came cape town.i felt ecstatic opportunity afforded yet side intimidated new environment. walked building first day heart filled much anticipation could wait shake everybody 's hand introduce myself. meeting talented young old bright sparks . never worked team came academy , know working team often many times worked alone everything myself. changed , teamed bright minds actually got work . really surprised much effective efficient working team. really enjoyed working team. believe travelling meeting new people potential widen ones horizons. learnt lot still learning lot different people academy , academic related lesson also life lesson carry ever go. wonderful fruitful time previous team , though ups downs .i learnt n't leave issues group grow rather need tackle emerge .  experience edsa great far. massive learning curve academically also personally. took myers-briggs test twice : , last night earlier today got slightly different results. first time came letter combination infp second time isfj. though slightly different , feel accurate overall representation various attributes exhibit. seen diplomat-mediator well ( second time taking test ) defender sentinel. naturally reserved nature , unfriendly , prefer working alone comfortable enough sit alone group people. however , approached , come across friendly. able read various personality types groups fairly quickly , allows quickly adapt different situations needed group times. n't usually take charge , know end goal something achieve , prefer clear cut plan achieve , one challenging parts working group , made need step comfort zone recently make decisions voice myself. times experienced stress due fact felt frustrated able comprehend certain concepts quick enough significant work project , deadlines much issue. overall , feel thoroughly enjoyed learning group dynamics far edsa .  bit loner times id say experience really taught engage people talk especially student kind goal. based academic level think got experience programming fact familiar sql python shows grown academic level. knowing wake everyday see new faces lab thought able manage knowing loner told working teams thought end much talker people thought going express opinions , participate happened best experience even started releasing need talk order opinion heard. last team really understood much teams challenges valued opinions decision making part even pimplier , first team agreed everybody gon na take part everything look strength weaknesses. assign one another activities concerning project still forgetting help strengthen academy weaknesses basically created fridays helping other. comes decision coming acadamy one hard decisions take cape town far home choose completing honors degree academy. ended choosing academy honors degree felt like needed experience data science marketing world academy would best place start fact ought learn everything beginning great computing skills wanted stipend also contributed decision coming here. would conclude regret coming instead happy here.its 5 months already feel like year going bit faster want even programming skills think would got much experience programming languages skills honors.the experience got far best. helped growing academically also helped gain confidence individual .  like meeting new people similar interests , many new people , connecting people time goes assigned new group. n't spoken students merely strong introverts , recent group nominated group coordinator take lead every meeting , every interaction acknowledge personal level group members feeling day , something never really thought experienced , even grew every mondays share weekend , really appreciated put position challenge grow personal academically. like make decisions based merits decision efficient come attention efficient time take account people 's feelings , made decisions past group people feel included left , worth noting individuals contribute project way , barely showed academy , never written one single line code left member take care making dashboard , presentation ultimately present , two us took uncomfortable route diplomacy , pleaded every morning attend briefings supervisor us , went group training sessions made impact , next group , understood academy meant worked together well. stallions , magnificent , sudden paradigm changed , enjoyed one idea best things , originally typed minutes meetings another member suggested trello effective , learnt something new , excited , day , well every day went bed knowing something ’ know woke ’ data scientist ask for. one important lesson got working two different groups leave time guide team work flow everyone miserable end project , however everything planned trello ( ’ trello ) ‘ tees ’ crossed ‘ eyes ’ dotted , pleasant experience bring best everyone involved , team creativity grow levels one planned beginning. , irrespective put group work , sit maybe second session “ getting know ” map strategy roles assigned , significant insignificant .  edsa reflections 24 november 2017 , filled joy , excitement anxious time opened email edsa titled “ congratulations ! ’ accepted explore data science academy ! ” fast forward 22 january 2018 , ready meet new people ( possibly form new friendships ) , meet edsa team learn lies ahead 2018 , overwelming day was. today 6 may 2018 , lot happened past 3 months , ’ learned much ! sql , stats , python , power bi , presentation skills… [ still learning , never stops : ) ] . grateful selected one top 100 , ’ appreciative opportunity , thank edsa making possible me. lab data scientists making , greatly enjoy lab , environment condusive learning , group data science enthusiasts one room feel receipe success. training limited online courses edsa staff students learn lot , different backrounds , skills experience making lab interesting fun place grow academically socially. teamwork water crisis project : team liquidata must say initially concerned amount individual work deliver team well. realized importance effective team , working well team reduced work load stress also helpful individual tasks. roses regards team dynamics , working different individuals perceptions opinions challenging first , frustrating decide certain matters , helped lot grounded problem statement kept us focused needed done. worked well ’ result driven given task , completing task successfully required relevant info important , refering back problem statement got stuck great tool us. emotional moments team due personality clashes end worked well contributed success team , awesome team ! rsa stock market project : wolves woodstock accepted ’ work new people , really problem , hope wish strong programmer team , wish realised statistician programmer team , lovely ! great team , felt like outspoken person ( ) bit much , still worked well though delivered. prefer working team different ideas , creativity different skills ’ huge advantage eveyone team result driven , absolutelty appreciate that. ’ good experience teams far .  year huge one change , involving many aspects : leaving stable job , moving cities , moving new field , going back student , long distance taking chance brand new data science academy. quite excited meet new data science students despite usual shyness. people taking similar plunges shared interest field. enjoyed ( slowly ) meeting new people learning backgrounds , interesting varied stories are. working lab new experience. complete self learning tough journey , glad academy experience supervisors help guide us way becoming data scientists. bit unusual area ’ computer lab learning , also sort office , also people socialise. ’ sure nailed exactly be. feel work better home quiet comfortable ( apparently line personality profile ) . group work element year probably one biggest pain points. see intent setting us way , ’ think quite worked yet . group work challenging. throw meeting new people different backgrounds , huge amount new content learned fairly significant stakes ( internship opportunity ) difficult situation. think resulted frustrations running high poor quality work delivered , individual group would wanted. think worrying getting project done sometimes gotten way actually trying learn content. terms decision making groups far , think one positive definitely logic valued field students. state reasoning behind plan solidly ’ problem. think challenge group member interpret instructions priorities differently , sorting get get everyone page take valuable time. personal level noticed always looking clear , decisive decisions plans , deal well “ seeing goes ” . think something need look team setting , expect manager well. overall , still pretty excited year regrets decision join. think academy still bit rough around edges needs refinement , part iterative process right ?"


In [52]:
TtXt = "  first 4 months edsa filled many new experiences ; challenges , others joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people everyone met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates within edsa , provided insight gained experience tools like python & sql. also cherish able build network data scientists tomorrow well established industry professionals.. -- -working together lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious con would noise distraction - least say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience would say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes full responsibility part ) members first group noobs sql & python , gather analyse phases really exposed technical deficiencies resulted lot frustration came time delivering tasks. experience took comfort zone important lesson patience us all. second group high level experience , strong team co-ordinator team members took full responsibility tasks. even though work complex comfortable team dynamic. -- -how decisions teams made consistent make decisions -- - team decisions groups made deliberation. decisions easier make person presented clear points opinion was. group dynamic strong decision making felt natural , similiar make decision own. -- -how rational emotional decision making process -- - somebody technical knowledge could provide interpretation best solution. problem required research would take place would rely rational logical thinking arrive solution. regarding emotions , used open communication channels facilitators supervisors discuss emotions opposed letting emotions making project decisions. -- -overall -- - previous role electrical designer smitten data analytics within electricity metering sector. edsa provided platform pursue passion broader scope data using market leading tools importantly upgrade skill-set thus opening new opportunities within workplace .  looking back one got academy right , confidently say grown intrapersonally interpersonally optimistic perception working also gradually shifted realistic end spectrum. came nothing positive outlook journey gotten opportunity embark , however , learned couple valuable lessons - one stand value understanding teammates expecting one understand you. put far put cap assertive nature gotten lot f pressure given opportunity even sharpen softer skills tends help quite lot even outside academy good vibe generator - one deflects negative energy positive energies whatever spheres happen find in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. learned focus making sure ’ expected delivered timely miss beauty process focussing relatively small hurdles ’ vast amount captivating material engage in. getting know colleagues exciting initially thought would best get know everyone due knowing underlying power networking much tend ripe rewards least expect it. however , due various factors including time constraints , one n't yet gotten know everyone. nonetheless , 've connected way pivoted range experiences 've exposed anything - 've found one connects colleagues duties get better understanding really drives bed morning tends relative ease synergies working people group. foundation good teamwork come play proven crucial implementing fulfilling common goal quicker. establishing people 's unique abilities also n't easiest tasks , one things far made role duty allocations bit messy groups. mind team makes decisions usually made voting process duties tend easily misallocated accord effectiveness leverage ( exploiting team member unique abilities motivate everyone play strengths avoid weaknesses proper balance ) . teamwork easy job exciting time takes one individual break firm structure especially individual wants make decisions- hinders progress flow ideas team turn leads dull average outcome. 've found decision-making techniques bit irrational due central tend especially find group nice people express feel whoever so. 've also learned never bring emotions workplace people tend step sensitive things without noticing- example someone staring shoulder commenting work 're quite unpleasant kills moral detaching focusing 's required best coping mechanisms .  attending explore data science academy already proven , continues amazing experience. ’ quite astonishing realize much ’ learned mastered past couple months , chief among would ( subject essay ) , teamwork. meeting new people daunting task ( including ) . however academy different start. sheer excitement students especially explore team ( supervisors facilitators ) acted social catalyst , smoothing social awkwardness , knew getting along like house fire. working teams people completely different backgrounds personalities challenging throughout first project , said , also interesting see individual adapts accommodate team help build team dynamic. starting cape town water crisis project honestly idea expect , certainly ’ expect team dynamic play huge role success deliverables. team overcome different levels knowledge , skills , self motivation. couple late nights flared tempers managed get work done , realised overlooked crucial part teamwork. one need self motivation , also create environment team motivate comfortable enough ask help ’ feeling overwhelmed struggling get grips difficult concept. team definitely lacking aspect , take chin regards deliverables , walked away learning priceless lesson moving next sprint. assigned new teams rsa stock market sprint , refreshing hear ’ team struggled certain aspects team dynamics. get go approached structured interactive way things. decided stand meetings every morning report previous day ’ goals well setting new goals day ahead. already made huge difference motivation team. another part success dividing work ’ get overwhelmed individuals rather sharing workload one-on-ones relevant team members gain deeper understanding section work. approach even advantages thought. structure communication team members lead lower stress levels ( even though still ever present ) , resulting cooler tempers clear headed decisions less emotionally-driven first project. goes show “ soft ” skills important technical skills working teams. ’ wait learn brilliant diverse people academy. '' individual commitment group effort -- makes team work , company work , society work , civilization work. '' -- vince lombardi  wonderful place , place waking morning easy task great desire learning something new everyday. meeting new people great thing learn work team also socialize since person well crowd. actually first time working lab like reason experience labs quite instrumentals labs testing experiments computers , n't computer labs first time coding lab reason coming edsa first choice . skills data scientist one drove apply wan na learn coding. learned lot lab also working team. one main problem faced working team group rather , groups perfect might wish change disaster.sometimes set time slots group meet discuss progress deliverable surprised one member pitch thought lot work , disappointing would mostly prefer working alone future. working team sometimes emotional draining decision divide conquer could work together well. hoping great team smart people need told since every sprint `` lists '' .  essay discussing personal experience regards academy like meeting new people especially introverted person am. speak different teams , different different felt , learned different characters. comparing decision making done teams versus personal decision making. getting plane cape town without place stay friends city big risk , certain plan academy kept praying want scam plot get specific building get kidnapped. fairly surprised get meet people walks life , also kind questioning legit , mean hard question , sounded way good true good things learned ’ happen people like us , guess time shine receive blessings. ’ think academy , blessing. said , experience academy ups downs like always ends up.i learned lot , fallen love data science , things small mindedness used consider geek ’ s. learned push , always try learn , practice computers always evolving. since met people different personalities academy , personality clashes occurred 1 biggest lessons , able work different people different personalities. working teams ’ easy thought would first , everyone idea assumed supposed done different opinions could dismissed everyone 's opinion valuable working team , opinions somehow seemed clash. working teams taught patient , quality possess whatsoever. second time changed teams though seemed though everyone gotten idea working teams ’ like , basically dipping toes working teams thing first sprint , second sprint bit confident meant worked better second teams , improving go. teams , split work equally amongst , making sure everyone gets chance tackle important lessons like coding dashboard. group coordinator state views , he/she would speak supervisor decision would made way , supervisor would guide us right direction hearing thought right way.its almost concept use guess making decisions , weigh options , see one best one worst outcome obviously go best outcome. conclusion working teams skill gaining developing everyday , ’ mastered , heck difficult master many personalities , getting .  experience academy far stressful yet fun. 'm one enjoys working groups , 'd working times. starting first sprint reminded prefer working alone. working group certain people carry weight , even frustrating. however , blame speak let concerns heard. end us saw way tackle task hand work towards finishing assignments given. last group complete opposite. three weeks complete sprint therefore knew work collective. tasks split one accountable part project. ended working well together got along outside work well. work previous group ? know , look forward working different people different dynamics .  experience academy enjoyable far. learning lot interesting valuable skills meeting many intelligent interesting people. meeting working new people lot fun rewarding experience. working lab challenging. open plan allows many diverse unusual conversions easily started joined also prevent work done people interrupt prevent others entering staying state flow necessary learning. especially destructive slower learners like myself. working teams also challenging insightful. sometimes tricky setting understanding team dynamics new teams especially people ’ know well , practice working teams always appreciated team dynamics always easy decipher understand. decisions made group ( least feeling ) without one person charge giving orders. could also part still learning try make effort person group feels opinion asked heard decisions made group decisions everyone opportunity raise differing opinions votes support. consistent way enjoy making decisions think allows everyone feel involved dedicated outcome everyone feeling others group think feel decisions made. say decision making rational emotional know accurately determine that. maybe mix , though slightly leaning towards rational .  intimidating meeting many new people , especially wildly different personalities backgrounds. sharing work space sometimes challenging everyone shares views communal space treated terms cleanliness keeping noise levels down. interacted lot people , small number individuals like background temperament. team dynamics far challenging part course. particularly struggle team-members lack self-motivation work ethic. found often instead insisting team member pulling weight would rather task myself. tend feel someone lacks motivation something unlikely perform task best abilities. also seem little confidence others completing task way would like done. generally , decision making teams easy mostly due apathy uncertainty surrounding decision behalf team members. found frustrating since often lead one-sided discussion resulted unspoken dissatisfaction resultant product. decisions also often ignored later , dont mind changing decisions believe communicated completing project still paramount. feel often good sense required complete project communicate steps team members , however rarely self-assurance properly delegate tasks hold people accountable meet deadlines complete task .  past months one challenging thrilling experience life. coming , expected engage people knew introvert , experience academy would force engaging people. something totally outside comfort zone. ready ; much prepared , ready challenged understood growth reached uncomfortable spaces. experience taught could imagined. learnt fearless embrace greatness within me. come across shy , willingness push things enabled speak room full people made realize get nervous things actual fact get nervous things capable also fail. way scared inabilities potential. understand still lot learn fully comfortable know feel still lot knowledge gained. coming bit theoretical statistical background , interesting see techniques learnt university applied real world scenarios. coming space , space full well equipped individuals , felt bit intimidated idea career going lacked confidence really good at. experience far definitely given better understanding skill set career take , importantly come learn alone. able meet lots people also similar situation , simply shown worries worth worrying things fall place. environment quite relaxed , yet taught behave professional workplace. support peers also tremendous. observing everyday events supervisors taught teamwork , people come together get things done . although sometimes remind use inside voice , feel adapting environment relatively well. going forward , expect learn develop skills .  data science academy amazing path far , one room people ambitious love learning got excited work also supervisors facilitators determined love job motivates even keeps thrilled like one day. thankful part academy , much learn everyday proven need specific field certain level education data scientist , way academy structured students fields various level education think great idea , would see someone never studied computer science statistics knowing concept better degrees , brief say need order survive academy focused know expected everyday. fortunate enough placed teams people determined never struggled get group mate deliver expected , hence said people academy focused know for. think selection team good job placing us academy. appreciate also fact much supervisors facilitators busy , always accessible help us work passion , see getting better better day become data scientist certain end year level corporate expects be. academy coaching sessions assists come across obstacles program , work related personal problems , someone talk share predicaments going , find helpful workload much might loose track things , especially something distracting going personal life , deviate attention work , think service useful personally used load shoulders shared problems someone trust judge share problems anyone. randomly placed teams believe help us come across different personalities people learn work around , although people hoped , believe prepares us fit enough deal people meet corporate world. also learn assist patient people might struggle certain concept also learn others .  attending academy first time like first year , however difference everyone age. idea meeting people really cool however issue forced work people things tend bit shaky everyone standards. make matters interesting grades directly dependent interaction team. varsity must admit best experiences working group tend find similar roles characters. however bit optimistic given diverse ages academy assumption folk work experience hopefully mature. furthermore , come realize establishing solid authentic team dynamics 's also lot stressful actually may seem without trying fake sake team trying accommodating. real issue trust decisions tends made around dominant personalities perspective may particularly best interest team whole. although always agree approach would agree fundamentally understanding skill set equally distributed also end day delivery project. overall happy rational decision making process. divide trust conquer works best teams something come learn. within teams lucky may find someone work ethic dedicated case would likely help could find someone understands work complements style. think initially everyone problem admitting n't get something deliverables due. journey emotional especially towards end tend give less less input become concerned delivery .  good day , really n't know start essay besides good day. anyway 's way lets start business. moments academy mostly up. let start day one meeting students south africa great , meeting people different personalities coming different background. people different races , culture tribes ( enjoy stuck cape town since day born. ) . getting first team , part crazy , fun team. guys n't look one another open , strength. first weeks everyone working well getting close end became frustrated team members started lose focus ( assume got tired sprint long ) . following team great came work bonding n't all. two members liked working home ( productive ) two quit time. came decision making , meter bouncing ideas around choosing one move stayed one teams. second team split two teams two teams thing 's believe new way things great. submitting individual team one learnt skill taught sprint. thanks .  good fun experience though 's often case pressure. lab allowed enough time practice get used coding , improved efficiency coding working computer. coming academy n't used idea working team , along year got better , knowing someone count , rely go need help. also got know personal level makes easy understand , strength weaknesses allows us form strong team. sometimes disagreements think right , effective solving , lucky enough co-operative team members since beginning year. though n't got know many colleagues ones got know would say pleasure meeting  academy amazing , love petry dish talent ! handful genius characters here.. backgrounds unique stories , explore data science academy best place develop data analysis skills. love time ! far , 've experienced whole lot pressure change something better. spend time studying , better feel , comfortable become closer get quality employee entrepreneur. everyday come office , feel home , learn something new.. something directly use business application. look around entire building get inspired start business 've learned here. speak among business academy helping us realize ideas clearly. without information back ideas , 's difficult starting. availability information outstanding. really appreciate nest. last two teams great work with. achieved bunch , , thanks resources available. first sprint tough though. team , realized strengths taught other. n't problem sql python used technologies before. understanding statistics applications data given difficult. worst , water crisis data n't collected well. n't worst experience data taught things career perfect. perfect skills data cleaning. realize limited number solutions data analysis. perfect outcomes data science , could breeze work future. believe hyper prepared things.. game theory data science would prove beneficial. emotionally , 'm great. academy brings best worst skills provides controlled environment mend holes analytical judgement. relief , emotionally physically becomes process bettering daily , bit bit without forcing stress .  imagine excited finally meet people like minded. maybe could rephrased meeting people like me. however , instead , meet small sample different types people different types personalities. yet find stuck world different types people different , different interest ambitions , translates make decisions every day. decisions affect , especially working teams. hand , decisions make affect team processes. according personality test , personality defined five letters entp-t , according classification debater. true debater , convinced , play along suck test discuss personality affected general experience academy. lab allowed us access well collaborate , although working effectively lab challenge , many distractions. found easier get know people type strengths , knowledge background passions general. type knowledge allows efficient knowing ask help different things. prefer asking facilitator. always done past prefer working challenge peer , along process learn would otherwise. comes working teams , always , first one speak give opinion came variety ideas , ’ extroverted side me. therefore , end leading , team. role prefer police people , therefore either allowed people lead would make sure project requirements completed end. last sprint dedicate responsibilities according people ’ backgrounds strengths interests. one person leading deliverable project end would ensure requirements submission met. employed similar strategies past , working tangibly people bring table. framed decision making teams worked in. someone shows work work have. approach made easier focus objective deliver projects. decision making rational time little emotive times , irrational judgements employed make assumption people attitudes toward project work. creates barrier people pulling weight past immediately , integrate teams later. however team dynamics forced finisher .  past 5 months explore academy challenging. make great adjustments order here. positive negative experiences. one positive experiences much learned. met lot people highly intelligent interests. worked 2 different teams. prefer work alone working team makes sense due sheer volume work. first team experience incredibly frustrating. team members highly introverted technically strong made discussions assigning tasks difficult. even tasks assigned accountability meet deadlines. prefer chip away steadily work opposed rushing close deadline finish. working second team much better. met regularly , communicated lot. tasks assigned everyone accountable. team members helpful towards .  experiences academy far life-changing say least. meeting smart , passionate , hardworking ambitious people different backgrounds happen many similar interests mine interesting journey. introvert prefers working alone pace within space , quickly find way adapt polish social skills order socialize 99 strangers going learning data science with. understood word go need socialize form meaningful relationships fellow interns could see forming strong network may helpful many years come career , long internship. academy ’ emphasis importance team work concept took warm to. teams exposing different individuals varying academic , social , work geographical background different personalities : introverts , extroverts , intuitive , artistic even really funny characters. teammates phenomenal experienced fields previous exposure to. learned lot them. equally ’ also encountered independent disagreeeable teammates required adjust levels patience , tolerance understanding may necessarily share similar beliefs me. need patient tolerant people ’ points view approaches solving completing team tasks. teams , due differing views , opinions approaches use democracy ( voting ) order reach decisions within groups. although easy one resort taking emotionally-driven decisions due one ’ firm beliefs ways things high level emotional intelligence critical favourable team dynamics. academy based cape town , city ’ visited holidays before. moving cape town challenges required solid ‘ family ’ away family fortunately found fellow interns resourceful many ways even outside ‘ labs ’ . example group share information accomodation , transport around city , social events extramural activities. chance test team-building skills virtues regular group sessions nuri believe helpful giving idea relate people less formal settings. credit due supervisors , facilitators oprerations officer ; helpful making internship experience manageable definitely one memorable years come. commitment programme makes easier people personalities settle nicely within diverse small community .  experience academy far interesting 4 months although apprehensive meeting work new people , still enjoyed work looking forward comes next. group work team dynamics particular challenge although ’ really socialised colleagues since arrived cape town , friendships formed mostly students live house me. team dynamics challenge fundamentals sprint ’ fair division responsibility well accountability team. group members would slack part hard hold anyone accountable concerned nice. decision-making group fundamentals sprint mixture rational emotional , things right things badly one wanted take responsibility , much passing buck. team also beset laziness unwillingness really get learn needed deliver project. rsa stock market sprint , group somewhat better people hard-workers divided responsibilities early based perceived strengths individual. really serious problem group communication different areas functional responsibility , clear group members prepared presentation fully understand analysis analysis team performed. decision-making group rational. working labs , nerve-wracking experience significant proportion people simply go socialise distracting us , prefer working environment absolute silence ( hence prefer working midnight ) simply think problem allow creative possible solving problem. people assist thought-processes terms thinking problem solutions irritating distraction. around many people long eventually lose focus problem focus people , would much better silence mandated enforced labs. also need individual work instead group-work. suggests ways could contribute group signs learnt lessons emerged previous group , still lot trepidation comes next. hope meet group members committed project , otherwise , everything end tears .  honesty enjoying time regret choices made. really educational love structure , environment one force work push much facilitator really helpful bounce ideas see far able explore imagination. meeting new people 's always nice stories people far go life , combine different cultural group things got learn people 's culture stories tell. work together 's really tricky fun times tricky , matter many times think always tricky talk wont enough words left. making decisions 's simple everyone agreeing goals deadline arrived people stories , 'm unable relate excuses , honestly wish everyone could adhere words , really would like decisions made like explore lot 'm working something due time constrain sometimes 's best expected enough time explore more. team members sensitive way sensitive leave part there. love academy wish time learn bit , enough resources even conduct maybe research , instance know write good machine learning algorithms n't understand work might even able improve .  time academy positive. 've unpleasant interactions person far n't believe 've cultivated many deep relationships aside perhaps one. two groups 've assigned socially pleasant. first group n't ideally organised second group better regard. n't particularly assertive presence first group unlike second positive dynamic present both. prefer make decisions group try best pull weight burden .  starting academy wonderful experience. cape town beautiful city location academy could better. meeting different characters really experience.in first weeks meeting new people daily basis sometimes intimidating. teamwork never really strongest skill always something would like improve upon , heard academy strong teamwork emphasis happy.the academy working lab strong social element , even though work learning somewhat technical. first sprint , quickly discovered diverse group was. vastly differing amounts experience fresh high-school graduate someone working decade. also differing skill sets. working groups challenging rewarding time. cape town sprint primary downfall lack planning what. resulted us almost late deadlines. communication within group could lot better. tend keep flexible approach completing tasks , flexibility ended costing establish proper deadlines .  social perspective , experiences academy refreshing impact personal development. prefer things myself. reflected interest gaming , reading fan-fiction writing. however , willing interact others necessary. enjoy paintball friends far sitting lounge rather making casual small talk. applies workplace well essentially things together. even talking , inevitably get topics related coding such. generally people ( outside academy known years ) seem identify introvert give misconception poor communication skills n't always case. many issues communicating team mates others class regarding various topics ranging work-related hobbies ( especially sport , gaming anime ) . prefer finish roleplaying video game lounge talk also like talking science nerdy things. words , need socialise , problems so. intuition observation valuable traits workplace data scientist team-based project. working teams allowed utilise both. found often analysis others n't done apply intuition. managed thing. different academics would essentially everything myself. regard , team-based nature course done well. often think good one working others stimulate us potentially become good trait previously thought n't naturally suited to. case , working teams sort force us find ways overcome weakness make strength instead. said , team dynamics sometimes involved lack coordinated planning among team members. good keep things open , applied plan rather plan. decide certain approach solving problem involves coding , dashboard , n't mean ca n't improvise. agree improvisation ultimate way things. make plan rely open-minded thinking plan n't appear working , improvise. generally experience academy , emotion seems rule roost logic seen potential barrier team work cooperation. mean students end becoming dependent affect teamwork negatively long term. overall course excelled bringing personality traits relevance workplace , providing opportunities overcome weaknesses utilise strengths instead .  academy challenging experience. expected prior coding knowledge help minimal help , greatest challenge statistics. slowing getting point n't actually try understand concept thoroughly rather get basic theory focus application. overall , insightful journey , learning interact different personalities group dynamics. really excited leaning nlp regression incorporate app development monetize it. genuinely regret applying academy although days doubt hurdles remembering greatness opportunity always great motivator .  four months since decided embark journey. packed bag , moved new city took first step living best moments life. love meeting new souls. always fascinating finding someone else ’ journey began , similar different footsteps left are. amongst 99 brilliant minds incredible , getting know refreshing. people stand , everyone unique. always good working , taking reins. ability work effectively team acquired skill , understand different personalities , people deal problems communicate. delegating seemed help terms getting work done teams , person taking piece , sure finish elephants. decision-making process structured , benefited whole team. guidelines rubric supervisors helped keep us right track. decision-making mostly based impulse , following process ideal getting results. great team members pay attention detail since mostly focus bigger picture. made sure work appealing. quite experience , constant learning. growing skill set , including soft skills. beginning , lot learn .  ‘ greatest opportunity part 100 chosen individuals wonderful programme. explore data science academy ( edsa ) environment makes feel welcome perfect terms collaboration networking different individuals interesting views personalities. ’ hard distinguish edsa work environment. type environment prepare real job. edsa provides one opportunity learn work diverse group people terms career background , culture religion. working groups different individuals project exciting. throughout time , ’ learned lot working teams. first ’ learned team work achieve terms results learning new things. secondly , come decision making , everyone opinion coming final decision another thing. learned ’ voicing opinion , convincing fellow team members indeed opinion fruitful. comes team strategies embark particular project , ’ one size fits situations. team come strategy going solve assigned problem. difficult , especially serve team coordinator serve , time complete different team whole set different personalities characters. instance learned strategy based skills , capabilities mostly team ’ attitude towards project hand. knowing strengths weaknesses team members good start. far learned lot. ’ still intrigued eager learn lot more. feel like becoming real data scientist day passes thanks edsa giving wonderful opportunity .  explore data science academy 19 weeks ( counted ) ; let ’ simplify 3 months shall ? exciting working group challenging problems , feel like world possibilities opened us. exciting overwhelming met many new people. right away seem made name myself. proud , even excited people feel someone come help - practical emotional. helping others role comfortable since typically excelled academic environment , ’ admit disappointment people ’ ask help more. like thought expert ( even like feel like expert , find happens less grow older ( experienced ? wiser ? one old greeks said something wisdom knowing ’ know ) . aspect larger group find overwhelming trying remember everyone , especially since many people seem know am. enjoy learning people ’ backgrounds amazed great selection people attending college. experience working intensely type team entirely new. worked teams school , university work , never amount diversity team. never sitting next team members everyday ; majority one ’ efforts focussed team work. immediate challenge first team working people skills ways operating different own. realised career worked people , background training. made less adaptable less tolerant different approaches problem. trying learn relax give people ’ method approaching problems chance. felt comfortable group leader first sprint , uncomfortable giving reigns second sprint onwards. second project usurped actual leader number occasions seldom pulled back. formed real bond first team , despite ( ) struggles. second team relationship seems formal , although things went much smoothly team arguably effective. found frustrating deal reserved people. hard read difficult reserve judgement ’ produce tangible outputs. still trying figure best deal people , draw shell patience hear trying say. also need consciously tell offer may valuable ( remain unconvinced evidence right table ) . experience also underlined much rely external encouragement praise , played interactions students staff. enjoy teaching students , specifically one one small groups , alot immediate feedback. first weeks felt really supported facilitators , later weeks felt like abandoned newer , exciting projects. similarly felt much secure first mentor ’ hands-on , available , approach second facilitator ’ reserved , hands-off approach .  young age always sheltered , reserved terrified getting comfort zone. knew year would challenge ’ challenged break shell. first day got real knew one person hundred. push open interact crowd ( struggle considering ’ shy standing front people makes extremely nervous ) . lucky people academy friendly understanding. working teams never problem , issue able fully center attention speak thoughts. taught importance communicate ideas without shy. one biggest challenge working team dealing teammates ’ fully committed tasks assigned them. personally prefer get things done certain way bit challenge team timelines ’ met team members ’ putting much effort everyone else. lab exposes us different personalities one needs learn work together effectively many personalities many conflicting personalities could recipe disaster. could say lucky teams assigned ’ difficult. understood importance contribution communication. misunderstanding ’ address becomes conflict , way able prevent it. teams decision-making process pretty much same. generally , challenge put team , communicate strengths weaknesses , able distribute tasks together based strengths weaknesses. decisions need made , every member given opportunity give opinion decide together team taking consideration every member 's contribution. overall , experience far great. learning new things everyday. workload get challenging time time managed. slowly coming shell , talking , interacting opening .  experience academy really mixed bag far. initially `` '' whatever decision making process needed order get ahead , new group , 've participate others made feel `` follower '' leader times. 's really nice working people myriad personalities. n't really think 've come across situation tempered emotionally although think quite emotional person. think far decision making way prefer ; logical straightforward also room change improve personal knowledge  find experiences academy interesting fulfilling. really like working smart people explore gives abundance. personality clashes character development hurdles point need growth adjustment. one real challenges find balance zoning focus work teaming discuss ideas others help others. choose start would rather acted like focused introvert would give time learn without harming learning curve. honestly contributed negatively group members know , well , deliver more. working together lab thing made real progress growth us individuals. crucial team decisions easier make. working lab also downsides feel still take full advantage privilege. missed couple key submissions already , messed two tests failed badly. made aware team dynamics past negatively affected catch-up work cost team process. honestly think hard also says high expectations myself. decision-making process usually one based voting. people seen getting things knowledgeable automatically obtain voting power. past experience work usually drive person chooses group role willing play. future , would prefer , rules engagement given thorough attention prior commencement group work. way , best obtained everyone. personally believe need place comfortable enough work done tools available. accomplished. achieve contribute .  experience academy thus far potent insightful year , changed life become person really enjoys importance cooperating others. brought together amazing minds space really helped acquire soft hard skills short space time , indeed helped elevate way learning new technical subjects efficiently effectively. also , management team large especially supervisors instrumental always improvising make sure led way help us walk away well informed junior data scientists end year , hence work tirelessly making sure break obstacle attractive easily comprehended possible us order put hard work. addition , nuri formidable way helping us cope psychological illnesses caused intensity program , always go extra mile making sure one feels solitary. supporting structure throughout program importance , help us understand essence behind excruciating vulnerability management enable see work not. last words , want close saying means life , 'm grateful opportunity , best thing ever happened life besides varsity , 'm pretty sure set life greater things achievements .  time academy far challenging , yet extremely rewarding self-fulfilling - especially regards working people different backgrounds domains expertise. although meeting different people interesting , concerned bring different people together towards achieving common goal. two teams worked , often one take lead , drawing ambition past experience relation data science related fields. challenging , since used - comfortable - working alone. hold personal contributions team-based projects highest standards , therefore difficult team members show dedication team-based projects do. working teams challenged personality positive negative ways. like control , therefore difficult allow people take lead different aspects projects , especially objectively feel experience completing particular objective. however , learned impossible control aspects project therefore welcomed aided contributions team members shown willingness aptitude contribute. contrast , difficult form motivate team members show apathy towards achieving common goal. although somewhat frustrating , ensured never derailed team following initial plan objectives. felt comfortable role coordinator , relished opportunity deliver high-quality deliverables high pressure situations. drive work ethic good , always productive worker. over-analysed spent excessive amounts time subjects interesting challenging personal level , instead focusing time pressing matters. generally tend spend time challenging interesting concepts , tend postpone often neglect routine administrative tasks. creates frantic dash line time runs , creates unnecessary anxiety. conscious shortcoming , actively try remind prioritise complete pressing matters regardless personal interest. know often regarded serious fellow team mates , since always focused driven comes task hand. little time chit-chat tend engage conversations related intriguing philosophical subjects. feel comfortable responsive. however , learned casual conversation regarding sports hobbies helps learn team members , forges professional , understanding relationships team members. overall , work experience academy enlightening one , far perfect. lots learn , eager improve aspect career benefit team mates .  positive experience academy far. enjoyed meeting people interest data science also made unusual choice learning institution. fascinating getting know people class. people incredibly wide range personalities backgrounds. generally enjoyed working lab though get rowdy lunch people 's work motivation decreases motivation socialise increases. found first team members rather like-able , felt team dynamic failed dismally properly tackling project. success beginning ultimately disappointed team ’ performance. lacked cohesiveness n't project manage effectively. people focused exclusively individual learning rather group work. second group considerably better. clear strategy allocated tasks effectively. regular meetings , took minutes , wrote tasks sub deadlines. member motivated pulled weight. also communicated frequently balanced individual group tasks much better previous group. decision making initial group rational however rarely executed decisions intended. lot procrastination members arrived late class left early. one group member spent time away group chatting friends. claimed working group tasks , ended contributing absolutely nothing group tasks. contrast , individual second group went beyond call duty. regular meetings ensured tasks divided properly. progress closely monitored , problem occurred , rallied together solve adapt strategy. one member group already quite experienced particular topic project tremendous advantage us. thought carefully team workflow set appropriate software working folders. analysed project marking rubric created gantt chart schedule tasks , understand dependencies. also manage group member availability member group required periods absence project. also scheduled tasks around class tests aware time allocated individual efforts. initial group unhealthy diffusion responsibility member assumed someone else would pick slack. lead group members becoming despondent refusing carry group members. also faced issue one group member ’ feel sufficient programming skills make decent contribution group effort. despite shortcomings group got reasonable marks. credit go two five group members. sure second group get good marks. credit also given equally group members one member particularly strong. however think efforts equal previous group everyone felt satisfied group effort end .  bit loner times id say experience really taught engage people talk especially student kind goal. based academic level think got experience programming fact familiar sql python shows grown academic level. knowing wake everyday see new faces lab thought able manage knowing loner told working teams thought end much talker people thought going express opinions , participate happened best experience even started releasing need talk order opinion heard. last team really understood much teams challenges valued opinions decision making part even pimplier , first team agreed everybody gon na take part everything look strength weaknesses. assign one another activities concerning project still forgetting help strengthen academy weaknesses basically created fridays helping other. comes decision coming acadamy one hard decisions take cape town far home choose completing honors degree academy. ended choosing academy honors degree felt like needed experience data science marketing world academy would best place start fact ought learn everything beginning great computing skills wanted stipend also contributed decision coming here. would conclude regret coming instead happy here.its 5 months already feel like year going bit faster want even programming skills think would got much experience programming languages skills honors.the experience got far best. helped growing academically also helped gain confidence individual .  like meeting new people similar interests , many new people , connecting people time goes assigned new group. n't spoken students merely strong introverts , recent group nominated group coordinator take lead every meeting , every interaction acknowledge personal level group members feeling day , something never really thought experienced , even grew every mondays share weekend , really appreciated put position challenge grow personal academically. like make decisions based merits decision efficient come attention efficient time take account people 's feelings , made decisions past group people feel included left , worth noting individuals contribute project way , barely showed academy , never written one single line code left member take care making dashboard , presentation ultimately present , two us took uncomfortable route diplomacy , pleaded every morning attend briefings supervisor us , went group training sessions made impact , next group , understood academy meant worked together well. stallions , magnificent , sudden paradigm changed , enjoyed one idea best things , originally typed minutes meetings another member suggested trello effective , learnt something new , excited , day , well every day went bed knowing something ’ know woke ’ data scientist ask for. one important lesson got working two different groups leave time guide team work flow everyone miserable end project , however everything planned trello ( ’ trello ) ‘ tees ’ crossed ‘ eyes ’ dotted , pleasant experience bring best everyone involved , team creativity grow levels one planned beginning. , irrespective put group work , sit maybe second session “ getting know ” map strategy roles assigned , significant insignificant .  edsa reflections 24 november 2017 , filled joy , excitement anxious time opened email edsa titled “ congratulations ! ’ accepted explore data science academy ! ” fast forward 22 january 2018 , ready meet new people ( possibly form new friendships ) , meet edsa team learn lies ahead 2018 , overwelming day was. today 6 may 2018 , lot happened past 3 months , ’ learned much ! sql , stats , python , power bi , presentation skills… [ still learning , never stops : ) ] . grateful selected one top 100 , ’ appreciative opportunity , thank edsa making possible me. lab data scientists making , greatly enjoy lab , environment condusive learning , group data science enthusiasts one room feel receipe success. training limited online courses edsa staff students learn lot , different backrounds , skills experience making lab interesting fun place grow academically socially. teamwork water crisis project : team liquidata must say initially concerned amount individual work deliver team well. realized importance effective team , working well team reduced work load stress also helpful individual tasks. roses regards team dynamics , working different individuals perceptions opinions challenging first , frustrating decide certain matters , helped lot grounded problem statement kept us focused needed done. worked well ’ result driven given task , completing task successfully required relevant info important , refering back problem statement got stuck great tool us. emotional moments team due personality clashes end worked well contributed success team , awesome team ! rsa stock market project : wolves woodstock accepted ’ work new people , really problem , hope wish strong programmer team , wish realised statistician programmer team , lovely ! great team , felt like outspoken person ( ) bit much , still worked well though delivered. prefer working team different ideas , creativity different skills ’ huge advantage eveyone team result driven , absolutelty appreciate that. ’ good experience teams far .  first day first looked familiar faces. started looking people would like associate self with. meeting new people fine long n't expect keep conversation going. found nice , guys hang with. love working together labs , could share idea 's , findings ways certain things. enjoy working teams. believe 2 minds better one. also another line validity checking. one also learns lot new things working groups. team dynamics like , everyone knows need achieve bigger picture. long term though ones ideas change clearer picture , completed task. ( problems change unforeseen problems come up. ) decisions made done still needs happen. times felt team members know exactly needed next ( floated around ) . decision making based possible without overworking everyone ."


In [53]:
FtXt = "  joined academy crossroads sorts life. academy offered opportunity pivot career engineering data science unfairly dismissed struggling find employment result. meeting 100+ ( faculty students ) new people would part life whole year something get used , even though worked multinational companies. ability deal people different backgrounds , beliefs , etc. tested group work. needless say lead tension one 's life team mates pulling weight. team dynamics always learning process/curve , thankfully bad experience till now. needless say , people try cheat system , act dishonestly sometimes plain lazy. many opinionated thus rendered incapable making great contribution. way things always consider everyone 's well-being , opinions contributions. somehow persons think ok cynical unhelpful. end day , supposed learn become data scientists. work , improve daily. .  far experience positive definitely see value course. experience working different people valuable me. worked well attitude everyone team deliver take responsibility. difficult step back trust team work , however cases team members exceeded expectations. felt bit frustrated team members applied taken substandard lazy approach problems. like get stuck something get immediate results improve there. think approach worked well however also learnt allot team members structured task based approach. allot time spent tangents add value projects however tangents either paid results learnings took them. decision making approach generate ideas create structured work flow around ideas distribute work among individuals take interested work flow. conflicting ideas typically took democratic approach combined healthy debate. find build relationships people 'm teams with. get know best. strongest connections team members also people academy learn from. took team bit data collecting tangent last sprint necessarily productive use time could avoided understood problem better. one case probably assertive team , however way like make decisions get stuck workflow stream adapt go n't supervisor direction point. find environment stimulative enjoy going around teams see working on. good relationship people happy share working great. also found environment distracting times , would like help people get satisfaction , however 'm also that. working different people mixing regularly something im excited everyone different learnings experience theres something learn everyone .  fortunate opportunity join academy year. sure career choice , academy provided great chance change career paths. help choosing new path allow eventually reach larger goals life. moving cape town joining academy challenging , already knew two people academy joining. however made bit difficult making new friends. typically reserved , easily make new friends , especially already good friends , case see reason make new friends. working teams academy great challenge me. normally prefer work pace working team forced cooperate people adjust skill levels. sometimes meant taking work allocated me. addition , found difficult working individuals invested projects aiming learning much experience intend to. within teams , leadership quite lacking. typically like forcing onto people , circumstances , forced take lead projects. taking lead tried best lead behind , making sure consensus within team deciding. generally due lack experience one teams , meant think plan , execution thereof , followed team implementing without challenging ideas laid out. approach worked well , would liked greater input team , unsustainable left would render team incapable functioning properly. team , people much competent four members , decisions much easier take. good idea , still elected leader , role much organic time , much balanced consensual way making decisions. however , completion tasks effectively still perfect left. teams struggled making decisions , never experienced conflict generally emotional making decisions .  overall experience academy far great somewhat challenging. firstly team leader elected team ; tasks disturbed among members. disagreement team meetings. achieved aim , members satisfied. team achieved aim could done time. allocation work workloads decided team leader discussing team. arguments workload. team leader assumed everyone level. questions answered answered satisfy members. individual members needed extra research questions answered properly. team able work together , reliable tried level best punctual meetings. allocated different tasks person researched thoroughly co-ordinated findings together team asked support needed. personally helped individual confident , however bit strain keep mates vast experience programming statistics sometimes felt incompetent tasks given , helped gain knowledge pushed learn harder. role played team members team player , several suggestions made everyone team adopted. team selective adopting new ideas. team worked well , agreements reached almost issues little new information gained members. believe team work ideal method quick information gathering , learning oneself time consuming tiresome. believe contributed effectively team reliable although bit hesitant first stages adjust different characters communicate freely afraid judged believe group work essential would mind working team aim team achieved .  got academy , felt like n't belong here. felt place. person n't like talk people own. people approached , got know them. getting know fellow classmates , made realise feeling similar emotions 'm feeling. 've met lots like minded people. personally like work alone , working group always difficult me. academy made easier me. forcing work teamwork skills. often , 's group member takes charge , keeping team focused shared end goal .  start academy quite tricking due meeting new people , unknown environment away home family proved tricky. first fundemnental sprint extrememely challenging memebers group failed help work assignment hence resulting lot friction stress. poor coomunication bad attendance critical times. resulting wasted time send message finding nessecary details perosn thus wasting time. problems poor team dymanics dealt memebers realise importanace good team dynamic order complete set task. next sprint alot better group organised everyone commited assigned task intially lacking prior sprint. group communicated effectiely term problems incountered. first sprint reveal emotional weakness team memebers unable keep instead moving foward broke away decided give instead still trying help team current assign task. second sprint clear cut division two teams helped left driven knew expected .  journey exciting. stress constructive sense forced work. everyday excited come academy interact vast personalities class offer. given nature work , days really tough days really good ( especially code running ) . academy taught , testing limits understanding. whole life 've always team player. although n't like charge , somehow found leadership positions based assertiveness became norm. shocking finding opposite position first sprint. zero communication whatsoever , made feel uncomfortable could n't share thoughts feelings , however found way work well collective. took forever wrap head around content work reason made feel futile. due lack communication , people felt right thing , would later put everything together. opinion , think structure , leader would keep us track. way decisions made , shared responsibilities based people 's strengths weaknesses. equally made decisions led productivity , sometimes would disagreements. times , would emotionally draining , every low , growth. throughout roller coaster journey , 's nothing growth , understanding learning. understanding workplace , understanding colleagues understanding .  started academy workload easily manageable students able keep learning curve , whereas struggled either asked peers caught , n't got stuck behind pack. workload difficulty increased , able keep better programming foundation others course. although might helped lot , appeared blessing curse. found losing lot time could spent work , helping students ( bad thing ) , questions asked topic currently working found difficult get back line doing. decision teams need made , seemed myth. team needed carry lot workload rest team members order get scores wanted , trying teach team thing two long way. similar situation second sprint , severe , 2 members team got flying start , 2 spectating participating one eventually found feet ended bang , last member , felt , never contributed final `` prediction '' , speculations. quite frustrating like working teams , however members carries weight. might sound selfish , n't want affecting results much goal year 1 30 students going bcx. feel working hard enough spending enough time working , still happy ultimately hand present group. finally happy course content. course already taught lot data science thought knew ca n't wait come. awesome experience happy chose .  coming academy best thing ever happened grateful opportunity. learned academy going encourage teamwork excited meet new people different spheres life exchange ideas network it. working teams thus far ok. things like team commitment work work ethic thereof. comes decision making n't enjoy part felt sense dictatorship member would defy group 's decision still want impose best team start delegating work teammates based vision. felt defied purpose brainstorming planning objectives. role played team always conscientize group imperativeness working within scope prioritizing required us form basis going structure project would help decision making. supervisor meetings reporting based `` '' , `` '' . sign individualism collective project perpetrated unnecessary competition. enjoyed networking variety people academy also participate win hackathon hosted city capetown one best networks team collaborations went well me. like random conversations technology trying link 'd connect inspire other. brief summary work prefer plan try much possible stick plan always room changes. consolidation key helps deciding conflicting ideas .  experiences explore data science academy experience meeting lots new people great one , months already learned lot , continue daily level. terms meeting new people much gain , especially terms offered different perspectives , challenged way thinking. also taken comfort zone turn taught lot handle different personalities. experience working together lab teams much appreciated close real world experience , 've already similar environment throughout university always found enjoyable interact people. level productivity performance exponentially increased , mainly deadlines needed meet wanting disappoint teammates , generally working others helps achieving individual. say interpersonal skills increased , example speaking listening others. come realize collaborating others helped identify strengths weaknesses , basically self-aware. experience decisions made teams was/am consistent like make decisions. lot people considerate want attain goal , appreciated , far people worked take time listen person say try take consideration relevant concise , decisions made depending everyone felt. experience rational emotional decision-making process been. decisions made rational group took time take people ’ emotions consideration possible. ’ obvious always agree completely certain things , occured , time taken explain person things certain way beneficial way disagreeing member thought. accepting possibility incorrect important role , especially comes working others. conclusion , experience positive anything academy .  experience far great though first bit anxious excited whole program , working teams challenging first still becoming less less challenging , far groups far completely different 's interesting terms knowing works n't , also helps experiencing different thinking processes ...  experiences edsa experience academy intense , interesting fun. even though met small subset people academy , great gotten know time. also particularly interesting , seeing different everyone ’ background perspective is. learning come want life chose apart academy. working lab great experience , reminds worked open plan office space feel like ’ great comfortable environment work in. also proven difficult times , slow computers noisy environment keeping focused sometimes tricky , views mountain really great ’ complain much. regards team dynamics , two groups part different terms worked , however overlap dynamics. one dynamic found two groups extroverted individuals groups took leadership role early team took cues two groups pushed leadership role. part groups introverted individuals extroverted ones. ’ major conflicts clashes dominance. first group differences personalities caused friction conflict. despite , got long worked together cohesively meet goals. natural personalities would create conflict never lost sight there. regards decision making two groups. first group lot uncertainty terms approach problems created environment aimlessness uncertainty times. second group came consensus based strengths team members would allocate roles accordingly. decisions first group made based high emotions think caused new environment learning new skills created stress , coupled trying meet deliverables made tensions high. whereas second group came decisions based logical approach. would say overall decision making rational tried deal problem statements thoughtful planned manner would allow us cope coordinate responsibilities effectively .  experience explore data science academy great far. goals explore year develop & master skills data science well improve leadership skills. felt empowered content ’ taught. four months , believe already learnt skills would allow enter & succeed job market. next 7 months plan spend refining data science skills using 1 hour evenings revising content learnt class. regards second goal ( improve leadership skills ) , ’ fortune team great coordinator & excellent team dynamics. also great team coordinator team. ’ taken many lessons experiences read book. given practical lessons improve leadership skills. ’ two teams far , worked first 1. coordinator nudged us always making progress towards bigger goal. 2. teammates voluntarily picked segment project would responsible for. 3. loose deadlines agreed , deadlines also allowances members equip skills required fulfil parts project. 4. team met twice week discuss progress. 5. time member grew comfortable asking help others well given feedback. team dynamic encouraged member give part project all. inevitably , team members technically advanced others ended taking advanced work rest , however time felt like overburden. given positive experience eager also try coordinator next sprint came around. new teams assigned & volunteered coordinate. aspects believe could done better. 1. ’ commonly accepted bigger goal. 2. ’ involve team identifying small goals were. worked work plan overnight & presented team. team loved plan , individual responsibilities work assigned. 3. led situation members team clear responsibilities were. 4. result power leadership , members team ended bulk work. able pull together & present satisfactory project , however believe could led better & helped team deliver far better project. member team would also proud contributed fully project. learnt great deal experience & would like give coordinating another try , like skill believe leadership learnt practice .  explore data science academy , road never thought would take lifetime. love passion cars , accepted data science internship one last thing never saw coming. love travelling exploring new avenues `` let 's '' tip. first day coming explore data science academy ( edsa ) beyond expectations , visually appealing facilities provided us. first could believe , cape town , comfort zone , pushing something involve anything studied. came expectations , however , told something need give order leave experience back future. speech supervisors gave us 22nd january reassured make mistake coming here. first week given work , install softwares would using throughout year , thought would pre-installed arrived. first , introvert side steering wheel interact lot people besides housemate. started interacting people water crisis sprint started. team people different background ; bioinformatics sciences , computer science , economic statistics mix , different parts south africa ( mostly gauteng ) . one things stood type people would start conversations loud side come order make everybody feel comfortable. first week gather sprint took team start making progress , thursday already figured system worked everyone order use something show supervisor. put weaknesses strengths table , started communicating better team. results shown supervisor told us think would deliver delivered , however , delivered end fundamental , faith would best one best. love good music , adventure socializing led connecting people group. time fundamentals spent helping people , building friendships would carry edsa year even greater , learning languages like python sql , channeling mind data scientist mode. weekends colleagues would tour cape town going beach , hike mountains could see edsa lab , visiting malls theme parks around cape town. python , one complex languages learn. well , thought since programming background. two teammates experience working languages , changing one language another , fairly doable. advantage since two people could consult whenever ran problem python. presentation fundamental one laid back fundamentals face since preplanned wanted add slides clear storyline. fact split fundamentals tore us apart , however , vowed exit whatsapp group created. review week , weekend , people never talked friends edsa lab suggested go theme park near canal walk mall take minds water crisis sprint. character part outting shy , yet interesting backgrounds. still friends them. stock market sprint came flew by. really feeling team since lot judgemental people felt like knew better. lacked communication , managed pull find sprint one piece. events bcx hosted highlights first months edsa , amount inpirational people engaged events. felt like meant every time spoke lucky waiting us greater wildest dreams. week takes cup. never expected go home least july. honestly grateful spending year edsa lab , could chose anything else .  experience edsa far combination challenging , fascinating , unpredictable. coming academy , excited would get meet new people learn new skills ; anticipated might somewhat difficult engaging new people ( since introverted ) coming grips new technical ideas ( creative technophile ) , since loved good challenge , still excited immerse experience. surprise , challenges faced none anticipated generally able interact cordially new people understand technicalities program. greatest challenge far working lab. environment many people ( different personalities , habits , learning techniques preferences ) forced natural inclination isolate working. easily distracted , think carefully navigate space simulate environment conducive learning. also found team consider optimally ration work , challenging , especially beginning n't yet good understanding 's strengths weaknesses. said , gotten know fairly well , able make decisions believed would amplify strengths diminish weaknesses could produce results best possible quality. sense , much decision-making teams rational rather emotional. would apply logic reason wanted achieve thought might able , found always room empathetic thoughtful ( thereby engaging emotions rather reason ) , sake team member , alter original plan. somewhat consistent personally make decisions : think times base decisions think yield best results terms trying achieve , even happens inconvenient taxing process ; times , especially decision made great consequence , make decisions based emotions believe 'rational ' 'logical ' decision likely hurt someone 's feelings cast bad light. , learned awful lot : preferences , strengths weaknesses , character ... found learning curve steep emotions rather turbulent throughout process , enjoyed every part challenge could mean future -- prospect holds together worst days exhilarates best days .  experience edsa interesting one thus far. pushed get shell interact people. since academy realised smarter think lot valuable knowledge insight share. biggest fear coming academy social aspect interact different people , since able overcome fear. first placed groups literally sweating , heart rate high felt sick stomach idea group strangers. second time split new groups ’ nervous. become social enjoy meeting working new people now. working groups challenge me. frustrating everyone different everyone different approach learning process project management. ’ learned lot working others. took backseat things even knew better way things. spoke saw helped team steer team better direction. become assertive. thought passive person ’ assertive save life , think assertive ’ find daunting. biggest frustration working teams dealing people think know everything ’ willing open learning something new fellow teammates. person like previous group. person would dictate team ’ want collaborate team , dominating personality rest us shy. realised none us ’ speak ’ fail sprint forced breakout shyness. think still fairly shy comes work fight shyness. decisions previous group initially made one person. prefer problems discussed everyone team different possible solutions explored come collaborative solution. found current team. still issues , people letting pride emotions get way work , people distracted ’ committed project. working labs challenge. lot distractions noise focus easily , prefer work home , get much done home. people come academy socialise anything distracting .  experience academy exceptional. meeting new people amazing , specifically working them. much challenging first inception relate , time things gets better begin know get use one another. odds events one two group member want participate ( eg. missing meetings ) thing way oppose agreed group , really problem participating also productive , especially meet group expectations group works , members end work. means group take collaborative decision planning expect everyone deliver agreed planned. lab one best way work , especially comes group work , find collaborative useful. 's unfortunate group members choose work home leave early come late. found behavior hinder progress , especially tasks divided sub groups among group members. critical aspect group always communication. members usually communicate inability complete task late work due. members get emotional approach , end day always encourage one another see right side approach task project hand. working many different people taught rely others , means trust member work , even better me. fun leaning since day one .  enjoy meeting new people different walks life constant , everyday social interaction. slightly reserved comes approaching people much older me. also fear inadequate unqualified many people already worked professional environments qualification , undergraduate otherwise. large emphasis group work big learning curve always preferred work alone everything way want , although improved since time here. tend take charge unless one else better talking people helping find role. versatile n't mind taking least desired role despite preferences. one group decisive. general guideline try stick although hardly case. prefer versatile approach work. rather consistent method making decisions. little less organised group though. tend procrastinate make major decisions complete important tasks last minute. tend sway easily depending emotions. know let emotions run ahead ( since something known ) put foot , take step back examine things rational perspective. hard easy put emotions beside .  life academy. wonderful experience , joyous full many possibilities. 22nd january 2018 took tax kraainfontein heading woodstock , place never , told far capetown cbd. exact location going woodstock exchange building 's explore data science academy located. help asking people taxi help gps able find building got around 10:30 , yes late got presentation room aidan ( supervisor ) giving overview academy expected us ( student ) . room full 100 students different provinces , races backgrounds. listened long bearded white guy speaking eloquent profound right. one thing struck intention emphasis speech intelligent ( students ) apparently 8000 people applied program , 3000 people able finish whole selection process 250 people interviewed top 100 made program. aidan 's words full passion compassionate , inspiring encourages , gave birth new hope life knew labelled people 'failure ' knew opportunity beginning great journey ahead. promised things different time , work hard play less , would make friends pray give thanks heavenly father. week later ( students ) split group five people work capetown water crisis using skill learnt past week i.e sql , python power bi. group name bean bags , group four males one female , embark project , use divide conquer approach help assign work member , group members wonderful people work , jelled much fun together . bean bags often worked together , using one computer helped tasks assigned member. worked together two months present findings regarding capetown water crisis solutions suggestions sprint came end. split south african stock market project , working different people adjust new team went name stockoverflow , bit nervous previous team already made friends working together forged friendship amongst adjust. stockoverflow team three males two females , sprint month much done. learning different models predict stock prices fascinating challenging me. used divide conquer approach assigning tasks member consolidating member done google drive , 26th april presented model predictions stock price one week break came back another sprint. realized short space time learned many many skills apply solve real world problems. wonderful experience grateful .  big part joining academy year meeting working lots new people. unexpectedly positive experience. despite everyone different backgrounds , sense camaraderie interns friendly. even though everyone tested graded environment non-competitive people willing help other. made enjoyable work lab. however , noise bustle lab sometimes overwhelming draining find productive sometimes spend hour two day lab working home quiet. challenging aspect year far teamwork. first project ( cape town water crisis ) times conflict team due team member contributing much therefore shifting workload team members. led resentment team members , expected. team projects work rarely equally divided team managed complete project despite difficulties. reflecting experience grateful done big share work gave opportunity learn lot would otherwise. second project ( jse stock market sprint ) much shorter time span therefore team approached project different. team divide work time everyone involved deliverables. hard meant trusting people work without directly involved dependant complete it. process decision making teams democratic. team discusses problems possible approaches usually obvious decision everyone agrees on. decision making far tended rational team members emotional. emotional part projects stress faced final deadlines neared. , experience academy far positive learnt lot. hope improve way approach team work upcoming sprints .  choosing spend year explore data science academy could easily best choice ever made. choice renewed feeling purpose educational/occupational direction. learnt past 3 months thought would entire year grateful presented opportunity. fellow students edsa staff added excellent experience encountered thus far. interacting brilliant minds results one , inevitably , attempting better experienced first hand. excited path edsa setting look forward future holds prospective data scientist : )  hardest moments life learning new things , matter simple easy might be. difficulty met new people academy. truth talkative nature , ’ always found difficult start conversations students anyone academy. however , amazing experience discover learn different students ’ cultural backgrounds diversities , interest data science life whole. academy , realized people learn discover interaction others. new environments nervous , yet exciting ones. explore data science academy best example scenarios. academy provided great exposure working environment challenges ought face work teams. able work team prerequisite skill needed data scientist. learning curve allowed self disciplined , self-motivated introspection little effort group work affect entire team ’ progress thus , results poor performance entire team work. people different cultural , educational programming backgrounds , learning difference right , right right became key determining factor understanding work teams. months academy , feel close becoming data scientist before. hunger efforts put understand data science seem rewarding now. great feeling among first group given opportunity learn data science. although pressure much times , hope believe everything designed meant prepare us real world working environments .  mixed emotions went came cape town.i felt ecstatic opportunity afforded yet side intimidated new environment. walked building first day heart filled much anticipation could wait shake everybody 's hand introduce myself. meeting talented young old bright sparks . never worked team came academy , know working team often many times worked alone everything myself. changed , teamed bright minds actually got work . really surprised much effective efficient working team. really enjoyed working team. believe travelling meeting new people potential widen ones horizons. learnt lot still learning lot different people academy , academic related lesson also life lesson carry ever go. wonderful fruitful time previous team , though ups downs .i learnt n't leave issues group grow rather need tackle emerge .  22nd january 2018 , time 7:00. seemed like normal day time. hit , difference location. 'm western cape , cape town precise. moved away home embark journey become data scientist explore data science academy ( edsa ) .today 8th may 2018 lot happened since ... first week got meet students different family background different level education. met explore team , included supervisors , facilitators nuri. told would working groups fives. nervous got used everything own. actually terrified. first group called optimize prime ( still cracks ) . made nervous everyone post matric qualification. beginning thought underdog , realised competition. mean must 've reason teamed guys , learn contribute wherever can. next group , named wolves woodstock ( named movie wolves wallstreet ) , confident engaging. developed previous group. even headed one sub projects. experience amazing great. even sister n't believe me. first group given opportunity work cape town water crisis , investigated demand supply factors affected cape town water crisis. project got use sql powerbi first time. saw another use python n't know capability , analysing data. second group got explore rsa stock market data. used data predict target return individual companies. interesting project accuracy predictions , limitations models. group assigned supervisors. got chance mentored two different supervisors. wish could create table comparison , two different , effective. times stats , python others gets much. group sessions nuri , refresh .  experience edsa great far. massive learning curve academically also personally. took myers-briggs test twice : , last night earlier today got slightly different results. first time came letter combination infp second time isfj. though slightly different , feel accurate overall representation various attributes exhibit. seen diplomat-mediator well ( second time taking test ) defender sentinel. naturally reserved nature , unfriendly , prefer working alone comfortable enough sit alone group people. however , approached , come across friendly. able read various personality types groups fairly quickly , allows quickly adapt different situations needed group times. n't usually take charge , know end goal something achieve , prefer clear cut plan achieve , one challenging parts working group , made need step comfort zone recently make decisions voice myself. times experienced stress due fact felt frustrated able comprehend certain concepts quick enough significant work project , deadlines much issue. overall , feel thoroughly enjoyed learning group dynamics far edsa .  nan  nan  enjoyed time edsa far. love meeting new people first time life talk people face-to-face programming topics around data science. previously strangers internet. great discussing algorithms coding challenges geeking people understand contribute , rather watching people 's eyes gloss describe trying achieve ( laymans terms ) . love prescribed uniform everyone comfortable naturally ease , opposed fashion parade. countless times struck conversation work , weekend even mountain. sometimes team members get distracted abit long guests talk abit loud , n't big thing. biggest challenge balancing studying implementation theory everyone page. team suffers individuals pulling weight put strain rest. luckily found colleagues kick around ideas discuss solutions current problems. decision making process voting best way forward. typically consensus reached roles , typically 2 3 members end bulk work. conflict resentment arise far real flare ups encountered , hard individuals argue decision made present put work. tend keep peace expect everyone act respectable adults. hate micromanagement love freedom given solve problem statement. glad able view teams ' presentations see went solving problem. continue learn lot data science well working time time constraint seeing people manage. even workload others offer time assist need help  year huge one change , involving many aspects : leaving stable job , moving cities , moving new field , going back student , long distance taking chance brand new data science academy. quite excited meet new data science students despite usual shyness. people taking similar plunges shared interest field. enjoyed ( slowly ) meeting new people learning backgrounds , interesting varied stories are. working lab new experience. complete self learning tough journey , glad academy experience supervisors help guide us way becoming data scientists. bit unusual area ’ computer lab learning , also sort office , also people socialise. ’ sure nailed exactly be. feel work better home quiet comfortable ( apparently line personality profile ) . group work element year probably one biggest pain points. see intent setting us way , ’ think quite worked yet . group work challenging. throw meeting new people different backgrounds , huge amount new content learned fairly significant stakes ( internship opportunity ) difficult situation. think resulted frustrations running high poor quality work delivered , individual group would wanted. think worrying getting project done sometimes gotten way actually trying learn content. terms decision making groups far , think one positive definitely logic valued field students. state reasoning behind plan solidly ’ problem. think challenge group member interpret instructions priorities differently , sorting get get everyone page take valuable time. personal level noticed always looking clear , decisive decisions plans , deal well “ seeing goes ” . think something need look team setting , expect manager well. overall , still pretty excited year regrets decision join. think academy still bit rough around edges needs refinement , part iterative process right ?  experience edsa one sparked lot introspection. socially , made affirm fact extroverted person cherishes human connection , communication cares 's well-being. regards team-work , made realize quite assertive prefer logical structure comes completing tasks fulfilling set roles. made also realize end taking charge situation group , even n't initially bargained , mainly seems like n't much direction group time seems going waste. decision- making process rational one mostly times based emotion people would choose certain roles based believing good certain thing interest end n't end achieving/delivering much end burning out. 've also realized things n't go according plan comes activities part role , stress feel defeated easily perfectionist side creeps end completing needs done time. future teams wish branch attempt master every different skill best ability instead focusing much individual role , lot potential know capable ."


In [61]:
PtXt = "  overall experience academy far great somewhat challenging. firstly team leader elected team ; tasks disturbed among members. disagreement team meetings. achieved aim , members satisfied. team achieved aim could done time. allocation work workloads decided team leader discussing team. arguments workload. team leader assumed everyone level. questions answered answered satisfy members. individual members needed extra research questions answered properly. team able work together , reliable tried level best punctual meetings. allocated different tasks person researched thoroughly co-ordinated findings together team asked support needed. personally helped individual confident , however bit strain keep mates vast experience programming statistics sometimes felt incompetent tasks given , helped gain knowledge pushed learn harder. role played team members team player , several suggestions made everyone team adopted. team selective adopting new ideas. team worked well , agreements reached almost issues little new information gained members. believe team work ideal method quick information gathering , learning oneself time consuming tiresome. believe contributed effectively team reliable although bit hesitant first stages adjust different characters communicate freely afraid judged believe group work essential would mind working team aim team achieved .  got academy , felt like n't belong here. felt place. person n't like talk people own. people approached , got know them. getting know fellow classmates , made realise feeling similar emotions 'm feeling. 've met lots like minded people. personally like work alone , working group always difficult me. academy made easier me. forcing work teamwork skills. often , 's group member takes charge , keeping team focused shared end goal .  wonderful place , place waking morning easy task great desire learning something new everyday. meeting new people great thing learn work team also socialize since person well crowd. actually first time working lab like reason experience labs quite instrumentals labs testing experiments computers , n't computer labs first time coding lab reason coming edsa first choice . skills data scientist one drove apply wan na learn coding. learned lot lab also working team. one main problem faced working team group rather , groups perfect might wish change disaster.sometimes set time slots group meet discuss progress deliverable surprised one member pitch thought lot work , disappointing would mostly prefer working alone future. working team sometimes emotional draining decision divide conquer could work together well. hoping great team smart people need told since every sprint `` lists '' .  start academy quite tricking due meeting new people , unknown environment away home family proved tricky. first fundemnental sprint extrememely challenging memebers group failed help work assignment hence resulting lot friction stress. poor coomunication bad attendance critical times. resulting wasted time send message finding nessecary details perosn thus wasting time. problems poor team dymanics dealt memebers realise importanace good team dynamic order complete set task. next sprint alot better group organised everyone commited assigned task intially lacking prior sprint. group communicated effectiely term problems incountered. first sprint reveal emotional weakness team memebers unable keep instead moving foward broke away decided give instead still trying help team current assign task. second sprint clear cut division two teams helped left driven knew expected .  experience academy enjoyable far. learning lot interesting valuable skills meeting many intelligent interesting people. meeting working new people lot fun rewarding experience. working lab challenging. open plan allows many diverse unusual conversions easily started joined also prevent work done people interrupt prevent others entering staying state flow necessary learning. especially destructive slower learners like myself. working teams also challenging insightful. sometimes tricky setting understanding team dynamics new teams especially people ’ know well , practice working teams always appreciated team dynamics always easy decipher understand. decisions made group ( least feeling ) without one person charge giving orders. could also part still learning try make effort person group feels opinion asked heard decisions made group decisions everyone opportunity raise differing opinions votes support. consistent way enjoy making decisions think allows everyone feel involved dedicated outcome everyone feeling others group think feel decisions made. say decision making rational emotional know accurately determine that. maybe mix , though slightly leaning towards rational .  journey exciting. stress constructive sense forced work. everyday excited come academy interact vast personalities class offer. given nature work , days really tough days really good ( especially code running ) . academy taught , testing limits understanding. whole life 've always team player. although n't like charge , somehow found leadership positions based assertiveness became norm. shocking finding opposite position first sprint. zero communication whatsoever , made feel uncomfortable could n't share thoughts feelings , however found way work well collective. took forever wrap head around content work reason made feel futile. due lack communication , people felt right thing , would later put everything together. opinion , think structure , leader would keep us track. way decisions made , shared responsibilities based people 's strengths weaknesses. equally made decisions led productivity , sometimes would disagreements. times , would emotionally draining , every low , growth. throughout roller coaster journey , 's nothing growth , understanding learning. understanding workplace , understanding colleagues understanding .  started academy workload easily manageable students able keep learning curve , whereas struggled either asked peers caught , n't got stuck behind pack. workload difficulty increased , able keep better programming foundation others course. although might helped lot , appeared blessing curse. found losing lot time could spent work , helping students ( bad thing ) , questions asked topic currently working found difficult get back line doing. decision teams need made , seemed myth. team needed carry lot workload rest team members order get scores wanted , trying teach team thing two long way. similar situation second sprint , severe , 2 members team got flying start , 2 spectating participating one eventually found feet ended bang , last member , felt , never contributed final `` prediction '' , speculations. quite frustrating like working teams , however members carries weight. might sound selfish , n't want affecting results much goal year 1 30 students going bcx. feel working hard enough spending enough time working , still happy ultimately hand present group. finally happy course content. course already taught lot data science thought knew ca n't wait come. awesome experience happy chose .  intimidating meeting many new people , especially wildly different personalities backgrounds. sharing work space sometimes challenging everyone shares views communal space treated terms cleanliness keeping noise levels down. interacted lot people , small number individuals like background temperament. team dynamics far challenging part course. particularly struggle team-members lack self-motivation work ethic. found often instead insisting team member pulling weight would rather task myself. tend feel someone lacks motivation something unlikely perform task best abilities. also seem little confidence others completing task way would like done. generally , decision making teams easy mostly due apathy uncertainty surrounding decision behalf team members. found frustrating since often lead one-sided discussion resulted unspoken dissatisfaction resultant product. decisions also often ignored later , dont mind changing decisions believe communicated completing project still paramount. feel often good sense required complete project communicate steps team members , however rarely self-assurance properly delegate tasks hold people accountable meet deadlines complete task .  experience far great though first bit anxious excited whole program , working teams challenging first still becoming less less challenging , far groups far completely different 's interesting terms knowing works n't , also helps experiencing different thinking processes ...  experiences edsa experience academy intense , interesting fun. even though met small subset people academy , great gotten know time. also particularly interesting , seeing different everyone ’ background perspective is. learning come want life chose apart academy. working lab great experience , reminds worked open plan office space feel like ’ great comfortable environment work in. also proven difficult times , slow computers noisy environment keeping focused sometimes tricky , views mountain really great ’ complain much. regards team dynamics , two groups part different terms worked , however overlap dynamics. one dynamic found two groups extroverted individuals groups took leadership role early team took cues two groups pushed leadership role. part groups introverted individuals extroverted ones. ’ major conflicts clashes dominance. first group differences personalities caused friction conflict. despite , got long worked together cohesively meet goals. natural personalities would create conflict never lost sight there. regards decision making two groups. first group lot uncertainty terms approach problems created environment aimlessness uncertainty times. second group came consensus based strengths team members would allocate roles accordingly. decisions first group made based high emotions think caused new environment learning new skills created stress , coupled trying meet deliverables made tensions high. whereas second group came decisions based logical approach. would say overall decision making rational tried deal problem statements thoughtful planned manner would allow us cope coordinate responsibilities effectively .  data science academy amazing path far , one room people ambitious love learning got excited work also supervisors facilitators determined love job motivates even keeps thrilled like one day. thankful part academy , much learn everyday proven need specific field certain level education data scientist , way academy structured students fields various level education think great idea , would see someone never studied computer science statistics knowing concept better degrees , brief say need order survive academy focused know expected everyday. fortunate enough placed teams people determined never struggled get group mate deliver expected , hence said people academy focused know for. think selection team good job placing us academy. appreciate also fact much supervisors facilitators busy , always accessible help us work passion , see getting better better day become data scientist certain end year level corporate expects be. academy coaching sessions assists come across obstacles program , work related personal problems , someone talk share predicaments going , find helpful workload much might loose track things , especially something distracting going personal life , deviate attention work , think service useful personally used load shoulders shared problems someone trust judge share problems anyone. randomly placed teams believe help us come across different personalities people learn work around , although people hoped , believe prepares us fit enough deal people meet corporate world. also learn assist patient people might struggle certain concept also learn others .  good day , really n't know start essay besides good day. anyway 's way lets start business. moments academy mostly up. let start day one meeting students south africa great , meeting people different personalities coming different background. people different races , culture tribes ( enjoy stuck cape town since day born. ) . getting first team , part crazy , fun team. guys n't look one another open , strength. first weeks everyone working well getting close end became frustrated team members started lose focus ( assume got tired sprint long ) . following team great came work bonding n't all. two members liked working home ( productive ) two quit time. came decision making , meter bouncing ideas around choosing one move stayed one teams. second team split two teams two teams thing 's believe new way things great. submitting individual team one learnt skill taught sprint. thanks .  imagine excited finally meet people like minded. maybe could rephrased meeting people like me. however , instead , meet small sample different types people different types personalities. yet find stuck world different types people different , different interest ambitions , translates make decisions every day. decisions affect , especially working teams. hand , decisions make affect team processes. according personality test , personality defined five letters entp-t , according classification debater. true debater , convinced , play along suck test discuss personality affected general experience academy. lab allowed us access well collaborate , although working effectively lab challenge , many distractions. found easier get know people type strengths , knowledge background passions general. type knowledge allows efficient knowing ask help different things. prefer asking facilitator. always done past prefer working challenge peer , along process learn would otherwise. comes working teams , always , first one speak give opinion came variety ideas , ’ extroverted side me. therefore , end leading , team. role prefer police people , therefore either allowed people lead would make sure project requirements completed end. last sprint dedicate responsibilities according people ’ backgrounds strengths interests. one person leading deliverable project end would ensure requirements submission met. employed similar strategies past , working tangibly people bring table. framed decision making teams worked in. someone shows work work have. approach made easier focus objective deliver projects. decision making rational time little emotive times , irrational judgements employed make assumption people attitudes toward project work. creates barrier people pulling weight past immediately , integrate teams later. however team dynamics forced finisher .  experience academy far interesting 4 months although apprehensive meeting work new people , still enjoyed work looking forward comes next. group work team dynamics particular challenge although ’ really socialised colleagues since arrived cape town , friendships formed mostly students live house me. team dynamics challenge fundamentals sprint ’ fair division responsibility well accountability team. group members would slack part hard hold anyone accountable concerned nice. decision-making group fundamentals sprint mixture rational emotional , things right things badly one wanted take responsibility , much passing buck. team also beset laziness unwillingness really get learn needed deliver project. rsa stock market sprint , group somewhat better people hard-workers divided responsibilities early based perceived strengths individual. really serious problem group communication different areas functional responsibility , clear group members prepared presentation fully understand analysis analysis team performed. decision-making group rational. working labs , nerve-wracking experience significant proportion people simply go socialise distracting us , prefer working environment absolute silence ( hence prefer working midnight ) simply think problem allow creative possible solving problem. people assist thought-processes terms thinking problem solutions irritating distraction. around many people long eventually lose focus problem focus people , would much better silence mandated enforced labs. also need individual work instead group-work. suggests ways could contribute group signs learnt lessons emerged previous group , still lot trepidation comes next. hope meet group members committed project , otherwise , everything end tears .  enjoy meeting new people different walks life constant , everyday social interaction. slightly reserved comes approaching people much older me. also fear inadequate unqualified many people already worked professional environments qualification , undergraduate otherwise. large emphasis group work big learning curve always preferred work alone everything way want , although improved since time here. tend take charge unless one else better talking people helping find role. versatile n't mind taking least desired role despite preferences. one group decisive. general guideline try stick although hardly case. prefer versatile approach work. rather consistent method making decisions. little less organised group though. tend procrastinate make major decisions complete important tasks last minute. tend sway easily depending emotions. know let emotions run ahead ( since something known ) put foot , take step back examine things rational perspective. hard easy put emotions beside .  honesty enjoying time regret choices made. really educational love structure , environment one force work push much facilitator really helpful bounce ideas see far able explore imagination. meeting new people 's always nice stories people far go life , combine different cultural group things got learn people 's culture stories tell. work together 's really tricky fun times tricky , matter many times think always tricky talk wont enough words left. making decisions 's simple everyone agreeing goals deadline arrived people stories , 'm unable relate excuses , honestly wish everyone could adhere words , really would like decisions made like explore lot 'm working something due time constrain sometimes 's best expected enough time explore more. team members sensitive way sensitive leave part there. love academy wish time learn bit , enough resources even conduct maybe research , instance know write good machine learning algorithms n't understand work might even able improve .  life academy. wonderful experience , joyous full many possibilities. 22nd january 2018 took tax kraainfontein heading woodstock , place never , told far capetown cbd. exact location going woodstock exchange building 's explore data science academy located. help asking people taxi help gps able find building got around 10:30 , yes late got presentation room aidan ( supervisor ) giving overview academy expected us ( student ) . room full 100 students different provinces , races backgrounds. listened long bearded white guy speaking eloquent profound right. one thing struck intention emphasis speech intelligent ( students ) apparently 8000 people applied program , 3000 people able finish whole selection process 250 people interviewed top 100 made program. aidan 's words full passion compassionate , inspiring encourages , gave birth new hope life knew labelled people 'failure ' knew opportunity beginning great journey ahead. promised things different time , work hard play less , would make friends pray give thanks heavenly father. week later ( students ) split group five people work capetown water crisis using skill learnt past week i.e sql , python power bi. group name bean bags , group four males one female , embark project , use divide conquer approach help assign work member , group members wonderful people work , jelled much fun together . bean bags often worked together , using one computer helped tasks assigned member. worked together two months present findings regarding capetown water crisis solutions suggestions sprint came end. split south african stock market project , working different people adjust new team went name stockoverflow , bit nervous previous team already made friends working together forged friendship amongst adjust. stockoverflow team three males two females , sprint month much done. learning different models predict stock prices fascinating challenging me. used divide conquer approach assigning tasks member consolidating member done google drive , 26th april presented model predictions stock price one week break came back another sprint. realized short space time learned many many skills apply solve real world problems. wonderful experience grateful .  time academy positive. 've unpleasant interactions person far n't believe 've cultivated many deep relationships aside perhaps one. two groups 've assigned socially pleasant. first group n't ideally organised second group better regard. n't particularly assertive presence first group unlike second positive dynamic present both. prefer make decisions group try best pull weight burden .  starting academy wonderful experience. cape town beautiful city location academy could better. meeting different characters really experience.in first weeks meeting new people daily basis sometimes intimidating. teamwork never really strongest skill always something would like improve upon , heard academy strong teamwork emphasis happy.the academy working lab strong social element , even though work learning somewhat technical. first sprint , quickly discovered diverse group was. vastly differing amounts experience fresh high-school graduate someone working decade. also differing skill sets. working groups challenging rewarding time. cape town sprint primary downfall lack planning what. resulted us almost late deadlines. communication within group could lot better. tend keep flexible approach completing tasks , flexibility ended costing establish proper deadlines .  four months since decided embark journey. packed bag , moved new city took first step living best moments life. love meeting new souls. always fascinating finding someone else ’ journey began , similar different footsteps left are. amongst 99 brilliant minds incredible , getting know refreshing. people stand , everyone unique. always good working , taking reins. ability work effectively team acquired skill , understand different personalities , people deal problems communicate. delegating seemed help terms getting work done teams , person taking piece , sure finish elephants. decision-making process structured , benefited whole team. guidelines rubric supervisors helped keep us right track. decision-making mostly based impulse , following process ideal getting results. great team members pay attention detail since mostly focus bigger picture. made sure work appealing. quite experience , constant learning. growing skill set , including soft skills. beginning , lot learn .  choosing spend year explore data science academy could easily best choice ever made. choice renewed feeling purpose educational/occupational direction. learnt past 3 months thought would entire year grateful presented opportunity. fellow students edsa staff added excellent experience encountered thus far. interacting brilliant minds results one , inevitably , attempting better experienced first hand. excited path edsa setting look forward future holds prospective data scientist : )  experience academy really mixed bag far. initially `` '' whatever decision making process needed order get ahead , new group , 've participate others made feel `` follower '' leader times. 's really nice working people myriad personalities. n't really think 've come across situation tempered emotionally although think quite emotional person. think far decision making way prefer ; logical straightforward also room change improve personal knowledge  find experiences academy interesting fulfilling. really like working smart people explore gives abundance. personality clashes character development hurdles point need growth adjustment. one real challenges find balance zoning focus work teaming discuss ideas others help others. choose start would rather acted like focused introvert would give time learn without harming learning curve. honestly contributed negatively group members know , well , deliver more. working together lab thing made real progress growth us individuals. crucial team decisions easier make. working lab also downsides feel still take full advantage privilege. missed couple key submissions already , messed two tests failed badly. made aware team dynamics past negatively affected catch-up work cost team process. honestly think hard also says high expectations myself. decision-making process usually one based voting. people seen getting things knowledgeable automatically obtain voting power. past experience work usually drive person chooses group role willing play. future , would prefer , rules engagement given thorough attention prior commencement group work. way , best obtained everyone. personally believe need place comfortable enough work done tools available. accomplished. achieve contribute .  mixed emotions went came cape town.i felt ecstatic opportunity afforded yet side intimidated new environment. walked building first day heart filled much anticipation could wait shake everybody 's hand introduce myself. meeting talented young old bright sparks . never worked team came academy , know working team often many times worked alone everything myself. changed , teamed bright minds actually got work . really surprised much effective efficient working team. really enjoyed working team. believe travelling meeting new people potential widen ones horizons. learnt lot still learning lot different people academy , academic related lesson also life lesson carry ever go. wonderful fruitful time previous team , though ups downs .i learnt n't leave issues group grow rather need tackle emerge .  positive experience academy far. enjoyed meeting people interest data science also made unusual choice learning institution. fascinating getting know people class. people incredibly wide range personalities backgrounds. generally enjoyed working lab though get rowdy lunch people 's work motivation decreases motivation socialise increases. found first team members rather like-able , felt team dynamic failed dismally properly tackling project. success beginning ultimately disappointed team ’ performance. lacked cohesiveness n't project manage effectively. people focused exclusively individual learning rather group work. second group considerably better. clear strategy allocated tasks effectively. regular meetings , took minutes , wrote tasks sub deadlines. member motivated pulled weight. also communicated frequently balanced individual group tasks much better previous group. decision making initial group rational however rarely executed decisions intended. lot procrastination members arrived late class left early. one group member spent time away group chatting friends. claimed working group tasks , ended contributing absolutely nothing group tasks. contrast , individual second group went beyond call duty. regular meetings ensured tasks divided properly. progress closely monitored , problem occurred , rallied together solve adapt strategy. one member group already quite experienced particular topic project tremendous advantage us. thought carefully team workflow set appropriate software working folders. analysed project marking rubric created gantt chart schedule tasks , understand dependencies. also manage group member availability member group required periods absence project. also scheduled tasks around class tests aware time allocated individual efforts. initial group unhealthy diffusion responsibility member assumed someone else would pick slack. lead group members becoming despondent refusing carry group members. also faced issue one group member ’ feel sufficient programming skills make decent contribution group effort. despite shortcomings group got reasonable marks. credit go two five group members. sure second group get good marks. credit also given equally group members one member particularly strong. however think efforts equal previous group everyone felt satisfied group effort end .  bit loner times id say experience really taught engage people talk especially student kind goal. based academic level think got experience programming fact familiar sql python shows grown academic level. knowing wake everyday see new faces lab thought able manage knowing loner told working teams thought end much talker people thought going express opinions , participate happened best experience even started releasing need talk order opinion heard. last team really understood much teams challenges valued opinions decision making part even pimplier , first team agreed everybody gon na take part everything look strength weaknesses. assign one another activities concerning project still forgetting help strengthen academy weaknesses basically created fridays helping other. comes decision coming acadamy one hard decisions take cape town far home choose completing honors degree academy. ended choosing academy honors degree felt like needed experience data science marketing world academy would best place start fact ought learn everything beginning great computing skills wanted stipend also contributed decision coming here. would conclude regret coming instead happy here.its 5 months already feel like year going bit faster want even programming skills think would got much experience programming languages skills honors.the experience got far best. helped growing academically also helped gain confidence individual .  first day first looked familiar faces. started looking people would like associate self with. meeting new people fine long n't expect keep conversation going. found nice , guys hang with. love working together labs , could share idea 's , findings ways certain things. enjoy working teams. believe 2 minds better one. also another line validity checking. one also learns lot new things working groups. team dynamics like , everyone knows need achieve bigger picture. long term though ones ideas change clearer picture , completed task. ( problems change unforeseen problems come up. ) decisions made done still needs happen. times felt team members know exactly needed next ( floated around ) . decision making based possible without overworking everyone .  enjoyed time edsa far. love meeting new people first time life talk people face-to-face programming topics around data science. previously strangers internet. great discussing algorithms coding challenges geeking people understand contribute , rather watching people 's eyes gloss describe trying achieve ( laymans terms ) . love prescribed uniform everyone comfortable naturally ease , opposed fashion parade. countless times struck conversation work , weekend even mountain. sometimes team members get distracted abit long guests talk abit loud , n't big thing. biggest challenge balancing studying implementation theory everyone page. team suffers individuals pulling weight put strain rest. luckily found colleagues kick around ideas discuss solutions current problems. decision making process voting best way forward. typically consensus reached roles , typically 2 3 members end bulk work. conflict resentment arise far real flare ups encountered , hard individuals argue decision made present put work. tend keep peace expect everyone act respectable adults. hate micromanagement love freedom given solve problem statement. glad able view teams ' presentations see went solving problem. continue learn lot data science well working time time constraint seeing people manage. even workload others offer time assist need help  experience edsa one sparked lot introspection. socially , made affirm fact extroverted person cherishes human connection , communication cares 's well-being. regards team-work , made realize quite assertive prefer logical structure comes completing tasks fulfilling set roles. made also realize end taking charge situation group , even n't initially bargained , mainly seems like n't much direction group time seems going waste. decision- making process rational one mostly times based emotion people would choose certain roles based believing good certain thing interest end n't end achieving/delivering much end burning out. 've also realized things n't go according plan comes activities part role , stress feel defeated easily perfectionist side creeps end completing needs done time. future teams wish branch attempt master every different skill best ability instead focusing much individual role , lot potential know capable ."


---
#### Unique words

In [153]:
# remove punct
rest_txt = (FtXt + TtXt + StXt + NtXt + EtXt + ItXt + JtXt + PtXt).split()

In [154]:
rest_set = set(rest_txt)

In [176]:
I_words = [word for word in set(ItXt.split()).difference(set(EtXt.split()))]

In [177]:
len(I_words)

575

In [179]:
I_words;

In [182]:
from collections import Counter

In [239]:
F_words = Counter(FtXt.split())

In [192]:
dict(I_words);

In [240]:
T_words = Counter(TtXt.split())

In [197]:
 a = dict(E_words)

TypeError: 'Counter' object is not callable

In [181]:
def dict_diff(dict_a, dict_b):
    return dict([
        (key, dict_b.get(key, dict_a.get(key)))
        for key in set(dict_a.keys()+dict_b.keys())
        if (
            (key in dict_a and (not key in dict_b or dict_a[key] != dict_b[key])) or
            (key in dict_b and (not key in dict_a or dict_a[key] != dict_b[key]))
        )
    ])

In [228]:
def dict_diff(d1, d2, NO_KEY='<KEYNOTFOUND>'):
    both = d1.keys() & d2.keys()
    diff = {k:(int(d1[k]), int(d2[k])) for k in both if d1[k] != d2[k]}
    diff.update({k:(d1[k], NO_KEY) for k in d1.keys() - both})
    diff.update({k:(NO_KEY, d2[k]) for k in d2.keys() - both})
    return diff

In [225]:
def dict_diff_none2(d1, d2, NO_KEY='<KEYNOTFOUND>'):
    both = d1.keys() & d2.keys()
    diff = {}
    diff.update({k:(d1[k]) for k in d1.keys() - both})
    #diff.update({k:(NO_KEY, d2[k]) for k in d2.keys() - both})
    return diff

In [241]:
FdiffT = dict_diff(F_words, T_words)

In [202]:
EdiffI = dict_diff(E_words, I_words)

In [210]:
testI = dict_diff_none2(I_words, E_words)

In [218]:
testE = dict_diff_none2( E_words, I_words)

In [214]:
testI

{"'": 2,
 "'logical": 1,
 "'rational": 1,
 '...': 2,
 '.today': 1,
 '1.': 2,
 '100+': 1,
 '2.': 2,
 '22nd': 2,
 '3.': 2,
 '4.': 2,
 '5.': 1,
 '7': 1,
 '7:00.': 1,
 '8th': 1,
 'accepting': 1,
 'accuracy': 1,
 'accurate': 1,
 'act': 1,
 'add': 2,
 'advanced': 2,
 'adventure': 1,
 'agrees': 1,
 'aiming': 1,
 'allot': 2,
 'allowances': 1,
 'alter': 1,
 'amplify': 1,
 'analysing': 1,
 'another.': 1,
 'anticipated': 2,
 'apart': 1,
 'apparently': 1,
 'appealing': 1,
 'approach.': 1,
 'area': 1,
 'around.': 1,
 'arrived.': 1,
 'assertive.': 1,
 'assigned.': 1,
 'attain': 1,
 'attributes': 1,
 'avenues': 1,
 'avoided': 1,
 'awful': 1,
 'backseat': 1,
 'bad': 2,
 'base': 1,
 'bcx': 1,
 'beach': 1,
 'been.': 1,
 'begin': 1,
 'behavior': 1,
 'believed': 1,
 'beneficial': 1,
 'bioinformatics': 1,
 'book.': 1,
 'brainstorming': 1,
 'brand': 1,
 'breakout': 1,
 'bulk': 1,
 'bustle': 1,
 'by.': 1,
 'called': 1,
 'camaraderie': 1,
 'can.': 1,
 'canal': 1,
 'capability': 1,
 'capetown': 1,
 'cars': 1,


In [219]:
topIuniq = sorted(testI.items(), key=operator.itemgetter(1), reverse=True)

In [220]:
topEuniq = sorted(testE.items(), key=operator.itemgetter(1), reverse=True)

In [238]:
from operator import itemgetter 

sorted(IdiffE.items(), key=operator.itemgetter(1), reverse=True)

TypeError: '<' not supported between instances of 'int' and 'str'

In [246]:
IdiffE;

In [167]:
len(rest_set)

3139

In [146]:
types_txt = PtXt.split()

In [147]:
types_set = set(types_txt)

In [155]:
print(set(types_txt).difference(rest_set))

set()


In [142]:
diff_set

set()

In [127]:
len(set(rest_txt))

3139

In [ ]:
# Done:  PtXt + 

 ---

In [106]:
from collections import Counter

Allcounts = Counter(All_txt.split())
#Pcounts = dict(Pcounts)

In [107]:
sorted_All = sorted(Allcounts.items(), key=lambda x: x[1], reverse=True)

In [68]:
#Pcounts = dict(Pcounts)

top words per type

In [108]:
sorted_All

[(',', 4333),
 ('team', 1340),
 ('people', 1075),
 ('work', 752),
 ('group', 725),
 ('working', 709),
 ('’', 652),
 ('academy', 602),
 ('different', 537),
 ('one', 487),
 ('.', 482),
 ('experience', 468),
 ('new', 425),
 ('time', 412),
 ('teams', 396),
 ('data', 369),
 ('like', 347),
 ('would', 345),
 ('also', 324),
 (')', 319),
 ('members', 317),
 ('(', 316),
 ('decisions', 299),
 ('making', 288),
 ('first', 286),
 ("n't", 285),
 ('lot', 274),
 ('things', 259),
 ('everyone', 253),
 ('learn', 246),
 ('know', 246),
 ('way', 246),
 ('far', 240),
 ('decision', 239),
 ('project', 237),
 ('get', 235),
 ('really', 233),
 ('best', 232),
 ('made', 229),
 ('learning', 222),
 ('much', 220),
 ('think', 212),
 ('meeting', 211),
 ("'s", 210),
 ('feel', 208),
 ('science', 207),
 ('skills', 197),
 ('us', 192),
 ('make', 186),
 ('good', 185),
 ('especially', 183),
 ('great', 180),
 ('edsa', 180),
 ('learned', 178),
 ('however', 175),
 ('always', 175),
 ('member', 173),
 ('lab', 170),
 ('sprint', 165),

In [104]:
sorted_J

[(',', 978),
 ('team', 296),
 ('people', 206),
 ('’', 166),
 ('work', 154),
 ('working', 130),
 ('academy', 130),
 ('one', 126),
 ('different', 118),
 ('group', 110),
 ('experience', 104),
 ('new', 98),
 ('data', 96),
 ('.', 94),
 ('time', 88),
 ('would', 86),
 ('teams', 84),
 ('(', 76),
 (')', 76),
 ('get', 72),
 ('also', 70),
 ('first', 68),
 ('everyone', 68),
 ('like', 68),
 ('members', 62),
 ('making', 60),
 ('things', 58),
 ('decisions', 56),
 ('way', 56),
 ('much', 56),
 ('project', 54),
 ('science', 54),
 ('best', 52),
 ('lot', 52),
 ('us', 52),
 ('however', 52),
 ('made', 50),
 ('far', 50),
 ('edsa', 48),
 ('learning', 48),
 ('learn', 48),
 ("n't", 48),
 ('always', 46),
 ('learned', 46),
 ('know', 44),
 ('decision', 44),
 ('think', 44),
 ('skills', 44),
 ('make', 42),
 ('especially', 42),
 ('well', 40),
 ('even', 40),
 ('something', 40),
 ('feel', 40),
 ('worked', 38),
 ('take', 38),
 ('good', 38),
 ('come', 38),
 ("'s", 36),
 ('felt', 36),
 ('could', 36),
 ('great', 36),
 ('bi

In [101]:
sorted_I

[(',', 489),
 ('team', 148),
 ('people', 103),
 ('’', 83),
 ('work', 77),
 ('working', 65),
 ('academy', 65),
 ('one', 63),
 ('different', 59),
 ('group', 55),
 ('experience', 52),
 ('new', 49),
 ('data', 48),
 ('.', 47),
 ('time', 44),
 ('would', 43),
 ('teams', 42),
 ('(', 38),
 (')', 38),
 ('get', 36),
 ('also', 35),
 ('first', 34),
 ('everyone', 34),
 ('like', 34),
 ('members', 31),
 ('making', 30),
 ('things', 29),
 ('decisions', 28),
 ('way', 28),
 ('much', 28),
 ('project', 27),
 ('science', 27),
 ('best', 26),
 ('lot', 26),
 ('us', 26),
 ('however', 26),
 ('made', 25),
 ('far', 25),
 ('edsa', 24),
 ('learning', 24),
 ('learn', 24),
 ("n't", 24),
 ('always', 23),
 ('learned', 23),
 ('know', 22),
 ('decision', 22),
 ('think', 22),
 ('skills', 22),
 ('make', 21),
 ('especially', 21),
 ('well', 20),
 ('even', 20),
 ('something', 20),
 ('feel', 20),
 ('worked', 19),
 ('take', 19),
 ('good', 19),
 ('come', 19),
 ("'s", 18),
 ('felt', 18),
 ('could', 18),
 ('great', 18),
 ('bit', 18),

In [98]:
sorted_E

[(',', 460),
 ('team', 142),
 ('people', 132),
 ('working', 87),
 ('group', 81),
 ('work', 80),
 ('academy', 69),
 ('’', 69),
 ('one', 66),
 ('different', 61),
 ('also', 55),
 ('.', 53),
 ('experience', 52),
 ('teams', 51),
 ('time', 50),
 ('would', 46),
 ('like', 43),
 ('new', 42),
 ('members', 41),
 ('get', 39),
 ('everyone', 38),
 ('made', 38),
 ('first', 37),
 ('decisions', 35),
 ("n't", 35),
 ('much', 35),
 ('lot', 35),
 ('(', 34),
 ('data', 33),
 (')', 33),
 ('making', 33),
 ('know', 33),
 ('way', 30),
 ('things', 30),
 ('always', 29),
 ('learn', 29),
 ('far', 29),
 ("'s", 26),
 ('great', 26),
 ('decision', 26),
 ('meeting', 25),
 ('learning', 25),
 ('think', 25),
 ('really', 24),
 ('even', 23),
 ('project', 23),
 ('us', 23),
 ('however', 22),
 ('skills', 22),
 ('member', 22),
 ('could', 21),
 ('found', 21),
 ('best', 21),
 ('good', 20),
 ('help', 20),
 ('got', 20),
 ('science', 19),
 ('felt', 19),
 ('sprint', 19),
 ('especially', 19),
 ('better', 19),
 ('feel', 19),
 ('since', 1

In [95]:
sorted_N

[(',', 460),
 ('team', 142),
 ('people', 132),
 ('working', 87),
 ('group', 81),
 ('work', 80),
 ('academy', 69),
 ('’', 69),
 ('one', 66),
 ('different', 61),
 ('also', 55),
 ('.', 53),
 ('experience', 52),
 ('teams', 51),
 ('time', 50),
 ('would', 46),
 ('like', 43),
 ('new', 42),
 ('members', 41),
 ('get', 39),
 ('everyone', 38),
 ('made', 38),
 ('first', 37),
 ('decisions', 35),
 ("n't", 35),
 ('much', 35),
 ('lot', 35),
 ('(', 34),
 ('data', 33),
 (')', 33),
 ('making', 33),
 ('know', 33),
 ('way', 30),
 ('things', 30),
 ('always', 29),
 ('learn', 29),
 ('far', 29),
 ("'s", 26),
 ('great', 26),
 ('decision', 26),
 ('meeting', 25),
 ('learning', 25),
 ('think', 25),
 ('really', 24),
 ('even', 23),
 ('project', 23),
 ('us', 23),
 ('however', 22),
 ('skills', 22),
 ('member', 22),
 ('could', 21),
 ('found', 21),
 ('best', 21),
 ('good', 20),
 ('help', 20),
 ('got', 20),
 ('science', 19),
 ('felt', 19),
 ('sprint', 19),
 ('especially', 19),
 ('better', 19),
 ('feel', 19),
 ('since', 1

In [92]:
sorted_S

[(',', 253),
 ('team', 80),
 ('people', 50),
 ('work', 47),
 ('group', 46),
 ('new', 33),
 ('one', 32),
 ('academy', 31),
 ('.', 29),
 ('would', 28),
 ('data', 27),
 ('working', 27),
 ('different', 27),
 ('experience', 26),
 ('’', 26),
 ('first', 24),
 ('edsa', 23),
 (')', 21),
 ('time', 20),
 ('(', 19),
 ('best', 16),
 ('project', 16),
 ('science', 16),
 ('like', 15),
 ('us', 15),
 ('decisions', 15),
 ('decision', 15),
 ('learning', 15),
 ("n't", 15),
 ('everyone', 14),
 ('also', 14),
 ('members', 14),
 ('making', 14),
 ('feel', 14),
 ('really', 13),
 ('lot', 13),
 ('made', 13),
 ('however', 13),
 ('think', 13),
 ('--', 12),
 ('well', 12),
 ('lab', 12),
 ('worked', 12),
 ('make', 12),
 ('much', 12),
 ('many', 11),
 ('meeting', 11),
 ("'s", 11),
 ('teams', 11),
 ('take', 11),
 ('far', 11),
 ('something', 11),
 ('could', 10),
 ('bit', 10),
 ('learn', 10),
 ('sprint', 10),
 ('still', 10),
 ('way', 10),
 ('year', 10),
 ('end', 10),
 ('day', 10),
 ('students', 10),
 ('felt', 9),
 ('good', 

In [89]:
sorted_T

[(',', 808),
 ('team', 252),
 ('people', 214),
 ('working', 154),
 ('group', 154),
 ('work', 142),
 ('’', 134),
 ('academy', 114),
 ('different', 102),
 ('.', 94),
 ('experience', 86),
 ('teams', 84),
 ('time', 76),
 ('like', 70),
 ('new', 68),
 ('one', 65),
 ('data', 62),
 ('decisions', 62),
 ('members', 60),
 ('really', 60),
 ('making', 58),
 ("n't", 58),
 ('know', 56),
 ('things', 56),
 ('(', 54),
 (')', 54),
 ('learn', 54),
 ('lot', 52),
 ('decision', 48),
 ('also', 47),
 ('meeting', 46),
 ('best', 46),
 ('far', 46),
 ('way', 46),
 ('feel', 46),
 ('project', 44),
 ("'s", 42),
 ('skills', 42),
 ('good', 40),
 ('learned', 38),
 ('especially', 38),
 ('would', 37),
 ('lab', 36),
 ('make', 36),
 ('learning', 36),
 ('think', 36),
 ('first', 34),
 ("'ve", 34),
 ('come', 34),
 ('tasks', 34),
 ('second', 32),
 ('got', 32),
 ('help', 32),
 ('getting', 32),
 ('bit', 32),
 ('member', 32),
 ('science', 32),
 ('personalities', 32),
 ('sprint', 32),
 ('prefer', 32),
 ('-', 30),
 ('end', 30),
 ('t

In [80]:
sorted_F

[(',', 313),
 ('team', 96),
 ('people', 77),
 ('work', 60),
 ('group', 53),
 ('academy', 44),
 ('working', 43),
 ('new', 41),
 ('different', 39),
 ('would', 37),
 ('experience', 36),
 ('.', 36),
 ('one', 35),
 ('time', 32),
 ('data', 29),
 ('first', 28),
 ('’', 28),
 (')', 27),
 ('(', 26),
 ('members', 25),
 ('made', 25),
 ('much', 24),
 ('everyone', 23),
 ('like', 23),
 ('learning', 22),
 ('also', 22),
 ('lot', 22),
 ("n't", 21),
 ('great', 20),
 ('think', 20),
 ('teams', 20),
 ('science', 19),
 ('however', 19),
 ('decisions', 19),
 ('worked', 18),
 ('making', 18),
 ('way', 18),
 ('far', 18),
 ('get', 17),
 ('decision', 17),
 ('project', 17),
 ("'s", 16),
 ('found', 16),
 ('two', 16),
 ('edsa', 16),
 ('always', 15),
 ('felt', 15),
 ('sprint', 15),
 ('could', 15),
 ('member', 15),
 ('best', 14),
 ('still', 14),
 ('able', 14),
 ('got', 14),
 ('us', 14),
 ('meeting', 13),
 ('life', 13),
 ('year', 13),
 ('well', 13),
 ('take', 13),
 ('approach', 13),
 ('groups', 13),
 ('students', 12),
 (

In [77]:
sorted_P

[(',', 224),
 ('people', 79),
 ('team', 74),
 ('group', 72),
 ('work', 50),
 ('working', 49),
 ('academy', 35),
 ('.', 35),
 ('one', 35),
 ('also', 34),
 ('would', 31),
 ('different', 29),
 ('first', 27),
 ('experience', 26),
 ('made', 26),
 ('new', 26),
 ('time', 26),
 ("n't", 26),
 ('members', 24),
 ('like', 24),
 ('lot', 22),
 ('decisions', 22),
 ('really', 21),
 ('teams', 20),
 ("'s", 19),
 ('meeting', 19),
 ('decision', 19),
 ('much', 19),
 ('everyone', 18),
 ('know', 17),
 ('making', 17),
 ('great', 16),
 ('tasks', 16),
 ('learning', 16),
 (')', 16),
 ('think', 16),
 ('far', 15),
 ('learn', 15),
 ('got', 15),
 ('member', 15),
 ('sprint', 15),
 ('two', 15),
 ('(', 15),
 ('end', 14),
 ('could', 13),
 ('always', 13),
 ('feel', 13),
 ('together', 12),
 ('lab', 12),
 ('data', 12),
 ('help', 12),
 ('better', 12),
 ('many', 12),
 ('’', 12),
 ('way', 12),
 ('found', 12),
 ('based', 12),
 ('us', 12),
 ('project', 12),
 ('best', 11),
 ('believe', 11),
 ('prefer', 11),
 ('need', 11),
 ('sti

In [55]:
def all_the_words(words):
    global TtXt
    txt=" "
    for word in words:
        a = "".join(word)
        txt += ' '+a
    TtXt += txt
    return txt

In [56]:
Etxt = T.apply(all_the_words)

In [59]:
TtXt

"  first 4 months edsa filled many new experiences ; challenges , others joyful , resulted positive growth person. -- -meeting new people -- - enjoy meeting new people everyone met edsa pleasant. particularly enjoyed listening backgrounds strongest candidates within edsa , provided insight gained experience tools like python & sql. also cherish able build network data scientists tomorrow well established industry professionals.. -- -working together lab -- - working computer lab formal work environment comes pros & cons. 's useful 99 colleagues , supervisors & facilitators provide technical/personal support net necessary. obvious con would noise distraction - least say know trending music albums 2018 , win ! -- -working teams & team dynamics -- - 've worked team projects solo projects professional work environment. enjoy both. experience would say team works best 's team co-ordinator plan work divided conquered ( provided everybody takes full responsibility part ) members first group n

In [368]:
len(TtXt)

51460

In [230]:
all_mbti.head()

,I/E,N/S,T/F,J/P,tokens
0,I,S,T,J,"[first, 4, months, edsa, filled, many, new, ex..."
1,I,N,F,J,"[joined, academy, crossroads, sorts, life., ac..."
2,E,N,F,J,"[far, experience, positive, definitely, see, v..."
3,I,N,F,J,"[fortunate, opportunity, join, academy, year.,..."
4,I,N,T,J,"[looking, back, one, got, academy, right, ,, c..."


In [231]:
all_mbti['type']= all_mbti['I/E']+all_mbti['N/S']+all_mbti['T/F']+all_mbti['J/P']

In [232]:
all_mbti.head()

,I/E,N/S,T/F,J/P,tokens,type
0,I,S,T,J,"[first, 4, months, edsa, filled, many, new, ex...",ISTJ
1,I,N,F,J,"[joined, academy, crossroads, sorts, life., ac...",INFJ
2,E,N,F,J,"[far, experience, positive, definitely, see, v...",ENFJ
3,I,N,F,J,"[fortunate, opportunity, join, academy, year.,...",INFJ
4,I,N,T,J,"[looking, back, one, got, academy, right, ,, c...",INTJ


In [5]:
all_mbti['type'].value_counts()

NameError: name 'all_mbti' is not defined

---
MBTI | amount | %EDSApop | %GENpop
--- | --- | --- | ---
INTJ   | 12 |17.9| 2.1  |
INTP   | 11 |16.4| 3.3  |
INFJ   |  8 |11.9| 1.5  |
ENFP   |  4 |6| 8.1  |
ENFJ   |  4 |6| 2.5  |
ISFP   |  4 |6| 1.5  |
ISTJ   |  4 |6| 11.6 |
ESFJ   |  3 |4.5| 12.3 |
INFP   |  3 |4.5| 4.4  |
ISTP   |  2 |3| 5.4  |
ENTP   |  2 |3| 3.2  |
ESFP   |  2 |3| 8.5  |
ESTJ   |  2 |3| 8.7  |
ISFJ   |  2 |3| 13.8 |
E???   |  1 |1.5|
ESTP   |  1 |1.5| 4.3  |
I?T?   |  1 |1.5|
ENTJ   |  1 |1.5| 1.8  |

In [233]:
# here we create a set of dictionaries
# one for each of the MBTI types
personality = {}
for pp in type_labels:
    df = all_mbti.groupby('type')
    personality[pp] = {}
    for row in df.get_group(pp)['tokens']:
        personality[pp] = bag_of_words_count(row, personality[pp])   

In [234]:
personality.keys()

dict_keys(['ISTJ', 'ISFJ', 'INFJ', 'INTJ', 'ISTP', 'ISFP', 'INFP', 'INTP', 'ESTP', 'ESFP', 'ENFP', 'ENTP', 'ESTJ', 'ESFJ', 'ENFJ', 'ENTJ'])

In [282]:
ENTJwords = sorted(personality['ENTJ'],key=personality['ENTJ'].get, reverse=True)

In [361]:
len(ENTJwords)

233

In [318]:
my_stopwords2 = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along',
                 'already', 'also', 'although', 'always', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 
                 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 
                 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being',
                 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 
                 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'do', 'done', 'down', 
                 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 
                 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'fill', 
                 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 
                 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'hasnt', 'have', 'hence', 'here', 
                 'hereafter', 'hereby', 'herein', 'hereupon', 'how', 'however', 
                 'hundred', 'ie', 'if', 'in', 'inc', 'indeed', 'interest', 'into', 'is', 'it', 'its', 'itself', 'keep', 
                 'last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made', 'many', 'may', 'meanwhile', 'might', 
                 'mill', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'name', 'namely', 
                 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'none', 'noone', 'nor', 'nothing', 
                 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 
                 'out', 'over', 'own', 'part', 'per', 'perhaps', 'please', 'put', 'rather', 're', 'same',
                 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'should', 'show', 'side', 'since', 
                 'sincere', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 
                 'still', 'such', 'system', 'take', 'ten', 'than', 'that', 'the', 'then', 'thence', 
                 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'thick', 'thin', 'third', 
                 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward', 
                 'towards', 'twelve', 'twenty', 'two', 'un', 'under', 'until', 'up', 'upon', 'us', 'very', 'via', 'was', 'well', 
                 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 
                 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 
                 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves']



In [344]:
set(INFPwords[:150]).difference(all_set) 


{'appeared',
 'belong',
 'blessing',
 'caught',
 'classmates',
 'currently',
 'curse.',
 'doing.',
 'feeling.',
 'losing',
 'minded',
 'pack.'}

In [ ]:
IE	NS	TF	JP

In [2]:
# INTPwords
INTPwords #, ISTJwords, ISFJwords, ENTJwords, ESFJwords, ESTPwords, ENFPwords

NameError: name 'INTPwords' is not defined

In [362]:
I_set = set(INTPwords + ISTJwords + INFJwords + INTJwords + ISTPwords + ISFPwords + ISFJwords + INFPwords)
E_set = set(ENFPwords + ESFJwords + ESFPwords + ESTPwords + ENTPwords + ESTJwords + ENTJwords + ENFJwords) 
my_stopwords2;

In [1]:
Tset = set(INTPwords + ISTJwords + INTJwords + ISTPwords + ESTPwords + ENTPwords + ESTJwords + ENTJwords)
Fset = set(INFJwords + ISFPwords + ISFJwords + INFPwords + ENFPwords + ESFJwords + ESFPwords + ENFJwords)

NameError: name 'INTPwords' is not defined

---
Introvert vs Extrovert

In [363]:
print(f'Number of words used by both: {len(I_set.intersection(E_set))}')

Number of words used by both: 868


In [364]:
print(f'Extrovert unique words: {len(E_set.difference(I_set))}')

Extrovert unique words: 632


In [365]:
print(f'Introvert unique words: {len(I_set.difference(E_set))}')

Introvert unique words: 1612


In [366]:
print(f'Total number of words: {len(I_set.union(E_set))}')

Total number of words: 3112


---
Thinking vs Feeling

In [ ]:
print(f'Number of words used by both: {len(I_set.intersection(E_set))}')

In [ ]:
print(f'Extrovert unique words: {len(E_set.difference(I_set))}')

In [ ]:
print(f'Introvert unique words: {len(I_set.difference(E_set))}')

In [ ]:
print(f'Total number of words: {len(I_set.union(E_set))}')